In [1]:
###IMPORT MODULES AS APPROPRIATE###

In [2]:
import nltk; nltk.download('stopwords')

C:\Users\brady\Anaconda3\lib\site-packages\sklearn\feature_extraction\image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):
C:\Users\brady\Anaconda3\lib\site-packages\sklearn\linear_model\least_angle.py:35: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

True

In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

C:\Users\brady\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
C:\Users\brady\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)
C:\Users\brady\Anaconda3\lib\site-packages\sklearn\decomposition\online_lda.py:31: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/relea

In [4]:
###INSERT STOP WORDS AS NEEDED###

In [5]:
#NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
#stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [6]:
###IMPORT DATA - THIS IS CURRENTLY UGLY BUT WHATEVER###

###try this later:
#import csv
#reviews = [row for row in csv.reader(open('reviews.csv'))]
#print reviews

In [8]:
###df = [
    "Individual animals frequently exhibit repeatable differences from other members of their population, differences now commonly referred to as 'animal personality'. Personality differences can arise, for example, from differences in permanent environmental effects-including parental and epigenetic contributors- and the effect of additive genetic variation. Although several studies have evaluated the heritability of behaviour, less is known about general patterns of heritability and additive genetic variation in animal personality. As overall variation in behaviour includes both the among-individual differences that reflect different personalities and temporary environmental effects, it is possible for personality to be largely genetically influenced even when heritability of behaviour per se is quite low. The relative contribution of additive genetic variation to personality variation can be estimated whenever both repeatability and heritability are estimated for the same data. Using published estimates to address this issue, we found that approximately 52% of animal personality variation was attributable to additive genetic variation. Thus, while the heritability of behaviour is often moderate or low, the heritability of personality is much higher. Our results therefore (i) demonstrate that genetic differences are likely to be a major contributor to variation in animal personality and (ii) support the phenotypic gambit: that evolutionary inferences drawn from repeatability estimates may often be justified.",
    "Environment has a critical role in the natural selection process for Darwinian evolution. The primary molecular component currently considered for neo-Darwinian evolution involves genetic alterations and random mutations that generate the phenotypic variation required for natural selection to act. The vast majority of environmental factors cannot directly alter DNA sequence. Epigenetic mechanisms directly regulate genetic processes and can be dramatically altered by environmental factors. Therefore, environmental epigenetics provides a molecular mechanism to directly alter phenotypic variation generationally. Lamarck proposed in 1802 the concept that environment can directly alter phenotype in a heritable manner. Environmental epigenetics and epigenetic transgenerational inheritance provide molecular mechanisms for this process. Therefore, environment can on a molecular level influence the phenotypic variation directly. The ability of environmental epigenetics to alter phenotypic and genotypic variation directly can significantly impact natural selection. Neo-Lamarckian concept can facilitate neo-Darwinian evolution. A unified theory of evolution is presented to describe the integration of environmental epigenetic and genetic aspects of evolution.",
    "DNA methylation contributes to gene and transcriptional regulation in eukaryotes, and therefore has been hypothesized to facilitate the evolution of plastic traits such as sociality in insects. However, DNA methylation is sparsely studied in insects. Therefore, we documented patterns of DNA methylation across a wide diversity of insects. We predicted that underlying enzymatic machinery is concordant with patterns of DNA methylation. Finally, given the suggestion that DNA methylation facilitated social evolution in Hymenoptera, we tested the hypothesis that the DNA methylation system will be associated with presence/absence of sociality among other insect orders. We found DNA methylation to be widespread, detected in all orders examined except Diptera (flies). Whole genome bisulfite sequencing showed that orders differed in levels of DNA methylation. Hymenopteran (ants, bees, wasps and sawflies) had some of the lowest levels, including several potential losses. Blattodea (cockroaches and termites) show all possible patterns, including a potential loss of DNA methylation in a eusocial species whereas solitary species had the highest levels. Species with DNA methylation do not always possess the typical enzymatic machinery. We identified a gene duplication event in the maintenance DNA methyltransferase 1 (DNMT1) that is shared by some Hymenoptera, and paralogs have experienced divergent, nonneutral evolution. This diversity and nonneutral evolution of underlying machinery suggests alternative DNA methylation pathways may exist. Phylogenetically corrected comparisons revealed no evidence that supports evolutionary association between sociality and DNA methylation. Future functional studies will be required to advance our understanding of DNA methylation in insects.",
    "Phenotypic characteristics of animals can change independently from changes in the genetic code. These plastic phenotypic responses are important for population persistence in changing environments. Plasticity can be induced during early development, with persistent effects on adult phenotypes, and it can occur reversibly throughout life (acclimation). These manifestations of plasticity have been viewed as separate processes. Here we argue that developmental conditions not only change mean trait values but also modify the capacity for acclimation. Acclimation counteracts the potentially negative effects of phenotype-environment mismatches resulting from epigenetic modifications during early development. Developmental plasticity is therefore also beneficial when environmental conditions change within generations. Hence, the evolution of reversible acclimation can no longer be viewed as independent from developmental processes.",
    "An organism's phenotype can be influenced by maternal cues and directly perceived environmental cues, as well as by its genotype at polymorphic loci, which can be interpreted as a genetic cue. In fluctuating environments, natural selection favors organisms that efficiently integrate different sources of information about the likely success of phenotypic alternatives. In such situations, it can be beneficial to pass on maternal cues that offspring can respond to. A maternal cue could be based on environmental cues directly perceived by the mother but also partly on cues that were passed on by the grandmother. We have used a mathematical model to investigate how the passing of maternal cues and the integration of different sources of information evolve in response to qualitatively different kinds of temporal and spatial environmental fluctuations. The model shows that the passing of maternal cues and the transgenerational integration of sources of information readily evolve. Factors such as the degree of temporal autocorrelation, the predictive accuracy of different environmental cues, and the level of gene flow strongly influence the expression of adaptive maternal cues and the relative weights given to different sources of information. We outline the main features of the relation between the characteristics of environmental fluctuations and the adaptive systems of phenotype determination and compare these predictions with empirical studies on cue integration.",
    "Understanding how organisms cope with global change is a major scientific challenge. The molecular pathways underlying rapid adaptive phenotypic responses to global change remain poorly understood. Here, we highlight the relevance of two environment-sensitive molecular elements: transposable elements (TEs) and epigenetic components (ECs). We first outline the sensitivity of these elements to global change stressors and review how they interact with each other. We then propose an integrative molecular engine coupling TEs and ECs and allowing organisms to fine-tune phenotypes in a real-time fashion, adjust the production of phenotypic and genetic variation, and produce heritable phenotypes with different levels of transmission fidelity. We finally discuss the implications of this molecular engine in the context of global change.",
    "Existing insight suggests that maternal effects have a substantial impact on evolution, yet these predictions assume that maternal effects themselves are evolutionarily constant. Hence, it is poorly understood how natural selection shapes maternal effects in different ecological circumstances. To overcome this, the current study derives an evolutionary model of maternal effects in a quantitative genetics context. In constant environments, we show that maternal effects evolve to slight negative values that result in a reduction of the phenotypic variance (canalization). By contrast, in populations experiencing abrupt change, maternal effects transiently evolve to positive values for many generations, facilitating the transmission of beneficial maternal phenotypes to offspring. In periodically fluctuating environments, maternal effects evolve according to the autocorrelation between maternal and offspring environments, favoring positive maternal effects when change is slow, and negative maternal effects when change is rapid. Generally, the strongest maternal effects occur for traits that experience very strong selection and for which plasticity is severely constrained. By contrast, for traits experiencing weak selection, phenotypic plasticity enhances the evolutionary scope of maternal effects, although maternal effects attain much smaller values throughout. As weak selection is common, finding substantial maternal influences on offspring phenotypes may be more challenging than anticipated.",
    "As climate change challenges organismal fitness by creating a phenotype-environment mismatch, phenotypic plasticity generated by epigenetic mechanisms (e.g., DNA methylation) can provide a temporal buffer for genetic adaptation. Epigenetic mechanisms may be crucial for sessile benthic marine organisms, such as reef-building corals, where ocean acidification (OA) and warming reflect in strong negative responses. We tested the potential for scleractinian corals to exhibit phenotypic plasticity associated with a change in DNA methylation in response to OA. Clonal coral fragments of the environmentally sensitive Pocillopora damicornis and more environmentally robust Montipora capitata were exposed to fluctuating ambient pH (7.9-7.65) and low pH (7.6-7.35) conditions in common garden tanks for similar to 6 weeks. M. capitata responded weakly, or acclimated more quickly, to OA, with no difference in calcification, minimal separation of metabolomic profiles, and no change in DNA methylation between treatments. Conversely, P. damicornis exhibited diminished calcification at low pH, stronger separation in metabolomic profiles, and responsiveness of DNA methylation to treatment. Our data suggest corals differ in their temporal dynamics and sensitivity for environmentally triggered real-time epigenetic reprogramming. The generation of potentially heritable plasticity via environmental induction of DNA methylation provides an avenue for assisted evolution applications in corals under rapid climate change.",
    "Migration is essential for the reproduction and survival of many animals, yet little is understood about its underlying molecular mechanisms. We used the salmonid Oncorhynchus mykiss to gain mechanistic insight into smoltification, which is a morphological, physiological and behavioural transition undertaken by juveniles in preparation for seaward migration. O. mykiss is experimentally tractable and displays intra- and interpopulation variation in migration propensity. Migratory individuals can produce nonmigratory progeny and vice versa, indicating a high degree of phenotypic plasticity. One potential way that phenotypic plasticity might be linked to variation in migration-related life history tactics is through epigenetic regulation of gene expression. To explore this, we quantitatively measured genome-scale DNA methylation in fin tissue using reduced representation bisulphite sequencing of F-2 siblings produced from a cross between steelhead (migratory) and rainbow trout (nonmigratory) lines. We identified 57 differentially methylated regions (DMRs) between smolt and resident O. mykiss juveniles. DMRs were high in magnitude, with up to 62% differential methylation between life history types, and over half of the gene-associated DMRs were in transcriptional regulatory regions. Many of the DMRs encode proteins with activity relevant to migration-related transitions (e.g. circadian rhythm pathway, nervous system development, protein kinase activity). This study provides the first evidence of a relationship between epigenetic variation and life history divergence associated with migration-related traits in any species.",
    "While traits and trait plasticity are partly genetically based, investigating epigenetic mechanisms may provide more nuanced understanding of the mechanisms underlying response to environment. Using AFLP and methylation-sensitive AFLP, we tested the hypothesis that differentiation to habitats along natural salt marsh environmental gradients occurs at epigenetic, but not genetic loci in two salt marsh perennials. We detected significant genetic and epigenetic structure among populations and among subpopulations, but we found multilocus patterns of differentiation to habitat type only in epigenetic variation for both species. In addition, more epigenetic than genetic loci were correlated with habitat in both species. When we analysed genetic and epigenetic variation simultaneously with partial Mantel, we found no correlation between genetic variation and habitat and a significant correlation between epigenetic variation and habitat in Spartina alterniflora. In Borrichia frutescens, we found significant correlations between epigenetic and/or genetic variation and habitat in four of five populations when populations were analysed individually, but there was no significant correlation between genetic or epigenetic variation and habitat when analysed jointly across the five populations. These analyses suggest that epigenetic mechanisms are involved in the response to salt marsh habitats, but also that the relationships among genetic and epigenetic variation and habitat vary by species. Site-specific conditions may also cloud our ability to detect response in replicate populations with similar environmental gradients. Future studies analysing sequence data and the correlation between genetic variation and DNA methylation will be powerful to identify the contributions of genetic and epigenetic response to environmental gradients.",
    "Epigenetic variation is being integrated into our understanding of adaptation, yet we lack models on how epigenetic mutations affect evolution that includes de novo genetic change. We model the effects of epigenetic mutations on the dynamics and endpoints of adaptive walks-a process where a series of beneficial mutations move a population towards a fitness optimum. We use an individual-based model of an asexual population, where mutational effects are drawn from Fisher's geometric model. We find cases where epigenetic mutations speed adaptation or result in populations with higher fitness. However, we also find cases where they slow adaptation or result in populations with lower fitness. The effect of epigenetic mutations on adaptive walks depends crucially on their stability and fitness effects relative to genetic mutations, with small-effect epigenetic mutations generally speeding adaptation, and epigenetic mutations with the same fitness effects as genetic mutations slowing adaptation. Our work reveals a complex relationship between epigenetic mutations and natural selection and highlights the need for empirical data.",
    "Environmental stresses experienced by individual parents can influence offspring phenotypes in ways that enhance survival under similar conditions. Although such adaptive transgenerational plasticity is well documented, its transmission mechanisms are generally unknown. One possible mechanism is environmentally induced DNA methylation changes. We tested this hypothesis in the annual plant Polygonum persicaria, a species known to express adaptive transgenerational plasticity in response to parental drought stress. Replicate plants of 12 genetic lines (sampled from natural populations) were grown in dry versus moist soil. Their offspring were exposed to the demethylating agent zebularine or to control conditions during germination and then grown in dry soil. Under control germination conditions, the offspring of drought-stressed parents grew longer root systems and attained greater biomass compared with offspring of well-watered parents of the same genetic lines. Demethylation removed these adaptive developmental effects of parental drought, but did not significantly alter phenotypic expression in offspring of well-watered parents. The effect of demethylation on the expression of the parental drought effect varied among genetic lines. Differential seed provisioning did not contribute to the effect of parental drought on offspring phenotypes. These results demonstrate that DNA methylation can mediate adaptive, genotype-specific effects of parental stress on offspring phenotypes.",
    "The role of natural selection in the evolution of adaptive phenotypes has undergone constant probing by evolutionary biologists, employing both theoretical and empirical approaches. As Darwin noted, natural selection can act together with other processes, including random changes in the frequencies of phenotypic differences that are not under strong selection, and changes in the environment, which may reflect evolutionary changes in the organisms themselves. As understanding of genetics developed after 1900, the new genetic discoveries were incorporated into evolutionary biology. The resulting general principles were summarized by Julian Huxley in his 1942 book Evolution: the modern synthesis. Here, we examine how recent advances in genetics, developmental biology and molecular biology, including epigenetics, relate to today's understanding of the evolution of adaptations. We illustrate how careful genetic studies have repeatedly shown that apparently puzzling results in a wide diversity of organisms involve processes that are consistent with neo-Darwinism. They do not support important roles in adaptation for processes such as directed mutation or the inheritance of acquired characters, and therefore no radical revision of our understanding of the mechanism of adaptive evolution is needed.",
    "The process of biological invasion exposes a species to novel pressures, in terms of both the environments it encounters and the evolutionary consequences of range expansion. Several invaders have been shown to exhibit rapid evolutionary changes in response to those pressures, thus providing robust opportunities to clarify the processes at work during rapid phenotypic transitions. The accelerating pace of invasion of cane toads (Rhinella marina) in tropical Australia during its 80-year history has been well characterized at the phenotypic level, including common-garden experiments that demonstrate heritability of several dispersal-relevant traits. Individuals from the invasion front (and their progeny) show distinctive changes in morphology, physiology and behaviour that, in combination, result in far more rapid dispersal than is true of conspecifics from long-colonized areas. The extensive body of work on cane toad ecology enables us to place into context studies of the genetic basis of these traits. Our analyses of differential gene expression from toads from both ends of this invasion-history transect reveal substantial upregulation of many genes, notably those involved in metabolism and cellular repair. Clearly, then, the dramatically rapid phenotypic evolution of cane toads in Australia has been accompanied by substantial shifts in gene expression, suggesting that this system is well suited to investigating the genetic underpinnings of invasiveness.",
    "DNA methylation in plants affects transposon silencing, transcriptional regulation and thus phenotypic variation. One unanswered question is whether DNA methylation could be involved in local adaptation of plant populations to their environments. If methylation alters phenotypes to improve plant response to the environment, then methylation sites or the genes that affect them could be a target of natural selection. Using reduced-representation bisulphite sequencing (RRBS) data, we assessed whether climate is associated with variation in DNA methylation levels among 58 naturally occurring, and species-wide samples of valley oak (Quercus lobata) collected across climate gradients. We identified the genomic context of these variants referencing a new draft valley oak genome sequence. Methylation data were obtained for 341 107 cytosines, of which we deemed 57 488 as single-methylation variants (SMVs), found in the CG, CHG and CHH sequence contexts. Environmental association analyses revealed 43 specific SMVs that are significantly associated with any of four climate variables, the majority of which are associated with mean maximum temperature. The 43 climate-associated SMVs tend to occur in or near genes, several of which have known involvement in plant response to environment. Multivariate analyses show that climate and spatial variables explain more overall variance in CG-SMVs among individuals than in SNPs, CHG-SMVs or CHH-SMVs. Together, these results from natural oak populations provide initial evidence for a role of CG methylation in locally adaptive evolution or plasticity in plant response.",
    "Comparative genomics of social insects has been intensely pursued in recent years with the goal of providing insights into the evolution of social behaviour and its underlying genomic and epigenomic basis. However, the comparative approach has been hampered by a paucity of data on some of the most informative social forms (e.g. incipiently and primitively social) and taxa (especially members of the wasp family Vespidae) for studying social evolution. Here, we provide a draft genome of the primitively eusocial model insect Polistes dominula, accompanied by analysis of caste-related transcriptome and methylome sequence data for adult queens and workers. Polistes dominula possesses a fairly typical hymenopteran genome, but shows very low genomewide GC content and some evidence of reduced genome size. We found numerous caste-related differences in gene expression, with evidence that both conserved and novel genes are related to caste differences. Most strikingly, these -omics data reveal a major reduction in one of the major epigenetic mechanisms that has been previously suggested to be important for caste differences in social insects: DNA methylation. Along with a conspicuous loss of a key gene associated with environmentally responsive DNA methylation (the de novo DNA methyltransferase Dnmt3), these wasps have greatly reduced genomewide methylation to almost zero. In addition to providing a valuable resource for comparative analysis of social insect evolution, our integrative -omics data for this important behavioural and evolutionary model system call into question the general importance of DNA methylation in caste differences and evolution in social insects.",
    "Epidemiological research since the 1980s has highlighted the consequences of early life adversity, particularly during gestation and early infancy, for adult health (the Barker hypothesis). The fast-evolving field of molecular epigenetics is providing explanatory mechanisms concerning phenotypic plasticity in response to developmental stressors and the accumulation of disease risk throughout life. In addition, there is now evidence for the heritability of poor health across generations via epigenetic modifications. This research has the potential to invoke a paradigmatic shift in how we interpret factors such as growth insults and immune response in past skeletal remains. It demonstrates that health cannot be understood in terms of immediate environmental circumstances alone. Furthermore, it requires both a theoretical and practical re-evaluation of disease biographies and the life course more generally. Individual life courses can no longer be regarded as discrete, bounded, life histories, with clearly defined beginning and end points. If socioeconomic circumstances can have intergenerational effects, including disease susceptibility and growth stunting, then individual biographies should be viewed as nested or embedded within the lives of others. This commingling of life courses may prove problematic to unravel; nevertheless, this review aims to consider the potential consequences for bioarchaeological interpretations. These include a greater consideration of: the temporal power of human skeletons and a life course approach to past health; infant health and the implications for maternal well-being; and the impact of non-proximate stressors (e.g., early life and ancestral environments) on the presence of health indicators. (C) 2015 Wiley Periodicals, Inc.",
    "Organisms develop through multiple life stages that differ in environmental tolerances. The seasonal timing, or phenology, of life-stage transitions determines the environmental conditions to which each life stage is exposed and the length of time required to complete a generation. Both environmental and genetic factors contribute to phenological variation, yet predicting their combined effect on life cycles across a geographic range remains a challenge. We linked submodels of the plasticity of individual life stages to create an integrated model that predicts life-cycle phenology in complex environments. We parameterized the model for Arabidopsis thaliana and simulated life cycles in four locations. We compared multiple genotypes by varying two parameters associated with natural genetic variation in phenology: seed dormancy and floral repression. The model predicted variation in life cycles across locations that qualitatively matches observed natural phenology. Seed dormancy had larger effects on life-cycle length than floral repression, and results suggest that a genetic cline in dormancy maintains a life-cycle length of 1 year across the geographic range of this species. By integrating across life stages, this approach demonstrates how genetic variation in one transition can influence subsequent transitions and the geographic distribution of life cycles more generally.",
    "It has long been known that adaptive evolution can occur through genetic mutations in DNA sequence, but it is unclear whether adaptive evolution can occur through analogous epigenetic mechanisms, such as through DNA methylation. If epigenetic variation contributes directly to evolution, species under threat of disease, invasive competition, climate change or other stresses would have greater stores of variation from which to draw. We looked for evidence of natural selection acting on variably methylated DNA sites using population genomic analysis across three climatologically distinct populations of valley oaks. We found patterns of genetic and epigenetic differentiations that indicate local adaptation is operating on large portions of the oak genome. While CHG methyl polymorphisms are not playing a significant role and would make poor targets for natural selection, our findings suggest that CpG methyl polymorphisms as a whole are involved in local adaptation, either directly or through linkage to regions under selection.",
    "Much work has shown that the environment can induce non-genetic changes in phenotype that span multiple generations. Theory predicts that predictable environmental variation selects for both increased within-and across-generation responses. Yet, to the best of our knowledge, there are no empirical tests of this prediction. We explored the relationship between within-versus across-generation plasticity by evaluating the influence of predator cues on the life-history traits of Daphnia ambigua. We measured the duration of predator-induced transgenerational effects, determined when transgenerational responses are induced, and quantified the cues that activate transgenerational plasticity. We show that predator exposure during embryonic development causes earlier maturation and increased reproductive output. Such effects are detectable two generations removed from predator exposure and are similar in magnitude in response to exposure to cues emitted by injured conspecifics. Moreover, all experimental contexts and traits yielded a negative correlation between within-versus across-generation responses. That is, responses to predator cues within- and across-generations were opposite in sign and magnitude. Although many models address transgenerational plasticity, none of them explain this apparent negative relationship between within- and across-generation plasticities. Our results highlight the need to refine the theory of transgenerational plasticity.",
    "Environmentally induced phenotypic plasticity may be a critical component of response to changing environments. We examined local differentiation and adaptive phenotypic plasticity in response to elevated temperature in half-sib lines collected across an elevation gradient for the alpine herb, Wahlenbergia ceracea. Using Amplified Fragment Length Polymorphism (AFLP), we found low but significant genetic differentiation between low- and high-elevation seedlings, and seedlings originating from low elevations grew faster and showed stronger temperature responses (more plasticity) than those from medium and high elevations. Furthermore, plasticity was more often adaptive for plants of low-elevation origin and maladaptive for plants of high elevation. With methylation sensitive-AFLP (MS-AFLP), we revealed an increase in epigenetic variation in response to temperature in low-elevation seedlings. Although we did not find significant direct correlations between MS-AFLP loci and phenotypes, our results demonstrate that adaptive plasticity in temperature response to warming varies over fine spatial scales and suggest the involvement of epigenetic mechanisms in this response.",
    "Phenotypic plasticity is an important mechanism for populations to buffer themselves from environmental change. While it has long been appreciated that natural populations possess genetic variation in the extent of plasticity, a surge of recent evidence suggests that epigenetic variation could also play an important role in shaping phenotypic responses. Compared with genetic variation, epigenetic variation is more likely to have higher spontaneous rates of mutation and a more sensitive reaction to environmental inputs. In our review, we first provide an overview of recent studies on epigenetically encoded thermal plasticity in animals to illustrate environmentally-mediated epigenetic effects within and across generations. Second, we discuss the role of epigenetic effects during adaptation by exploring population epigenetics in natural animal populations. Finally, we evaluate the evolutionary potential of epigenetic variation depending on its autonomy from genetic variation and its transgenerational stability. Although many of the causal links between epigenetic variation and phenotypic plasticity remain elusive, new data has explored the role of epigenetic variation in facilitating evolution in natural populations. This recent progress in ecological epigenetics will be helpful for generating predictive models of the capacity of organisms to adapt to changing climates.",
    "Methylation of DNA cytosines affects whether transposons are silenced and genes are expressed, and is a major epigenetic mechanism whereby plants respond to environmental change. Analyses of methylation-sensitive amplification polymorphism (MS-AFLP or MSAP) have been often used to assess methyl-cytosine changes in response to stress treatments and, more recently, in ecological studies of wild plant populations. MSAP technique does not require a sequenced reference genome and provides many anonymous loci randomly distributed over the genome for which the methylation status can be ascertained. Scoring of MSAP data, however, is not straightforward, and efforts are still required to standardize this step to make use of the potential to distinguish between methylation at different nucleotide contexts. Furthermore, it is not known how accurately MSAP infers genome-wide cytosine methylation levels in plants. Here, we analyse the relationship between MSAP results and the percentage of global cytosine methylation in genomic DNA obtained by HPLC analysis. A screening of literature revealed that methylation of cytosines at cleavage sites assayed by MSAP was greater than genome-wide estimates obtained by HPLC, and percentages of methylation at different nucleotide contexts varied within and across species. Concurrent HPLC and MSAP analyses of DNA from 200 individuals of the perennial herb Helleborus foetidus confirmed that methyl-cytosine was more frequent in CCGG contexts than in the genome as a whole. In this species, global methylation was unrelated to methylation at the inner CG site. We suggest that global HPLC and context-specific MSAP methylation estimates provide complementary information whose combination can improve our current understanding of methylation-based epigenetic processes in nonmodel plants.",
    "Testing for conserved and novel mechanisms underlying phenotypic evolution requires a diversity of genomes available for comparison spanning multiple independent lineages. For example, complex social behavior in insects has been investigated primarily with eusocial lineages, nearly all of which are Hymenoptera. If conserved genomic influences on sociality do exist, we need data from a wider range of taxa that also vary in their levels of sociality. Here, we present the assembled and annotated genome of the subsocial beetle Nicrophorus vespilloides, a species long used to investigate evolutionary questions of complex social behavior. We used this genome to address two questions. First, do aspects of life history, such as using a carcass to breed, predict overlap in gene models more strongly than phylogeny? We found that the overlap in gene models was similar between N. vespilloides and all other insect groups regardless of life history. Second, like other insects with highly developed social behavior but unlike other beetles, does N. vespilloides have DNA methylation? We found strong evidence for an active DNA methylation system. The distribution of methylation was similar to other insects with exons having the most methylated C pGs. Methylation status appears highly conserved; 85% of the methylated genes in N. vespilloides are also methylated in the hymentopteran Nasonia vitripennis. The addition of this genome adds a coleopteran resource to answer questions about the evolution and mechanistic basis of sociality and to address questions about the potential role of methylation in social behavior.",
    "Telomeres are emerging as a biomarker for ageing and survival, and are likely important in shaping life-history trade-offs. In particular, telomere length with which one starts in life has been linked to lifelong survival, suggesting that early telomere dynamics are somehow related to life-history trajectories. This result highlights the importance of determining the extent to which telomere length is inherited, as a crucial factor determining early life telomere length. Given the scarcity of species for which telomere length inheritance has been studied, it is pressing to assess the generality of telomere length inheritance patterns. Further, information on how this pattern changes over the course of growth in individuals living under natural conditions should provide some insight on the extent to which environmental constraints also shape telomere dynamics. To fill this gap partly, we followed telomere inheritance in a population of king penguins (Aptenodytes patagonicus). We tested for paternal and maternal influence on chick initial telomere length (10 days old after hatching), and how these relationships changed with chick age (at 70, 200 and 300 days old). Based on a correlative approach, offspring telomere length was positively associated with maternal telomere length early in life (at 10 days old). However, this relationship was not significant at older ages. These data suggest that telomere length in birds is maternally inherited. Nonetheless, the influence of environmental conditions during growth remained an important factor shaping telomere length, as the maternal link disappeared with chicks' age.",
    "Transgenerational effects can buffer populations against environmental change, yet little is known about underlying mechanisms, their persistence or the influence of environmental cue timing. We investigated mitochondrial respiratory capacity (MRC) and gene expression of marine sticklebacks that experienced acute or developmental acclimation to simulated ocean warming (21 degrees C) across three generations. Previous work showed that acute acclimation of grandmothers to 21 degrees C led to lower (optimized) offspring MRCs. Here, developmental acclimation of mothers to 21 degrees C led to higher, but more efficient offspring MRCs. Offspring with a 21 degrees C x 17 degrees C grandmother-mother environment mismatch showed metabolic compensation: their MRCs were as low as offspring with a 17 degrees C thermal history across generations. Transcriptional analyses showed primarily maternal but also grandmaternal environment effects: genes involved in metabolism and mitochondrial protein biosynthesis were differentially expressed when mothers developed at 21 degrees C, whereas 21 degrees C grandmothers influenced genes involved in hemostasis and apoptosis. Genes involved in mitochondrial respiration all showed higher expression when mothers developed at 21 degrees and lower expression in the 21 degrees C x 17 degrees C group, matching the phenotypic pattern for MRCs. Our study links transcriptomics to physiology under climate change, and demonstrates that mechanisms underlying transgenerational effects persist across multiple generations with specific outcomes depending on acclimation type and environmental mismatch between generations.",
    "The effects from multigenerational exposures to engineered nanoparticles (ENPs) in their pristine and transformed states are currently unknown despite such exposures being an increasingly common scenario in natural environments. Here, we examine how exposure over 10 generations affects the sensitivity of the nematode Caenorhabditis elegans to pristine and sulfidized Ag ENPs and AgNO3. We also include populations that were initially exposed over six generations but kept unexposed for subsequent four generations to allow recovery from exposure. Toxicity of the different silver forms decreased in the order AgNO3, Ag ENPs and Ag2S ENPs. Continuous exposure to Ag ENPs and AgNO3 caused pronounced sensitization (approx. 10-fold) in the F2 generation, which was sustained until F10. This sensitization was less pronounced for Ag2S ENP exposures, indicating different toxicity mechanisms. Subtle changes in size and lifespan were also measured. In the recovery populations, the sensitivity to Ag ENPs and AgNO3 resulting from the initial multigenerational exposure persisted. Their response sensitivity for all endpoints was most closely related to the last ancestral exposed generation (F5), rather than unexposed controls. The mechanisms of transgenerational transfer of sensitivity are probably organized through the epigenome, and we encourage others to investigate such effects as a priority for mechanistic toxicology.",
    "Epigenetic variation has been proposed to contribute to the success of asexual plants, either as a contributor to phenotypic plasticity or by enabling transient adaptation via selection on transgenerationally stable, but reversible, epialleles. While recent studies in experimental plant populations have shown the potential for epigenetic mechanisms to contribute to adaptive phenotypes, it remains unknown whether heritable variation in ecologically relevant traits is at least partially epigenetically determined in natural populations. Here, we tested the hypothesis that DNA methylation variation contributes to heritable differences in flowering time within a single widespread apomictic clonal lineage of the common dandelion (Taraxacum officinale s. lat.). Apomictic clone members of the same apomictic lineage collected from different field sites showed heritable differences in flowering time, which was correlated with inherited differences in methylation-sensitive AFLP marker profiles. Differences in flowering between apomictic clone members were significantly reduced after in vivo demethylation using the DNA methyltransferase inhibitor zebularine. This synchronization of flowering times suggests that flowering time divergence within an apomictic lineage was mediated by differences in DNA methylation. While the underlying basis of the methylation polymorphism at functional flowering time-affecting loci remains to be demonstrated, our study shows that epigenetic variation contributes to heritable phenotypic divergence in ecologically relevant traits in natural plant populations. This result also suggests that epigenetic mechanisms can facilitate adaptive divergence within genetically uniform asexual lineages.",
    "Variation in resource availability commonly exerts strong effects on fitness-related traits in wild animals. However, we know little about the molecular mechanisms that mediate these effects, or about their persistence over time. To address these questions, we profiled genome-wide whole-blood DNA methylation levels in two sets of wild baboons: (i) 'wild-feeding' baboons that foraged naturally in a savanna environment and (ii) 'Lodge' baboons that had ready access to spatially concentrated human food scraps, resulting in high feeding efficiency and low daily travel distances. We identified 1014 sites (0.20% of sites tested) that were differentially methylated between wild-feeding and Lodge baboons, providing the first evidence that resource availability shapes the epigenome in a wild mammal. Differentially methylated sites tended to occur in contiguous stretches (i.e., in differentially methylated regions or DMRs), in promoters and enhancers, and near metabolism-related genes, supporting their functional importance in gene regulation. In agreement, reporter assay experiments confirmed that methylation at the largest identified DMR, located in the promoter of a key glycolysis-related gene, was sufficient to causally drive changes in gene expression. Intriguingly, all dispersing males carried a consistent epigenetic signature of their membership in a wild-feeding group, regardless of whether males dispersed into or out of this group as adults. Together, our findings support a role for DNA methylation in mediating ecological effects on phenotypic traits in the wild and emphasize the dynamic environmental sensitivity of DNA methylation levels across the life course.",
    "Epigenetic modifications are expected to occur at a much faster rate than genetic mutations, potentially causing isolated populations to stochastically drift apart, or if they are subjected to different selective regimes, to directionally diverge. A high level of genome-wide epigenetic divergence between individuals occupying distinct habitats is therefore predicted. Here, we introduce bisulfite-converted restriction site associated DNA sequencing (bsRADseq), an approach to quantify the level of DNA methylation differentiation across multiple individuals. This reduced representation method is flexible in the extent of DNA sequence interrogated. We showcase its applicability in three natural systems, each comprising individuals adapted to divergent environments: a diploid plant (Heliosperma, Caryophyllaceae), a tetraploid plant (Dactylorhiza, Orchidaceae) and an animal (Gasterosteusaculeatus, Gasterosteidae). We present a robust bioinformatic pipeline, combining tools for RAD locus assembly, SNP calling, bisulfite-converted read mapping and DNA methylation calling to analyse bsRADseq data with or without a reference genome. Importantly, our approach accurately distinguishes between SNPs and methylation polymorphism (SMPs). Although DNA methylation frequency between different positions of a genome varies widely, we find a surprisingly high consistency in the methylation profile between individuals thriving in divergent ecological conditions, particularly in Heliosperma. This constitutive stability points to significant molecular or developmental constraints acting on DNA methylation variation. Altogether, by combining the flexibility of RADseq with the accuracy of bisulfite sequencing in quantifying DNA methylation, the bsRADseq methodology and our bioinformatic pipeline open up the opportunity for genome-wide epigenetic investigations of evolutionary and ecological relevance in non-model species, independent of their genomic features.",
    "The growing knowledge about the influence of transposable elements (TEs) on (a) long-term genome and transcriptome evolution; (b) genomic, transcriptomic and epigenetic variation within populations; and (c) patterns of somatic genetic differences in individuals continues to spur the interest of evolutionary biologists in the role of TEs in adaptive evolution. As TEs can trigger a broad range of molecular variation in a population with potentially severe fitness and phenotypic consequences for individuals, different mechanisms evolved to keep TE activity in check, allowing for a dynamic interplay between the host, its TEs and the environment in evolution. Here, we review evidence for adaptive phenotypic changes associated with TEs and the basic molecular mechanisms by which the underlying genetic changes arise: (a) domestication, (b) exaptation, (c) host gene regulation, (d) TE-mediated formation of intronless gene copies-so-called retrogenes and (e) overall increased genome plasticity. Furthermore, we review and discuss how the stress-dependent incapacitation of defence mechanisms against the activity of TEs might facilitate adaptive responses to environmental challenges and how such mechanisms might be particularly relevant in species frequently facing novel environments, such as invasive, pathogenic or parasitic species.",
    "The decreasing cost and increasing scope and power of emerging genomic technologies are reshaping the field of molecular ecology. However, many modern genomic approaches (e.g., RAD-seq) require large amounts of high-quality template DNA. This poses a problem for an active branch of conservation biology: genetic monitoring using minimally invasive sampling (MIS) methods. Without handling or even observing an animal, MIS methods (e.g., collection of hair, skin, faeces) can provide genetic information on individuals or populations. Such samples typically yield low-quality and/or quantities of DNA, restricting the type of molecular methods that can be used. Despite this limitation, genetic monitoring using MIS is an effective tool for estimating population demographic parameters and monitoring genetic diversity in natural populations. Genetic monitoring is likely to become more important in the future as many natural populations are undergoing anthropogenically driven declines, which are unlikely to abate without intensive adaptive management efforts that often include MIS approaches. Here, we profile the expanding suite of genomic methods and platforms compatible with producing genotypes from MIS, considering factors such as development costs and error rates. We evaluate how powerful new approaches will enhance our ability to investigate questions typically answered using genetic monitoring, such as estimating abundance, genetic structure and relatedness. As the field is in a period of unusually rapid transition, we also highlight the importance of legacy data sets and recommend how to address the challenges of moving between traditional and next-generation genetic monitoring platforms. Finally, we consider how genetic monitoring could move beyond genotypes in the future. For example, assessing microbiomes or epigenetic markers could provide a greater understanding of the relationship between individuals and their environment.",
    "ObjectivesDifferences in DNA methylation have been associated with early life adversity, suggesting that alterations in methylation function as one pathway through which adverse early environments are biologically embedded. This study examined associations between exposure to institutional care, quantified as the proportion of time in institutional care at specified follow-up assessment ages, and DNA methylation status in two stress-related genes: FKBP5 and SLC6A4. Materials and MethodsWe analyzed data from the Bucharest Early Intervention Project, which is a prospective study in which children reared in institutional settings were randomly assigned (mean age 22 months) to either newly created foster care or care as usual (to remain in their current placement) and prospectively followed. A group of children from the same geographic area, with no history of institutionalized caregiving, were also recruited. DNA methylation status was determined in DNA extracted from buccal epithelial cells of children at age 12. ResultsAn inverse association was identified such that more time spent in institutional care was associated with lower DNA methylation at specific CpG sites within both genes. DiscussionThese results suggest a lasting impact of early severe social deprivation on methylation patterns in these genes, and contribute to a growing literature linking early adversity and epigenetic variation in children. Am J Phys Anthropol 161:84-93, 2016.. (c) 2016 Wiley Periodicals, Inc.",
    "DNA methylation is an epigenetic mark that plays an inadequately understood role in gene regulation, particularly in nonmodel species. Because it can be influenced by the environment, DNA methylation may contribute to the ability of organisms to acclimatize and adapt to environmental change. We evaluated the distribution of gene body methylation in reef-building corals, a group of organisms facing significant environmental threats. Gene body methylation in six species of corals was inferred from in silico transcriptome analysis of CpG O/E, an estimate of germline DNA methylation that is highly correlated with patterns of methylation enrichment. Consistent with what has been documented in most other invertebrates, all corals exhibited bimodal distributions of germline methylation suggestive of distinct fractions of genes with high and low levels of methylation. The hypermethylated fractions were enriched with genes with housekeeping functions, while genes with inducible functions were highly represented in the hypomethylated fractions. High transcript abundance was associated with intermediate levels of methylation. In three of the coral species, we found that genes differentially expressed in response to thermal stress and ocean acidification exhibited significantly lower levels of methylation. These results support a link between gene body hypomethylation and transcriptional plasticity that may point to a role of DNA methylation in the response of corals to environmental change.",
    "Recent studies show that epigenetic variation in the form of DNA methylation may serve as a substrate for selection. Theory suggests that heritable epigenetic marks that increase fitness should increase in frequency in a population, and these changes may result in novel morphology, behaviour, or physiology, and ultimately reproductive isolation. Therefore, epigenetic variation might provide the first substrate for selection during the course of evolutionary divergence. This hypothesis predicts that populations in the earliest stages of divergence will differentiate in their methylome prior to any genetic differentiation. While several studies have investigated natural epigenetic variation, empirical studies that test predictions about its role in speciation are surprisingly scarce. Here, we investigate DNA methylation variation using an isoschizomeric digest method, Methyl-Sensitive Amplified Polymorphism, across multiple stages of evolutionary divergence in natural populations of North American stream fishes. We show that epigenetic differentiation between methylomes is greater than genetic divergence among closely related populations across two river drainages. Additionally, we demonstrate that epigenetic divergence is a stronger predictor of the strength of behavioural reproductive isolation and suggest that changes in the methylome could influence the evolution of reproductive isolation between species. Our findings suggest a role for epigenetics not only in the initiation of divergence, but also in the maintenance of species boundaries over greater evolutionary timescales.",
    "The development of multicellular organisms involves a delicate interplay between genetic and environmental influences. It is often useful to think of developmental systems as integrating available sources of information about current conditions to produce organisms. Genes and inherited physiology provide cues, as does the state of the environment during development. The integration systems themselves are under genetic control and subject to Darwinian selection, so we expect them to evolve to produce organisms that fit well with current ecological (including social) conditions. We argue for the scientific value of this explicitly informational perspective by providing detailed examples of how it can elucidate taxonomically diverse phenomena. We also present a general framework for linking genetic and phenotypic variation from an informational perspective. This application of Darwinian logic at the organismal level can elucidate genetic influences on phenotypic variation in novel and counterintuitive ways.",
    "Organisms can adapt to variable environments by using environmental cues to modulate developmental gene expression. In principle, maternal influences can adaptively adjust offspring phenotype when early life and adult environments match, but they may be maladaptive when future environments are not predictable. One of the best-studied 'maternal effects' is through modification of the offspring's hypothalamic-pituitary-adrenal (HPA) axis, the neuroendocrine system that controls responses to stress. In addition to the direct transfer of glucocorticoids from mother to offspring, offspring HPA function and other phenotypes can also be affected by epigenetic modifications like DNA methylation of the glucocorticoid receptor promoter. Here we examine how among-year variation in rainfall is related to DNA methylation during development and fitness in adulthood in the superb starling (Lamprotornis superbus), which lives in a climatically unpredictable environment where early life and adult environments are unlikely to match. We found that DNA methylation in the putative promoter of the glucocorticoid receptor gene is reduced in chicks - particularly in males - born following drier prebreeding periods. Additionally, DNA methylation is lower in males that become breeders than those that never breed. However, there is no relationship in females between DNA methylation and the likelihood of dispersing from the natal group to breed elsewhere. These results suggest that early life conditions may positively affect fitness in a sex-specific manner through chemical modification of an HPA-associated gene. This study is the first to show that epigenetic modifications during early life may influence the fitness of free-living organisms adapted to unpredictable environments.",
    "Personality traits are heritable and respond to natural selection, but are at the same time influenced by the ontogenetic environment. Epigenetic effects, such as DNA methylation, have been proposed as a key mechanism to control personality variation. However, to date little is known about the contribution of epigenetic effects to natural variation in behaviour. Here, we show that great tit (Parus major) lines artificially selected for divergent exploratory behaviour for four generations differ in their DNA methylation levels at the dopamine receptor D4 (DRD4) gene. This D4 receptor is statistically associated with personality traits in both humans and nonhuman animals, including the great tit. Previous work in this songbird failed to detect functional genetic polymorphisms within DRD4 that could account for the gene-trait association. However, our observation supports the idea that DRD4 is functionally involved in exploratory behaviour but that its effects are mediated by DNA methylation. While the exact mechanism underlying the transgenerational consistency of DRD4 methylation remains to be elucidated, this study shows that epigenetic mechanisms are involved in shaping natural variation in personality traits. We outline how this first finding provides a basis for investigating the epigenetic contribution to personality traits in natural systems and its subsequent role for understanding the ecology and evolution of behavioural consistency.",
    "Gene sequence similarity due to shared ancestry after a duplication event, that is paralogy, complicates the assessment of genetic variation, as sequences originating from paralogs can be difficult to distinguish. These confounded sequences are often removed prior to further analyses, leaving the underlying loci uncharacterized. Salmonids have only partially rediploidized subsequent to a whole-genome duplication; residual tetrasomic inheritance has been observed in males. We present a maximum-likelihood-based method to resolve confounded paralogous loci by observing the segregation of alleles in gynogenetic haploid offspring and demonstrate its effectiveness by constructing two linkage maps for chum salmon (Oncorhynchus keta), with and without these newly resolved loci. We find that the resolved paralogous loci are not randomly distributed across the genome. A majority are clustered in expanded subtelomeric regions of 14 linkage groups, suggesting a significant fraction of the chum salmon genome may be missed by the exclusion of paralogous loci. Transposable elements have been proposed as drivers of genome evolution and, in salmonids, may have an important role in the rediploidization process by driving differentiation between homeologous chromosomes. Consistent with that hypothesis, we find a reduced fraction of transposable element annotations among paralogous loci, and these loci predominately occur in the genomic regions that lag in the rediploidization process.",
    "Heat-shock proteins (Hsps) and their cognates are primary mitigators of cell stress. With increasingly severe impacts of climate change and other human modifications of the biosphere, the ability of the heat-shock system to affect evolutionary fitness in environments outside the laboratory and to evolve in response is topic of growing importance. Since the last major reviews, several advances have occurred. First, demonstrations of the heat-shock response outside the laboratory now include many additional taxa and environments. Many of these demonstrations are only correlative, however. More importantly, technical advances in omic quantification of nucleic acids and proteins, genomewide association analysis, and manipulation of genes and their expression have enabled the field to move beyond correlation. Several consequent advances are already evident: The pathway from heat-shock gene expression to stress tolerance in nature can be extremely complex, mediated through multiple biological processes and systems, and even multiple species. The underlying genes are more numerous, diverse and variable than previously appreciated, especially with respect to their regulatory variation and epigenetic changes. The impacts and limitations (e.g., due to trade-offs) of natural selection on these genes have become more obvious and better established. At last, as evolutionary capacitors, Hsps may have distinctive impacts on the evolution of other genes and ecological consequences.",
    "Resetting of epigenetic marks, such as DNA methylation, in germ cells or early embryos is not always complete. Epigenetic states may therefore persist, decay or accumulate across generations. In spite of mounting empirical evidence for incomplete resetting, it is currently poorly understood whether it simply reflects stochastic noise or plays an adaptive role in phenotype determination. Here, we use a simple model to show that incomplete resetting can be adaptive in heterogeneous environments. Transmission of acquired epigenetic states prevents mismatched phenotypes when the environment changes infrequently relative to generation time and when maternal and environmental cues are unreliable. We discuss how these results may help to interpret the emerging data on transgenerational epigenetic inheritance in plants and animals.",
    "Genomes of invertebrates and vertebrates exhibit highly divergent patterns of DNA methylation. Invertebrate genomes tend to be sparsely methylated, and DNA methylation is mostly targeted to a subset of transcription units (gene bodies). In a drastic contrast, vertebrate genomes are generally globally and heavily methylated, punctuated by the limited local hypo-methylation of putative regulatory regions such as promoters. These genomic differences also translate into functional differences in DNA methylation and gene regulation. Although promoter DNA methylation is an important regulatory component of vertebrate gene expression, its role in invertebrate gene regulation has been little explored. Instead, gene body DNA methylation is associated with expression of invertebrate genes. However, the evolutionary steps leading to the differentiation of invertebrate and vertebrate genomic DNA methylation remain unresolved. Here we analyzed experimentally determined DNA methylation maps of several species across the invertebrate-vertebrate boundary, to elucidate how vertebrate gene methylation has evolved. We show that, in contrast to the prevailing idea, a substantial number of promoters in an invertebrate basal chordate Ciona intestinalis are methylated. Moreover, gene expression data indicate significant, epigenomic context-dependent associations between promoter methylation and expression in C. intestinalis. However, there is no evidence that promoter methylation in invertebrate chordate has been evolutionarily maintained across the invertebrate-vertebrate boundary. Rather, body-methylated invertebrate genes preferentially obtain hypo-methylated promoters among vertebrates. Conversely, promoter methylation is preferentially found in lineage- and tissue-specific vertebrate genes. These results provide important insights into the evolutionary origin of epigenetic regulation of vertebrate gene expression.",
    "Animal and plant centromeres are embedded in repetitive satellite DNA, but are thought to be epigenetically specified. To define genetic characteristics of centromeres, we surveyed satellite DNA from diverse eukaryotes and identified variation in <10-bp dyad symmetries predicted to adopt non-B-form conformations. Organisms lacking centromeric dyad symmetries had binding sites for sequence-specific DNA-binding proteins with DNA-bending activity. For example, human and mouse centromeres are depleted for dyad symmetries, but are enriched for non-B-form DNA and are associated with binding sites for the conserved DNA-binding protein CENP-B, which is required for artificial centromere function but is paradoxically nonessential. We also detected dyad symmetries and predicted non-B-form DNA structures at neocentromeres, which form at ectopic loci. We propose that centromeres form at non-B-form DNA because of dyad symmetries or are strengthened by sequence-specific DNA binding proteins. This may resolve the CENP-B paradox and provide a general basis for centromere specification.",
    "Epigenetic mechanisms such as changes in DNA methylation have the potential to affect the resilience of species to climate change, but little is known about the response of the methylome to changes in environmental temperature in animals. Using reduced representation bisulfite sequencing, we assessed the effects of development temperature and adult acclimation temperature on DNA methylation levels in threespine stickleback (Gasterosteus aculeatus). Across all treatments, we identified 2130 differentially methylated cytosines distributed across the genome. Both increases and decreases in temperature during development and with thermal acclimation in adults increased global DNA methylation levels. Approximately 25% of the differentially methylated regions (DMRs) responded to both developmental temperature and adult thermal acclimation, and 50 DMRs were common to all treatments, demonstrating a core response of the epigenome to thermal change at multiple time scales. We also identified differentially methylated loci that were specific to a particular developmental or adult thermal response, which could facilitate the accumulation of epigenetic variation between natural populations that experience different thermal regimes. These data demonstrate that thermal history can have long-lasting effects on the epigenome, highlighting the role of epigenetic modifications in the response to temperature change across multiple time scales.",
    "The diversity of sex chromosomes among amniotes is the product of independent evolution of different systems in different lineages, defined by novel sex-determining genes. Convergent evolution is very common, suggesting that some genes are particularly adept at taking on a sex-determining role. Comparative gene mapping, and more recently whole genome sequencing, have now turned up other surprising relationships; different regions of the amniote genome that have become sex determining in some taxa seem to share synteny, or share sequence, in others. Is this, after all, evidence that these regions were once linked in a super-sex chromosome that underwent multiple fission in different ways in different amniote lineages? Or does it signify that special properties of sex chromosomes (paucity of active genes, low recombination, epigenetic regulation to achieve dosage compensation) predispose particular chromosomes to a sex-determining role?",
    "Epigenetic changes can occur due to extracellular environmental conditions. Consequently, epigenetic mechanisms can play an intermediate role to translate environmental signals to intracellular changes. Such a role might be particularly important in plants, which often show strong local adaptation and have the potential for heritable epigenetic states. However, little is currently known about the role of epigenetic variation in the ecological mechanisms of adaptation. Here, we used multivariate redundancy analyses to examine genomewide associations between DNA methylation polymorphisms and climate variation in two independent panels of Arabidopsis accessions, including 122 Eurasian accessions as well as in a regional panel of 148 accessions in Sweden. At the single-nucleotide methylation level, climate and space (geographic spatial structure) explain small yet significant amount of variation in both panels. On the other hand, when viewed in a context of genomic clusters of methylated and unmethylated cytosines, climate and space variables explain much greater amounts of variation in DNA methylation than those explained by variation at the single-nucleotide level. We found that the single-nucleotide methylation polymorphisms with the strongest associations with climate were enriched in transposable elements and in potentially RNA-directed methylation contexts. When viewed in the context of genomic clusters, variation of DNA methylation at different sequence contexts exhibit distinctive segregation along different axes of variation in the redundancy analyses. Genomewide methylation showed much stronger associations with climate within the regional panel (Sweden) compared to the global (Eurasia). Together, these findings indicate that genetic and epigenetic variation across the genome may play a role in response to climate conditions and local adaptation.",
    "The three-spined stickleback (Gasterosteus aculeatus) represents a convenient model to study microevolution-adaptation to a freshwater environment. Although genetic adaptations to freshwater environments are well-studied, epigenetic adaptations have attracted little attention. In this work, we investigated the role of DNA methylation in the adaptation of the marine stickleback population to freshwater conditions. DNA methylation profiling was performed in marine and freshwater populations of sticklebacks, as well as in marine sticklebacks placed into a freshwater environment and freshwater sticklebacks placed into seawater. We showed that the DNA methylation profile after placing a marine stickleback into fresh water partially converged to that of a freshwater stickleback. For six genes including ATP4A ion pump and NELL1, believed to be involved in skeletal ossification, we demonstrated similar changes in DNA methylation in both evolutionary and short-term adaptation. This suggested that an immediate epigenetic response to freshwater conditions can be maintained in freshwater population. Interestingly, we observed enhanced epigenetic plasticity in freshwater sticklebacks that may serve as a compensatory regulatory mechanism for the lack of genetic variation in the freshwater population. For the first time, we demonstrated that genes encoding ion channels KCND3, CACNA1FB, and ATP4A were differentially methylated between the marine and the freshwater populations. Other genes encoding ion channels were previously reported to be under selection in freshwater populations. Nevertheless, the genes that harbor genetic and epigenetic changes were not the same, suggesting that epigenetic adaptation is a complementary mechanism to selection of genetic variants favorable for freshwater environment.",
    "Epigenetic modifications, such as DNA methylation or histone modifications, can be transmitted between cellular or organismal generations. However, there are no experiments measuring their role in adaptation, so here we use experimental evolution to investigate how epigenetic variation can contribute to adaptation. We manipulated DNA methylation and histone acetylation in the unicellular green alga Chlamydomonas reinhardtii both genetically and chemically to change the amount of epigenetic variation generated or transmitted in adapting populations in three different environments (salt stress, phosphate starvation, and high CO2) for two hundred asexual generations. We find that reducing the amount of epigenetic variation available to populations can reduce adaptation in environments where it otherwise happens. From genomic and epigenomic sequences from a subset of the populations, we see changes in methylation patterns between the evolved populations over-represented in some functional categories of genes, which is consistent with some of these differences being adaptive. Based on whole genome sequencing of evolved clones, the majority of DNA methylation changes do not appear to be linked to cis-acting genetic mutations. Our results show that transgenerational epigenetic effects play a role in adaptive evolution, and suggest that the relationship between changes in methylation patterns and differences in evolutionary outcomes, at least for quantitative traits such as cell division rates, is complex.",
  ]

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 50)

In [34]:
df = [
"Chronic inflammation is involved in the onset and development of many diseases, including obesity, atherosclerosis, type 2 diabetes, osteoarthritis, autoimmune and degenerative diseases, asthma, periodontitis, and cirrhosis. The inflammation process is mediated by chemokines, cytokines, and different inflammatory cells. Although the molecules and mechanisms that regulate this primary defense mechanism are not fully understood, recent findings offer a putative role of noncoding RNAs, especially microRNAs (miRNAs), in the progression and management of the inflammatory response. These noncoding RNAs are crucial for the stability and maintenance of gene expression patterns that characterize some cell types, tissues, and biologic responses. Several miRNAs, such as miR-126, miR-132, miR-146, miR-155, and miR-221, have emerged as important transcriptional regulators of some inflammation-related mediators. Additionally, little is known about the involvement of long noncoding RNAs, long intergenic noncoding RNAs, and circular RNAs in inflammation-mediated processes and the homeostatic imbalance associated with metabolic disorders. These noncoding RNAs are emerging as biomarkers with diagnosis value, in prognosis protocols, or in the personalized treatment of inflammation-related alterations. In this context, this review summarizes findings in the field, highlighting those noncoding RNAs that regulate inflammation, with emphasis on recognized mediators such as TNF-alpha, IL-1, IL-6, IL-18, intercellular adhesion molecule 1, VCAM-1, and plasminogen activator inhibitor 1. The down-regulation or antagonism of the noncoding RNAs and the administration of exogenous miRNAs could be, in the near future, a promising therapeutic strategy in the treatment of inflammation-related diseases.", 
"Kinetochores are large protein assemblies that connect chromosomes to microtubules of the mitotic and meiotic spindles in order to distribute the replicated genome from a mother cell to its daughters. Kinetochores also control feedback mechanisms responsible for the correction of incorrect microtubule attachments, and for the coordination of chromosome attachment with cell cycle progression. Finally, kinetochores contribute to their own preservation, across generations, at the specific chromosomal loci devoted to host them, the centromeres. They achieve this in most species by exploiting an epigenetic, DNA-sequence-independent mechanism; notable exceptions are budding yeasts where a specific sequence is associated with centromere function. In the last 15 years, extensive progress in the elucidation of the composition of the kinetochore and the identification of various physical and functional modules within its substructure has led to a much deeper molecular understanding of kinetochore organization and the origins of its functional output. Here, we provide a broad summary of this progress, focusing primarily on kinetochores of humans and budding yeast, while highlighting work from other models, and present important unresolved questions for future studies.", 
"Individual animals frequently exhibit repeatable differences from other members of their population, differences now commonly referred to as 'animal personality'. Personality differences can arise, for example, from differences in permanent environmental effects-including parental and epigenetic contributors- and the effect of additive genetic variation. Although several studies have evaluated the heritability of behaviour, less is known about general patterns of heritability and additive genetic variation in animal personality. As overall variation in behaviour includes both the among-individual differences that reflect different personalities and temporary environmental effects, it is possible for personality to be largely genetically influenced even when heritability of behaviour per se is quite low. The relative contribution of additive genetic variation to personality variation can be estimated whenever both repeatability and heritability are estimated for the same data. Using published estimates to address this issue, we found that approximately 52% of animal personality variation was attributable to additive genetic variation. Thus, while the heritability of behaviour is often moderate or low, the heritability of personality is much higher. Our results therefore (i) demonstrate that genetic differences are likely to be a major contributor to variation in animal personality and (ii) support the phenotypic gambit: that evolutionary inferences drawn from repeatability estimates may often be justified.", 
"Cytosine DNA methylation (mC) is a genome modification that can regulate the expression of coding and non-coding genetic elements. However, little is known about the involvement of mC in response to environmental cues. Using whole genome bisulfite sequencing to assess the spatio-temporal dynamics of mC in rice grown under phosphate starvation and recovery conditions, we identified widespread phosphate starvation-induced changes in mC, preferentially localized in transposable elements (TEs) close to highly induced genes. These changes in mC occurred after changes in nearby gene transcription, were mostly DCL3a-independent, and could partially be propagated through mitosis, however no evidence of meiotic transmission was observed. Similar analyses performed in Arabidopsis revealed a very limited effect of phosphate starvation on mC, suggesting a species-specific mechanism. Overall, this suggests that TEs in proximity to environmentally induced genes are silenced via hypermethylation, and establishes the temporal hierarchy of transcriptional and epigenomic changes in response to stress.", 
"Experience and memory of environmental stimuli that indicate future stress can prepare (prime) organismic stress responses even in species lacking a nervous system. The process through which such organisms prepare their phenotype for an improved response to future stress has been termed 'priming'. however, other terms are also used for this phenomenon, especially when considering priming in different types of organisms and when referring to different stressors. Here we propose a conceptual framework for printing of stress responses in bacteria, fungi and plants winch allows comparison of priming with other terms, e.g. adaptation, acclimation, induction, acquired resistance and cross protection. We address spatial and temporal aspects of priming and highlight current knowledge about the mechanisms necessary for information storage which range from epigenetic marks to the accunmlation of (dormant.) signalling molecules. Furthermore, we outline possible patterns of primed stress responses. Finally, we link the ability of organisms to become primed for stress responses (their 'primability) with evolutionary ecology aspects and discuss which properties of an organism and its environment may favour the evolution of priming of stress responses.", 
"Long noncoding RNAs (lncRNAs) are the major elements of the mammalian transcriptome that is emerging as a central player controlling diverse cellular mechanisms. Most of the well-studied lncRNAs so far are found to be crucial in regulating cellular processes such as cell cycle, growth, and apoptosis that ensure homeostasis. Owing to their location and distribution in the genome, lncRNAs influence the transcription of a wide range of proteins directly or indirectly by transcriptional and posttranscriptional alterations, which opens up the LncRNA-cancer paradigm in a context-dependent manner, i.e., either oncogenic or tumor suppressive. Thus, this chapter is a consolidation of lncRNA association in exhibiting or suppressing the typical cancer hallmarks such as continuous proliferation, surpassing apoptosis, genomic instability, drug resistance, invasion, and metastasis studied till date. In addition, special focus has been given on the efficient application of lncRNAs as potential targets for therapeutics that holds a great promise for future cancer therapy.", 
"Inducible epigenetic changes in eukaryotes are believed to enable rapid adaptation to environmental fluctuations. We have found distinct regions of the Arabidopsis genome that are susceptible to DNA (de)methylation in response to hyperosmotic stress. The stress-induced epigenetic changes are associated with conditionally heritable adaptive phenotypic stress responses. However, these stress responses are primarily transmitted to the next generation through the female lineage due to widespread DNA glycosylase activity in the male germline, and extensively reset in the absence of stress. Using the CNI1/ATL31 locus as an example, we demonstrate that epigenetically targeted sequences function as distantly-acting control elements of antisense long non-coding RNAs, which in turn regulate targeted gene expression in response to stress. Collectively, our findings reveal that plants use a highly dynamic maternal 'short-term stress memory' with which to respond to adverse external conditions. This transient memory relies on the DNA methylation machinery and associated transcriptional changes to extend the phenotypic plasticity accessible to the immediate offspring.", 
"Environmental toxicants can alter epigenetic regulatory features such as DNA methylation and microRNA expression. As the sensitivity of epigenomic regulatory features may be greatest during the in utero period, when critical windows are narrow, and when epigenomic profiles are being set, this review will highlight research focused on that period. I will focus on work in human populations, where the impact of environmental toxicants in utero, including cigarette smoke and toxic trace metals such as arsenic, mercury and manganese, on genome-wide, gene-specific DNA methylation has been assessed. In particular, arsenic is highlighted, as this metalloid has been the focus of a number of studies and its detoxification mechanisms are well understood. Importantly, the tissues and cells being examined must be considered in context in order to interpret the findings of these studies. For example, by studying the placenta, it is possible to identify potential epigenetic adaptations of key genes and pathways that may alter the developmental course in line with the developmental origins of health and disease paradigm. Alternatively, studies of newborn cord blood can be used to examine how environmental exposure in utero can impact the composition of cells within the peripheral blood, leading to immunological effects of exposure. The results suggest that in humans, like other vertebrates, there is a susceptibility for epigenomic alteration by the environment during intrauterine development, and this may represent a mechanism of plasticity of the organism in response to its environment as well as a mechanism through which long-term health consequences can be shaped.", 
"For the last four decades, we have known that noncoding RNAs maintain critical housekeeping functions such as transcription, RNA processing, and translation. However, in the late 1990s and early 2000s, the advent of high-throughput sequencing technologies and computational tools to analyze these large sequencing datasets facilitated the discovery of thousands of small and long noncoding RNAs (lncRNAs) and their functional role in diverse biological functions. For example, lncRNAs have been shown to regulate dosage compensation, genomic imprinting, pluripotency, cell differentiation and development, immune response, etc. Here we review how lncRNAs bring about such copious functions by employing diverse mechanisms such as translational inhibition, mRNA degradation, RNA decoys, facilitating recruitment of chromatin modifiers, regulation of protein activity, regulating the availability of miRNAs by sponging mechanism, etc. In addition, we provide a detailed account of different mechanisms as well as general principles by which lncRNAs organize functionally different nuclear sub-compartments and their impact on nuclear architecture.", 
"The recent and rapid worldwide increase in non-communicable diseases challenges the assumption that genetic factors are the primary contributors to such diseases. A new concept of the 'developmental origins of health and disease' (DOHaD) is at stake and therefore requires a paradigm shift. Maternal obesity and malnutrition predispose offspring to develop metabolic syndrome, a vicious cycle leading to transmission to subsequent generation(s), with differences in response and susceptibility according to the sex of the individual. The placenta is a programming agent of adult health and disease. Adaptations of placental phenotype in response to maternal diet and metabolic status alter fetal nutrient supply. This implies important epigenetic changes that are, however, still poorly documented in DOHaD studies, particularly concerning overnutrition. The aim of this review is to discuss the emerging knowledge on the relationships between the effect of maternal nutrition or metabolic status on placental function and the risk of diseases later in life, with a specific focus on epigenetic mechanisms and sexual dimorphism. Explaining the sex-specific causal variables and how males versus females respond and adapt to environmental perturbations should help physicians and patients to anticipate disease susceptibility.", 
"DNA methylation is extensively remodeled during mammalian gametogenesis and embryogenesis. Most transposons become hypomethylated, raising the question of their regulation in the absence of DNA methylation. To reproduce a rapid and extensive demethylation, we subjected mouse ES cells to chemically defined hypomethylating culture conditions. Surprisingly, we observed two phases of transposon regulation. After an initial burst of de-repression, various transposon families were efficiently re-silenced. This was accompanied by a reconfiguration of the repressive chromatin landscape: while H3K9me3 was stable, H3K9me2 globally disappeared and H3K27me3 accumulated at transposons. Interestingly, we observed that H3K9me3 and H3K27me3 occupy different transposon families or different territories within the same family, defining three functional categories of adaptive chromatin responses to DNA methylation loss. Our work highlights that H3K9me3 and, most importantly, polycomb-mediated H3K27me3 chromatin pathways can secure the control of a large spectrum of transposons in periods of intense DNA methylation change, ensuring longstanding genome stability.", 
"Coral reefs are globally threatened by climate change-related ocean warming and ocean acidification (OA). To date, slow-response mechanisms such as genetic adaptation have been considered the major determinant of coral reef persistence, with little consideration of rapid-response acclimatization-mechanisms. These rapid mechanisms such as parental effects that can contribute to trans-generational acclimatization (e.g. epigenetics) have, however, been identified as important contributors to offspring response in other systems. We present the first evidence of parental effects in a cross-generational exposure to temperature and OA in reef-building corals. Here, we exposed adults to high (28.9 degrees C, 805 mu atm P-CO2) or ambient (26.5 degrees C, 417 mu atm P-CO2) temperature and OA treatments during the larval brooding period. Exposure to high treatment negatively affected adult performance, but their larvae exhibited size differences and metabolic acclimation when subsequently re-exposed, unlike larvae from parents exposed to ambient conditions. Understanding the innate capacity corals possess to respond to current and future climatic conditions is essential to reef protection and maintenance. Our results identify that parental effects may have an important role through (1) ameliorating the effects of stress through preconditioning and adaptive plasticity, and/or (2) amplifying the negative parental response through latent effects on future life stages. Whether the consequences of parental effects and the potential for trans-generational acclimatization are beneficial or maladaptive, our work identifies a critical need to expand currently proposed climate change outcomes for corals to further assess rapid response mechanisms that include non-genetic inheritance through parental contributions and classical epigenetic mechanisms.", 
"Purpose: To review the data concerning the role of endogenously generated reactive oxygen species (ROS) in the non-targeted ionizing radiation (IR) effects and in determination of the cell population's fate, both early after exposure and after many generations. Conclusions: The short-term as well as chronic oxidative stress responses mainly are produced due to ROS generation by the electron transport chain (ETC) of the mitochondria and by the cytoplasmic NADPH oxidases. Whether the induction of the oxidative stress and its consequences occur or are hampered in a single cell largely depends on the interaction between the nucleus and the cellular population of several hundred or thousands of mitochondria that are genetically heterogeneous. High intra-mitochondrial ROS level is damaging the mitochondrial (mt) DNA and its mutations affect the epigenetic control mechanisms of the nuclear (n) DNA, by decreasing the activity of methyltransferases and thus, causing global DNA hypomethylation. These changes are transmitted to the progeny of the irradiated cells. The chronic oxidative stress is the main cause of the late post-radiation effects, including cancer, and this makes it an important adverse effect of exposure to IR and a target for radiological protection.", 
"For many animals, survival of severe environmental stress (e. g. to extremes of heat or cold, drought, oxygen limitation, food deprivation) is aided by entry into a hypometabolic state. Strong depression of metabolic rate, often to only 1-20% of normal resting rate, is a core survival strategy of multiple forms of hypometabolism across the animal kingdom, including hibernation, anaerobiosis, aestivation and freeze tolerance. Global biochemical controls are needed to suppress and reprioritize energy use; one such well-studied control is reversible protein phosphorylation. Recently, we turned our attention to the idea that mechanisms previously associated mainly with epigenetic regulation can also contribute to reversible suppression of gene expression in hypometabolic states. Indeed, situations as diverse as mammalian hibernation and turtle anoxia tolerance show coordinated changes in histone post-translational modifications (acetylation, phosphorylation) and activities of histone deacetylases, consistent with their use as mechanisms for suppressing gene expression during hypometabolism. Other potential mechanisms of gene silencing in hypometabolic states include altered expression of miRNAs that can provide post-transcriptional suppression of mRNA translation and the formation of ribonuclear protein bodies in the nucleus and cytoplasm to allow storage of mRNA transcripts until animals rouse themselves again. Furthermore, mechanisms first identified in epigenetic regulation (e.g. protein acetylation) are now proving to apply to many central metabolic enzymes (e.g. lactate dehydrogenase), suggesting a new layer of regulatory control that can contribute to coordinating the depression of metabolic rate.", 
"Recombination, the exchange of DNA between maternal and paternal chromosomes during meiosis, is an essential feature of sexual reproduction in nearly all multicellular organisms. While the role of recombination in the evolution of sex has received theoretical and empirical attention, less is known about how recombination rate itself evolves and what influence this has on evolutionary processes within sexually reproducing organisms. Here, we explore the patterns of, and processes governing recombination in eukaryotes. We summarize patterns of variation, integrating current knowledge with an analysis of linkage map data in 353 organisms. We then discuss proximate and ultimate processes governing recombination rate variation and consider how these influence evolutionary processes. Genome-wide recombination rates (cM/Mb) can vary more than tenfold across eukaryotes, and there is large variation in the distribution of recombination events across closely related taxa, populations and individuals. We discuss how variation in rate and distribution relates to genome architecture, genetic and epigenetic mechanisms, sex, environmental perturbations and variable selective pressures. There has been great progress in determining the molecular mechanisms governing recombination, and with the continued development of new modelling and empirical approaches, there is now also great opportunity to further our understanding of how and why recombination rate varies. This article is part of the themed issue 'Evolutionary causes and consequences of recombination rate variation in sexual organisms'.", 
"In recent years, the bidirectional communication between the gut microbiome and the brain has emerged as a factor that influences immunity, metabolism, neurodevelopment and behaviour. Cross-talk between the gut and brain begins early in life immediately following the transition from a sterile in utero environment to one that is exposed to a changing and complex microbial milieu over a lifetime. Once established, communication between the gut and brain integrates information from the autonomic and enteric nervous systems, neuroendocrine and neuroimmune signals, and peripheral immune and metabolic signals. Importantly, the composition and functional potential of the gut microbiome undergoes many transitions that parallel dynamic periods of brain development and maturation for which distinct sex differences have been identified. Here, we discuss the sexually dimorphic development, maturation and maintenance of the gut microbiome-brain axis, and the sex differences therein important in disease risk and resilience throughout the lifespan.", 
"Aicardi-Goutieres syndrome (AGS) is a severe childhood inflammatory disorder that shows clinical and genetic overlap with systemic lupus erythematosus (SLE). AGS is thought to arise from the accumulation of incompletely metabolized endogenous nucleic acid species owing to mutations in nucleic acid-degrading enzymes TREX1 (AGS1), RNase H2 (AGS2, 3 and 4), and SAMHD1 (AGS5). However, the identity and source of such immunogenic nucleic acid species remain undefined. Using genome-wide approaches, we show that fibroblasts from AGS patients with AGS1-5 mutations are burdened by excessive loads of RNA: DNA hybrids. Using MethylC-seq, we show that AGS fibroblasts display pronounced and global loss of DNA methylation and demonstrate that AGS-specific RNA: DNA hybrids often occur within DNA hypomethylated regions. Altogether, our data suggest that RNA: DNA hybrids may represent a common immunogenic form of nucleic acids in AGS and provide the first evidence of epigenetic perturbations in AGS, furthering the links between AGS and SLE.", 
"The field described as 'epigenetics' has captured the imagination of scientists and the lay public. Advances in our understanding of chromatin and gene regulatory mechanisms have had impact on drug development, fueling excitement in the lay public about the prospects of applying this knowledge to address health issues. However, when describing these scientific advances as 'epigenetic', we encounter the problem that this term means different things to different people, starting within the scientific community and amplified in the popular press. To help researchers understand some of the misconceptions in the field and to communicate the science accurately to each other and the lay audience, here we review the basis for many of the assumptions made about what are currently referred to as epigenetic processes.", 
"The host innate immune response mediated by type I interferon (IFN) and the resulting up-regulation of hundreds of interferon-stimulated genes (ISGs) provide an immediate barrier to virus infection. Studies of the type I 'interferome' have mainly been carried out at a single species level, often lacking the power necessary to understand key evolutionary features of this pathway. Here, using a single experimental platform, we determined the properties of the interferomes of multiple vertebrate species and developed a webserver to mine the dataset. This approach revealed a conserved 'core' of 62 ISGs, including genes not previously associated with IFN, underscoring the ancestral functions associated with this antiviral host response. We show that gene expansion contributes to the evolution of the IFN system and that interferomes are shaped by lineage-specific pressures. Consequently, each mammal possesses a unique repertoire of ISGs, including genes common to all mammals and others unique to their specific species or phylogenetic lineages. An analysis of genes commonly down-regulated by IFN suggests that epigenetic regulation of transcription is a fundamental aspect of the IFN response. Our study provides a resource for the scientific community highlighting key paradigms of the type I IFN response.", 
"Epigenetics plays an important role in orchestrating key biologic processes. Epigenetic marks, including DNA methylation, histones, chromatin structure, and noncoding RNAs, are modified throughout life in response to environmental and behavioral influences. With each new generation, DNA methylation patterns are erased in gametes and reset after fertilization, probably to prevent these epigenetic marks from being transferred from parents to their offspring. However, some recent animal studies suggest an apparent resistance to complete erasure of epigenetic marks during early development, enabling transgenerational epigenetic inheritance. Whether there are similar mechanisms in humans remains unclear, with the exception of epigenetic imprinting. Nevertheless, a distinctly different mechanism-namely, intrauterine exposure to environmental stressors that may affect establishment of the newly composing epigenetic patterns after fertilization-is often confused with transgenerational epigenetic inheritance. In this review, we delineate the definition of and requirement for transgenerational epigenetic inheritance, differentiate it from the consequences of intrauterine exposure, and discuss the available evidence in both animal models and humans.", 
"In flowering plants, seed development is initiated by the fusion of the maternal egg and central cells with two paternal sperm cells, leading to the formation of embryo and endosperm, respectively. The fertilization products are surrounded by the maternally derived seed coat, whose development prior to fertilization is blocked by epigenetic regulators belonging to the Polycomb Group (PcG) protein family. Here we show that fertilization of the central cell results in the production of auxin and most likely its export to the maternal tissues, which drives seed coat development by removing PcG function. We furthermore show that mutants for the MADS-box transcription factor AGL62 have an impaired transport of auxin from the endosperm to the integuments, which results in seed abortion. We propose that AGL62 regulates auxin transport from the endosperm to the integuments, leading to the removal of the PcG block on seed coat development.", 
"Identifying the genetic input for fetal growth will help to understand common, serious complications of pregnancy such as fetal growth restriction. Genomic imprinting is an epigenetic process that silences one parental allele, resulting in monoallelic expression. Imprinted genes are important in mammalian fetal growth and development. Evidence has emerged showing that genes that are paternally expressed promote fetal growth, whereas maternally expressed genes suppress growth. We have assessed whether the expression levels of key imprinted genes correlate with fetal growth parameters during pregnancy, either early in gestation, using chorionic villus samples (CVS), or in term placenta. We have found that the expression of paternally expressing insulin-like growth factor 2 (IGF2), its receptor IGF2R, and the IGF2/IGF1R ratio in CVS tissues significantly correlate with crown-rump length and birthweight, whereas term placenta expression shows no correlation. For the maternally expressing pleckstrin homology-like domain family A, member 2 (PHLDA2), there is no correlation early in pregnancy in CVS but a highly significant negative relationship in term placenta. Analysis of the control of imprinted expression of PHLDA2 gave rise to a maternally and compounded grand-maternally controlled genetic effect with a birthweight increase of 93/155 g, respectively, when one copy of the PHLDA2 promoter variant is inherited. Expression of the growth factor receptor-bound protein 10 (GRB10) in term placenta is significantly negatively correlated with head circumference. Analysis of the paternally expressing delta-like 1 homologue (DLK1) shows that the paternal transmission of type 1 diabetes protective G allele of rs941576 single nucleotide polymorphism (SNP) results in significantly reduced birth weight (2132 g). In conclusion, we have found that the expression of key imprinted genes show a strong correlation with fetal growth and that for both genetic and genomics data analyses, it is important not to overlook parent-of-origin effects.", 
"Aging is a major risk factor in many forms of late-onset neurodegenerative disorders. The ability to recapitulate age-related characteristics of human neurons in culture will offer unprecedented opportunities to study the biological processes underlying neuronal aging. Here, we show that using a recently demonstrated microRNA-based cellular reprogramming approach, human fibroblasts from postnatal to near centenarian donors can be efficiently converted into neurons that maintain multiple age-associated signatures. Application of an epigenetic biomarker of aging (referred to as epigenetic clock) to DNA methylation data revealed that the epigenetic ages of fibroblasts were highly correlated with corresponding age estimates of reprogrammed neurons. Transcriptome and microRNA profiles reveal genes differentially expressed between young and old neurons. Further analyses of oxidative stress, DNA damage and telomere length exhibit the retention of age-associated cellular properties in converted neurons from corresponding fibroblasts. Our results collectively demonstrate the maintenance of age after neuronal conversion.", 
"Mammalian sperm are carriers of not only the paternal genome, but also the paternal epigenome in the forms of DNA methylation, retained histones and noncoding RNAs. Although paternal DNA methylation and histone retention sites have been correlated with protein-coding genes that are critical for preimplantation embryonic development, physiological evidence of an essential role of these epigenetic marks in fertilization and early development remains lacking. Two miRNA clusters consisting of five miRNAs (miR-34b/c and miR-449a/b/c) are present in sperm, but absent in oocytes, and miR-34c has been reported to be essential for the first cleavage division in vitro. Here, we show that both miR-34b/c- and miR-449-null male mice displayed normal fertility, and that intracytoplasmic injection of either miR-34b/c- or miR-449-null sperm led to normal fertilization, normal preimplantation development and normal birth rate. However, miR-34b/c and miR-449 double knockout (miR-dKO) males were infertile due to severe spermatogenic disruptions and oligo-astheno-teratozoospermia. Injection of miR-dKO sperm into wild-type oocytes led to a block at the two-pronucleus to zygote transition, whereas normal preimplantation development and healthy pups were obtained through injection of miR-dKO round spermatids. Our data demonstrate that miR-34b/c and miR-449a/b/c are essential for normal spermatogenesis and male fertility, but their presence in sperm is dispensable for fertilization and preimplantation development.", 
"Epigenetic mechanisms play a pivotal role in the expression of genes and can be influenced by both the quality and quantity of diet. Dietary compounds such as sulforaphane (SFN) found in cruciferous vegetables and epigallocatechin-3-gallate (EGCG) in green tea exhibit the ability to affect various epigenetic mechanisms such as DNA methyltransferase (DNMT) inhibition, histone modifications via histone deacetylase (HDAC), histone acetyltransferase (HAT) inhibition, or noncoding RNA expression. Regulation of these epigenetic mechanisms has been shown to have notable influences on the formation and progression of various neoplasms. We have shown that an epigenetic diet can influence both cellular longevity and carcinogenesis through the modulation of certain key genes that encode telomerase and p16. Caloric restriction (CR) can also play a crucial role in aging and cancer. Reductions in caloric intake have been shown to increase both the life- and health-span in a variety of animal models. Moreover, restriction of glucose has been demonstrated to decrease the incidence of age-related diseases such as cancer and diabetes. A diet rich in compounds such as genistein, SFN and EGCG can positively modulate the epigenome and lead to many health benefits. Also, reducing the quantity of calories and glucose in the diet can confer an increased health-span, including reduced cancer incidence.", 
"Telomerase is expressed in early human development and then becomes silenced in most normal tissues. Because similar to 90% of primary human tumors express telomerase and generally maintain very short telomeres, telomerase is carefully regulated, particularly in large, long-lived mammals. In the current report, we provide substantial evidence for a new regulatory control mechanism of the rate limiting catalytic protein component of telomerase (hTERT) that is determined by the length of telomeres. We document that normal, young human cells with long telomeres have a repressed hTERT epigenetic status (chromatin and DNA methylation), but the epigenetic status is altered when telomeres become short. The change in epigenetic status correlates with altered expression of TERT and genes near to TERT, indicating a change in chromatin. Furthermore, we identified a chromosome 5p telomere loop to a region near TERT in human cells with long telomeres that is disengaged with increased cell divisions as telomeres progressively shorten. Finally, we provide support for a role of the TRF2 protein, and possibly TERRA, in the telomere looping maintenance mechanism through interactions with interstitial TTAGGG repeats. This provides new insights into how the changes in genome structure during replicative aging result in an increased susceptibility to age-related diseases and cancer prior to the initiation of a DNA damage signal.", 
"Variation in the presence or absence of transposable elements (TEs) is a major source of genetic variation between individuals. Here, we identified 23,095 TE presence/absence variants between 216 Arabidopsis accessions. Most TE variants were rare, and we find these rare variants associated with local extremes of gene expression and DNA methylation levels within the population. Of the common alleles identified, two thirds were not in linkage disequilibrium with nearby SNPs, implicating these variants as a source of novel genetic diversity. Many common TE variants were associated with significantly altered expression of nearby genes, and a major fraction of inter-accession DNA methylation differences were associated with nearby TE insertions. Overall, this demonstrates that TE variants are a rich source of genetic diversity that likely plays an important role in facilitating epigenomic and transcriptional differences between individuals, and indicates a strong genetic basis for epigenetic variation.", 
"Social insects are promising model systems for epigenetics due to their immense morphological and behavioral plasticity. Reports that DNA methylation differs between the queen and worker castes in social insects [1-4] have implied a role for DNA methylation in regulating division of labor. To better understand the function of DNA methylation in social insects, we performed whole-genome bisulfite sequencing on brains of the clonal raider ant Cerapachys biroi, whose colonies alternate between reproductive (queen-like) and brood care (workerlike) phases [5]. Many cytosines were methylated in all replicates (on average 29.5% of the methylated cytosines in a given replicate), indicating that a large proportion of the C. biroi brain methylome is robust. Robust DNA methylation occurred preferentially in exonic CpGs of highly and stably expressed genes involved in core functions. Our analyses did not detect any differences in DNA methylation between the queen-like and worker-like phases, suggesting that DNA methylation is not associated with changes in reproduction and behavior in C. biroi. Finally, many cytosines were methylated in one sample only, due to either biological or experimental variation. By applying the statistical methods used in previous studies [1-4, 6] to our data, we show that such sample-specific DNA methylation may underlie the previous findings of queen-and worker-specific methylation. We argue that there is currently no evidence that genome-wide variation in DNA methylation is associated with the queen and worker castes in social insects, and we call for a more careful interpretation of the available data.", 
"Environmental stresses experienced by individual parents can influence offspring phenotypes in ways that enhance survival under similar conditions. Although such adaptive transgenerational plasticity is well documented, its transmission mechanisms are generally unknown. One possible mechanism is environmentally induced DNA methylation changes. We tested this hypothesis in the annual plant Polygonum persicaria, a species known to express adaptive transgenerational plasticity in response to parental drought stress. Replicate plants of 12 genetic lines (sampled from natural populations) were grown in dry versus moist soil. Their offspring were exposed to the demethylating agent zebularine or to control conditions during germination and then grown in dry soil. Under control germination conditions, the offspring of drought-stressed parents grew longer root systems and attained greater biomass compared with offspring of well-watered parents of the same genetic lines. Demethylation removed these adaptive developmental effects of parental drought, but did not significantly alter phenotypic expression in offspring of well-watered parents. The effect of demethylation on the expression of the parental drought effect varied among genetic lines. Differential seed provisioning did not contribute to the effect of parental drought on offspring phenotypes. These results demonstrate that DNA methylation can mediate adaptive, genotype-specific effects of parental stress on offspring phenotypes.", 
"Since the detection of cell-free DNA (cfDNA) in human plasma in 1948, it has been investigated as a non-invasive screening tool for many diseases, especially solid tumours and foetal genetic abnormalities. However, to date our lack of knowledge regarding the origin and purpose of cfDNA in a physiological environment has limited its use to more obvious diagnostics, neglecting, for example, its potential utility in the identification of predisposition to disease, earlier detection of cancers, and lifestyle-induced epigenetic changes. Moreover, the concept or mechanism of cfDNA could also have potential therapeutic uses such as in immuno- or gene therapy. This review presents an extensive compilation of the putative origins of cfDNA and then contrasts the contributions of cellular breakdown processes with active mechanisms for the release of cfDNA into the extracellular environment. The involvement of cfDNA derived from both cellular breakdown and active release in lateral information transfer is also discussed. We hope to encourage researchers to adopt a more holistic view of cfDNA research, taking into account all the biological pathways in which cfDNA is involved, and to give serious consideration to the integration of in vitro and in vivo research. We also wish to encourage researchers not to limit their focus to the apoptotic or necrotic fraction of cfDNA, but to investigate the intercellular messaging capabilities of the actively released fraction of cfDNA and to study the role of cfDNA in pathogenesis.", 
"The role of natural selection in the evolution of adaptive phenotypes has undergone constant probing by evolutionary biologists, employing both theoretical and empirical approaches. As Darwin noted, natural selection can act together with other processes, including random changes in the frequencies of phenotypic differences that are not under strong selection, and changes in the environment, which may reflect evolutionary changes in the organisms themselves. As understanding of genetics developed after 1900, the new genetic discoveries were incorporated into evolutionary biology. The resulting general principles were summarized by Julian Huxley in his 1942 book Evolution: the modern synthesis. Here, we examine how recent advances in genetics, developmental biology and molecular biology, including epigenetics, relate to today's understanding of the evolution of adaptations. We illustrate how careful genetic studies have repeatedly shown that apparently puzzling results in a wide diversity of organisms involve processes that are consistent with neo-Darwinism. They do not support important roles in adaptation for processes such as directed mutation or the inheritance of acquired characters, and therefore no radical revision of our understanding of the mechanism of adaptive evolution is needed.", 
"Genomic imprinting is an epigenetic phenomenon causing parent-of-origin specific differential expression of maternally and paternally inherited alleles. While many imprinted genes have been identified in plants, the functional roles of most of them are unknown. In this study, we systematically examine the functional requirement of paternally expressed imprinted genes (PEGs) during seed development in Arabidopsis thaliana. While none of the 15 analyzed peg mutants has qualitative or quantitative abnormalities of seed development, we identify three PEGs that establish postzygotic hybridization barriers in the endosperm, revealing that PEGs have a major role as speciation genes in plants. Our work reveals that a subset of PEGs maintains functional roles in the inbreeding plant Arabidopsis that become evident upon deregulated expression.", 
"LINE-1 (L1) retrotransposons represent approximately one sixth of the human genome, but only the human-specific L1HS-Ta subfamily acts as an endogenous mutagen in modern humans, reshaping both somatic and germline genomes. Due to their high levels of sequence identity and the existence of many polymorphic insertions absent from the reference genome, the transcriptional activation of individual genomic L1HS-Ta copies remains poorly understood. Here we comprehensively mapped fixed and polymorphic L1HS-Ta copies in 12 commonly-used somatic cell lines, and identified transcriptional and epigenetic signatures allowing the unambiguous identification of active L1HS-Ta copies in their genomic context. Strikingly, only a very restricted subset of L1HS-Ta loci - some being polymorphic among individuals - significantly contributes to the bulk of L1 expression, and these loci are differentially regulated among distinct cell lines. Thus, our data support a local model of L1 transcriptional activation in somatic cells, governed by individual-, locus-, and cell-type-specific determinants.", 
"Two parallel pathways produce cholesterol: the Bloch and Kandutsch-Russell pathways. Here we used stable isotope labeling and isotopomer analysis to trace sterol flux through the two pathways in mice. Surprisingly, no tissue used the canonical K-R pathway. Rather, a hybrid pathway was identified that we call the modified K-R (MK-R) pathway. Proportional flux through the Bloch pathway varied from 8% in preputial gland to 97% in testes, and the tissue-specificity observed in vivo was retained in cultured cells. The distribution of sterol isotopomers in plasma mirrored that of liver. Sterol depletion in cultured cells increased flux through the Bloch pathway, whereas overexpression of 24-dehydrocholesterol reductase (DHCR24) enhanced usage of the MK-R pathway. Thus, relative use of the Bloch and MK-R pathways is highly variable, tissue-specific, flux dependent, and epigenetically fixed. Maintenance of two interdigitated pathways permits production of diverse bioactive sterols that can be regulated independently of cholesterol.", 
"The number of children born since the origin of Assisted Reproductive Technologies (ART) exceeds 5 million. The majority seem healthy, but a higher frequency of defects has been reported among ART-conceived infants, suggesting an epigenetic cost. We report the first wholegenome DNA methylation datasets from single pig blastocysts showing differences between in vivo and in vitro produced embryos. Blastocysts were produced in vitro either without (C-IVF) or in the presence of natural reproductive fluids (Natur-IVF). Natur-IVF embryos were of higher quality than C-IVF in terms of cell number and hatching ability. RNA-Seq and DNA methylation analyses showed that Natur-IVF embryos have expression and methylation patterns closer to in vivo blastocysts. Genes involved in reprogramming, imprinting and development were affected by culture, with fewer aberrations in Natur-IVF embryos. Methylation analysis detected methylated changes in C-IVF, but not in Natur-IVF, at genes whose methylation could be critical, such as IGF2R and NNAT.", 
"Upon fertilization, the highly specialised sperm and oocyte genomes are remodelled to confer totipotency. The mechanisms of the dramatic reprogramming events that occur have remained unknown, and presumed roles of histone modifying enzymes are just starting to be elucidated. Here, we explore the function of the oocyte-inherited pool of a histone H3K4 and K9 demethylase, LSD1/KDM1A during early mouse development. KDM1A deficiency results in developmental arrest by the two-cell stage, accompanied by dramatic and stepwise alterations in H3K9 and H3K4 methylation patterns. At the transcriptional level, the switch of the maternal-to-zygotic transition fails to be induced properly and LINE-1 retrotransposons are not properly silenced. We propose that KDM1A plays critical roles in establishing the correct epigenetic landscape of the zygote upon fertilization, in preserving genome integrity and in initiating new patterns of genome expression that drive early mouse development.", 
"DNA methylation regulates many cellular processes, including embryonic development, transcription, chromatin structure, X-chromosome inactivation, genomic imprinting and chromosome stability. DNA methyltransferases establish and maintain the presence of 5-methylcytosine (5mC), and ten-eleven translocation cytosine dioxygenases (TETs) oxidise 5mC to 5-hydroxymethylcytosine (5hmC), 5-formylcytosine (5fC) and 5-carboxylcytosine (5caC), which can be removed by base excision repair (BER) proteins. Multiple forms of DNA methylation are recognised by methyl-CpG binding proteins (MeCPs), which play vital roles in chromatin-based transcriptional regulation, DNA repair and replication. Accordingly, defects in DNA methylation and its mediators may cause silencing of tumour suppressor genes and misregulation of multiple cell cycles, DNA repair and chromosome stability genes, and hence contribute to genome instability in various human diseases, including cancer. Thus, understanding functional genetic mutations and aberrant expression of these DNA methylation mediators is critical to deciphering the crosstalk between concurrent genetic and epigenetic alterations in specific cancer types and to the development of new therapeutic strategies.", 
"Ten-eleven translocation (TET) enzymes oxidize 5-methylcytosine, facilitating DNA demethylation and generating new epigenetic marks. Here we show that concomitant loss of Tet2 and Tet3 in mice at early B cell stage blocked the pro- to pre-B cell transition in the bone marrow, decreased Irf4 expression and impaired the germline transcription and rearrangement of the Ig kappa locus. Tet2/3-deficient pro-B cells showed increased CpG methylation at the Ig kappa 3' and distal enhancers that was mimicked by depletion of E2A or PU.1, as well as a global decrease in chromatin accessibility at enhancers. Importantly, re-expression of the Tet2 catalytic domain in Tet2/3-deficient B cells resulted in demethylation of the Igx enhancers and restored their chromatin accessibility. Our data suggest that TET proteins and lineage-specific transcription factors cooperate to influence chromatin accessibility and Igx enhancer function by modulating the modification status of DNA.", 
"Group-living animals must adjust the expression of their social behaviour to changes in their social environment and to transitions between life-history stages, and this social plasticity can be seen as an adaptive trait that can be under positive selection when changes in the environment outpace the rate of genetic evolutionary change. Here, we propose a conceptual framework for understanding the neuromolecular mechanisms of social plasticity. According to this framework, social plasticity is achieved by rewiring or by biochemically switching nodes of a neural network underlying social behaviour in response to perceived social information. Therefore, at the molecular level, it depends on the social regulation of gene expression, so that different genomic and epigenetic states of this brain network correspond to different behavioural states, and the switches between states are orchestrated by signalling pathways that interface the social environment and the genotype. Different types of social plasticity can be recognized based on the observed patterns of inter- versus intra-individual occurrence, time scale and reversibility. It is proposed that these different types of social plasticity rely on different proximate mechanisms at the physiological, neural and genomic level.", 
"While N-6-methyladenosine (m(6)A) is a well-known epigenetic modification inbacterial DNA, it remained largely unstudied in eukaryotes. Recent studies have brought to fore its potential epigenetic role across diverse eukaryotes with biological consequences, which are distinct and possibly even opposite to the well-studied 5-methylcytosine mark. Adenine methyltransferases appear to have been independently acquired by eukaryotes on at least 13 occasions from prokaryotic restriction-modification and counter-restriction systems. On at least four to five instances, these methyltransferases were recruited as RNA methylases. Thus, m(6)A marks in eukaryotic DNA and RNA might be more wide spread and diversified than previously believed. Several m(6)A-binding protein domains from prokaryotes were also acquired by eukaryotes, facilitating prediction of potential readers for these marks. Further, multiple lineages of the AlkB family of dioxygenases have been recruited as m(6)A demethylases. Although members of the TET/JBP family of dioxygenases have also been suggested to be m(6)A demethylases, this proposal needs more careful evaluation.", 
"N-6-methyladenosine (m(6)A) is a vital post-transcriptional modification, which adds another layer of epigenetic regulation at RNA level. It chemically modifies mRNA that effects protein expression. RNA sequence contains many genetic code motifs (GAC). Among these codes, identification of methylated or not methylated GAC motif is highly indispensable. However, with a large number of RNA sequences generated in post-genomic era, it becomes a challenging task how to accurately and speedily characterize these sequences. In view of this, the concept of an intelligent is incorporated with a computational model that truly and fast reflects the motif of the desired classes. An intelligent computational model iMethyl-STTNC model is proposed for identification of methyladenosine sites in RNA. In the proposed study, four feature extraction techniques, such as; Pseudo-dinucleotide-composition, Pseudo-trinucleotide-composition, split-trinucleotide-composition, and split-tetra-nucleotides-composition (STTNC) are utilized for genuine numerical descriptors. Three different classification algorithms including probabilistic neural network, Support vector machine (SVM), and K-nearest neighbor are adopted for prediction. After examining the outcomes of prediction model on each feature spaces, SVM using STTNC feature space reported the highest accuracy of 69.84%, 91.84% on dataset1 and dataset2, respectively. The reported results show that our proposed predictor has achieved encouraging results compared to the present approaches, so far in the research. It is finally reckoned that our developed model might be beneficial for in-depth analysis of genomes and drug development. (C) 2018 Elsevier Ltd. All rights reserved.", 
"The epigenetic inheritance of DNA methylation requires UHRF1, a histone- and DNA-binding RING E3 ubiquitin ligase that recruits DNMT1 to sites of newly replicated DNA through ubiquitylation of histone H3. UHRF1 binds DNA with selectivity towards hemi-methylated CpGs (HeDNA); however, the contribution of HeDNA sensing to UHRF1 function remains elusive. Here, we reveal that the interaction of UHRF1 with HeDNA is required for DNA methylation but is dispensable for chromatin interaction, which is governed by reciprocal positive cooperativity between the UHRF1 histone- and DNA-binding domains. HeDNA recognition activates UHRF1 ubiquitylation towards multiple lysines on the H3 tail adjacent to the UHRF1 histone-binding site. Collectively, our studies are the first demonstrations of a DNA-protein interaction and an epigenetic modification directly regulating E3 ubiquitin ligase activity. They also define an orchestrated epigenetic control mechanism involving modifications both to histones and DNA that facilitate UHRF1 chromatin targeting, H3 ubiquitylation, and DNA methylation inheritance.", 
"Inheritance of gene expression states is fundamental for cells to 'remember' past events, such as environmental or developmental cues. The conserved Polycomb Repressive Complex 2 (PRC2) maintains epigenetic repression of many genes in animals and plants and modifies chromatin at its targets. Histones modified by PRC2 can be inherited through cell division. However, it remains unclear whether this inheritance can direct long-term memory of individual gene expression states (cis memory) or instead if local chromatin states are dictated by the concentrations of diffusible factors (trans memory). By monitoring the expression of two copies of the Arabidopsis Polycomb target gene FLOWERING LOCUS C (FLC) in the same plants, we show that one copy can be repressed while the other is active. Furthermore, this 'mixed' expression state is inherited through many cell divisions as plants develop. These data demonstrate that epigenetic memory of FLC expression is stored not in trans but in cis.", 
"Loss of polarity correlates with progression of epithelial cancers, but how plasma membrane misorganization drives oncogenic transcriptional events remains unclear. The polarity regulators of the Drosophila Scribble (Scrib) module are potent tumor suppressors and provide a model to investigate these mechanisms. RNA profiling of Scrib module mutant tumors reveals multiple signatures of neoplasia, including altered metabolism and dedifferentiation. A prominent gene expression change is upregulation of the cytokine-like Unpaired (Upd) ligands, which drive tumor overgrowth. We identified a polarity-responsive enhancer in upd3, which is activated in a coincident manner by both JNK-dependent Fos and aPKC-mediated Yki transcription. This enhancer, and Scrib module mutant overgrowth in general, are also sensitive to activity of the Polycomb Group (PcG), suggesting that PcG attenuation upon polarity loss potentiates select targets for activation by JNK and Yki. Our results link epithelial organization to signaling and epigenetic regulators that control tissue repair programs, and provide insight into why epithelial polarity is tumor-suppressive.", 
"One of the most striking patterns of genome structure is the tight, typically negative, association between transposable elements (TEs) and meiotic recombination rates. While this is a highly recurring feature of eukaryotic genomes, the mechanisms driving correlations between TEs and recombination remain poorly understood, and distinguishing cause versus effect is challenging. Here, we review the evidence for a relation between TEs and recombination, and discuss the underlying evolutionary forces. Evidence to date suggests that overall TE densities correlate negatively with recombination, but the strength of this correlation varies across element types, and the pattern can be reversed. Results suggest that heterogeneity in the strength of selection against ectopic recombination and gene disruption can drive TE accumulation in regions of low recombination, but there is also strong evidence that the regulation of TEs can influence local recombination rates. We hypothesize that TE insertion polymorphism may be important in driving within-species variation in recombination rates in surrounding genomic regions. Furthermore, the interaction between TEs and recombination may create positive feedback, whereby TE accumulation in non-recombining regions contributes to the spread of recombination suppression. Further investigation of the coevolution between recombination and TEs has important implications for our understanding of the evolution of recombination rates and genome structure. This article is part of the themed issue 'Evolutionary causes and consequences of recombination rate variation in sexual organisms'.", 
"Organisms can change their physiological/behavioural traits to adapt and survive in changed environments. However, whether these acquired traits can be inherited across generations through non-genetic alterations has been a topic of debate for over a century. Emerging evidence indicates that both ancestral and parental experiences, including nutrition, environmental toxins, nurturing behaviour, and social stress, can have powerful effects on the physiological, metabolic and cellular functions in an organism. In certain circumstances, these effects can be transmitted across several generations through epigenetic (i.e. non-DNA sequence-based rather than mutational) modifications. In this review, we summarize recent evidence on epigenetic inheritance from parental environment-induced developmental and physiological alterations in nematodes, fruit flies, zebrafish, rodents, and humans. The epigenetic modifications demonstrated to be both susceptible to modulation by environmental cues and heritable, including DNA methylation, histone modification, and small non-coding RNAs, are also summarized. We particularly focus on evidence that parental environment-induced epigenetic alterations are transmitted through both the maternal and paternal germlines and exert sex-specific effects. The thought-provoking data presented here raise fundamental questions about the mechanisms responsible for these phenomena. In particular, the means that define the specificity of the response to parental experience in the gamete epigenome and that direct the establishment of the specific epigenetic change in the developing embryos, as well as in specific tissues in the descendants, remain obscure and require elucidation. More precise epigenetic assessment at both the genome-wide level and single-cell resolution as well as strategies for breeding at relatively sensitive periods of development and manipulation aimed at specific epigenetic modification are imperative for identifying parental environment-induced epigenetic marks across generations. Considering their diverse epigenetic architectures, the conservation and prevalence of the mechanisms underlying epigenetic inheritance in non-mammals require further investigation in mammals. Interpretation of the consequences arising from epigenetic inheritance on organisms and a better understanding of the underlying mechanisms will provide insight into how gene-environment interactions shape developmental processes and physiological functions, which in turn may have wide-ranging implications for human health, and understanding biological adaptation and evolution.", 
"In yeast and humans, previous experiences can lead to epigenetic transcriptional memory: repressed genes that exhibit mitotically heritable changes in chromatin structure and promoter recruitment of poised RNA polymerase II preinitiation complex (RNAPII PIC), which enhances future reactivation. Here, we show that INO1 memory in yeast is initiated by binding of the Sfl1 transcription factor to the cis-acting Memory Recruitment Sequence, targeting INO1 to the nuclear periphery. Memory requires a remodeled form of the Set1/COMPASS methyltransferase lacking Spp1, which dimethylates histone H3 lysine 4 (H3K4me2). H3K4me2 recruits the SET3C complex, which plays an essential role in maintaining this mark. Finally, while active INO1 is associated with Cdk8(-) Mediator, during memory, Cdk8(+) Mediator recruits poised RNAPII PIC lacking the Kin28 CTD kinase. Aspects of this mechanism are generalizable to yeast and conserved in human cells. Thus, COMPASS and Mediator are repurposed to promote epigenetic transcriptional poising by a highly conserved mechanism.", 
"The advent of in vitro fertilization (IVF) in animals and humans implies an extraordinary change in the environment where the beginning of a new organism takes place. In mammals fertilization occurs in the maternal oviduct, where there are unique conditions for guaranteeing the encounter of the gametes and the first stages of development of the embryo and thus its future. During this period a major epigenetic reprogramming takes place that is crucial for the normal fate of the embryo. This epigenetic reprogramming is very vulnerable to changes in environmental conditions such as the ones implied in IVF, including in vitro culture, nutrition, light, temperature, oxygen tension, embryo-maternal signaling, and the general absence of protection against foreign elements that could affect the stability of this process. The objective of this review is to update the impact of the various conditions inherent in the use of IVF on the epigenetic profile and outcomes of mammalian embryos, including superovulation, IVF technique, embryo culture and manipulation and absence of embryo-maternal signaling. It also covers the possible transgenerational inheritance of the epigenetic alterations associated with assisted reproductive technologies (ART), including its phenotypic consequences as is in the case of the large offspring syndrome (LOS). Finally, the important scientific and bioethical implications of the results found in animals are discussed in terms of the ART in humans.", 
"Facioscapulohumeral muscular dystrophy (FSHD) is a muscular dystrophy caused by inefficient epigenetic repression of the D4Z4 macrosatellite array and somatic expression of the DUX4 retrogene. DUX4 is a double homeobox transcription factor that is normally expressed in the testis and causes apoptosis and FSHD when mis-expressed in skeletal muscle. The mechanism(s) of DUX4 toxicity in muscle is incompletely understood. We report that DUX4-triggered proteolytic degradation of UPF1, a central component of the nonsense-mediated decay (NMD) machinery, is associated with profound NMD inhibition, resulting in global accumulation of RNAs normally degraded as NMD substrates. DUX4 mRNA is itself degraded by NMD, such that inhibition of NMD by DUX4 protein stabilizes DUX4 mRNA through a double-negative feedback loop in FSHD muscle cells. This feedback loop illustrates an unexpected mode of autoregulatory behavior of a transcription factor, is consistent with bursts of DUX4 expression in FSHD muscle, and has implications for FSHD pathogenesis.", 
"Long noncoding RNAs belong to a class of noncoding RNAs longer than 200 nucleotides with the epigenetic regulation potential. As a novel molecular regulator, IncRNAs are often dysregulated in various pathological conditions and display multiple functions in a wide range of biological processes. Given that recent studies have indicated that IncRNAs are involved in atherosclerosis-related smooth muscle cell, endothelial cell, macrophage and lipid metabolism regulation, it is pertinent to understand the potential function of IncRNAs in atherosclerosis development. This review will highlight the recent updates of IncRNAs in atherogenesis and also discuss their potential roles as novel therapeutic targets.", 
"The H19 gene produces a non-coding RNA, which is abundantly expressed during embryonic development and down-regulated after birth. Although this gene was discovered over 20 years ago, its function has remained unclear. Only recently a role was identified for the non-coding RNA and/or its microRNA partner, first as a tumour suppressor gene in mice, then as a trans-regulator of a group of co-expressed genes belonging to the imprinted gene network that is likely to control foetal and early postnatal growth in mice. The mechanisms underlying this transcriptional or post-transcriptional regulation remain to be discovered, perhaps by identifying the protein partners of the full-ength H19 RNA or the targets of the microRNA. This first in vivo evidence of a functional role for the H19 locus provides new insights into how genomic imprinting helps to control embryonic growth.", 
"Obesity is highly prevalent, and its incidence is increasing. The previous study showing a major effect of paternal obesity on metabolic health of offspring is confounded by comorbidity with diabetes. Therefore, we investigated the effect of diet-induced paternal obesity, in the absence of diabetes, on the metabolic health of two resultant generations and the molecular profiles of the testes and sperm. Founder (F-0) male C57BL6 mice were fed either a high-fat diet (HFD) or a control diet (CD); n = 10/diet for a period of 10 wk. Testis expression of mRNA/microRNAs was analyzed by microarray and qPCR and sperm microRNA abundance by qPCR. Two subsequent generations were generated by mating F-0 and then F-1 mice to CD mice, and their metabolic health was investigated. All mice, other than F-0 males, were maintained on a CD. HFD feeding induced paternal obesity with a 21% increase in adiposity, but not overt diabetes, and initiated intergenerational transmission of obesity and insulin resistance in two generations of offspring. This distinct phenotypic constellation is either partially or fully transmitted to both female and male F-1 offspring and further transmitted through both parental lineages to the F-2 generation, with a heightened effect on female F-1 offspring (+67% in adiposity) and their F-2 sons (+24% in adiposity). Founder male obesity altered the testes expression of 414 mRNAs by microarray and 11 microRNAs by qPCR, concomitant with alterations in sperm microRNA content and a 25% reduction in global methylation of germ cell DNA. Diet-induced paternal obesity modulates sperm microRNA content and germ cell methylation status, which are potential signals that program offspring health and initiate the transmission of obesity and impaired metabolic health to future generations. This study implicates paternal obesity in the transgenerational amplification of obesity and type 2 diabetes in humans.Fullston, T., Ohlsson Teague, E. M. C., Palmer, N. O., DeBlasio, M. J., Mitchell, M., Corbett, M., Print, C. G., Owens, J. A., Lane, M. Paternal obesity initiates metabolic disturbances in two generations of mice with incomplete penetrance to the F-2 generation and alters the transcriptional profile of testis and sperm microRNA content.", 
"CpG islands (CGIs) are associated with most mammalian gene promoters. A subset of CGIs act as polycomb response elements (PREs) and are recognized by the polycomb silencing systems to regulate expression of genes involved in early development. How CGIs function mechanistically as nucleation sites for polycomb repressive complexes remains unknown. Here we discover that KDM2B (FBXL10) specifically recognizes non-methylated DNA in CGIs and recruits the polycomb repressive complex 1 (PRC1). This contributes to histone H2A lysine 119 ubiquitylation (H2AK119ub1) and gene repression. Unexpectedly, we also find that CGIs are occupied by low levels of PRC1 throughout the genome, suggesting that the KDM2B-PRC1 complex may sample CGI-associated genes for susceptibility to polycomb-mediated silencing. These observations demonstrate an unexpected and direct link between recognition of CGIs by KDM2B and targeting of the polycomb repressive system. This provides the basis for a new model describing the functionality of CGIs as mammalian PREs.", 
"In mammalian development, epigenetic modifications, including DNA methylation patterns, play a crucial role in defining cell fate but also represent epigenetic barriers that restrict developmental potential. At two points in the life cycle, DNA methylation marks are reprogrammed on a global scale, concomitant with restoration of developmental potency. DNA methylation patterns are subsequently re-established with the commitment towards a distinct cell fate. This reprogramming of DNA methylation takes place firstly on fertilization in the zygote, and secondly in primordial germ cells (PGCs), which are the direct progenitors of sperm or oocyte. In each reprogramming window, a unique set of mechanisms regulates DNA methylation erasure and re-establishment. Recent advances have uncovered roles for the TET3 hydroxylase and passive demethylation, together with base excision repair (BER) and the elongator complex, in methylation erasure from the zygote. Deamination by AID, BER and passive demethylation have been implicated in reprogramming in PGCs, but the process in its entirety is still poorly understood. In this review, we discuss the dynamics of DNA methylation reprogramming in PGCs and the zygote, the mechanisms involved and the biological significance of these events. Advances in our understanding of such natural epigenetic reprogramming are beginning to aid enhancement of experimental reprogramming in which the role of potential mechanisms can be investigated in vitro. Conversely, insights into in vitro reprogramming techniques may aid our understanding of epigenetic reprogramming in the germline and supply important clues in reprogramming for therapies in regenerative medicine.", 
"DNA methylation is an essential epigenetic mark whose role in gene regulation and its dependency on genomic sequence and environment are not fully understood. In this study we provide novel insights into the mechanistic relationships between genetic variation, DNA methylation and transcriptome sequencing data in three different cell-types of the GenCord human population cohort. We find that the association between DNA methylation and gene expression variation among individuals are likely due to different mechanisms from those establishing methylation-expression patterns during differentiation. Furthermore, cell-type differential DNA methylation may delineate a platform in which local inter-individual changes may respond to or act in gene regulation. We show that unlike genetic regulatory variation, DNA methylation alone does not significantly drive allele specific expression. Finally, inferred mechanistic relationships using genetic variation as well as correlations with TF abundance reveal both a passive and active role of DNA methylation to regulatory interactions influencing gene expression.", 
"The prospect of finding epigenetic risk factors for complex diseases would be greatly enhanced if DNA from existing biobanks, which is generally extracted from whole blood, could be used to perform epigenetic association studies. We characterized features of DNA methylation at 16 candidate loci, 8 of which were imprinted, in DNA samples from the Netherlands Twin Register biobank. Except for un-methylated or fully methylated sites, CpG methylation varied considerably in a sample of 30 unrelated individuals. This variation remained after accounting for the cellular heterogeneity of blood. Methylation of CpG sites was correlated within loci and, for 4 imprinted loci, across chromosomes. In 34 additional individuals, we investigated the DNA methylation of 8 representative loci in 2 longitudinal blood and 2 longitudinal buccal cell samples (follow-up 11-20 and 2-8 yr, respectively). Five of 8 loci were stable over time (rho > 0.75) in both tissues, indicating that prospective epigenetic studies may be possible. For 4 loci, the DNA methylation in blood (mesoderm) correlated with that in the buccal cells (ectoderm) (rho > 0.75). Our data suggest that epigenetic studies on complex diseases may be feasible for a proportion of genomic loci provided that they are carefully designed.-Talens, R. P., Boomsma, D. I., Tobi, E. W., Kremer, D., Jukema, J. W., Willemsen, G., Putter, H., Slagboom, P. E., Heijmans, B. T. Variation, patterns, and temporal stability of DNA methylation: considerations for epigenetic epidemiology. FASEB J. 24, 3135-3144 (2010). www.fasebj.org", 
"Autism is currently considered a multigene disorder with epigenetic influences. To investigate the contribution of DNA methylation to autism spectrum disorders, we have recently completed large-scale methylation profiling by CpG island microarray analysis of lymphoblastoid cell lines derived from monozygotic twins discordant for diagnosis of autism and their nonautistic siblings. Methylation profiling revealed many candidate genes differentially methylated between discordant MZ twins as well as between both twins and nonautistic siblings. Bioinformatics analysis of the differentially methylated genes demonstrated enrichment for high-level functions including gene transcription, nervous system development, cell death/survival, and other biological processes implicated in autism. The methylation status of 2 of these candidate genes, BCL-2 and retinoic acid-related orphan receptor alpha (RORA), was further confirmed by bisulfite sequencing and methylation-specific PCR, respectively. Immunohistochemical analyses of tissue arrays containing slices of the cerebellum and frontal cortex of autistic and age-and sex-matched control subjects revealed decreased expression of RORA and BCL-2 proteins in the autistic brain. Our data thus confirm the role of epigenetic regulation of gene expression via differential DNA methylation in idiopathic autism, and furthermore link molecular changes in a peripheral cell model with brain pathobiology in autism.-Nguyen, A., Rauch, T. A., Pfeifer, G. P., Hu, V. W. Global methylation profiling of lymphoblastoid cell lines reveals epigenetic contributions to autism spectrum disorders and a novel autism candidate gene, RORA, whose protein product is reduced in autistic brain. FASEB J. 24, 3036-3051 (2010). www.fasebj.org", 
"The development of multicellular organisms is controlled by differential gene expression whereby cells adopt distinct fates. A spatially resolved view of gene expression allows the elucidation of transcriptional networks that are linked to cellular identity and function. The haploid female gametophyte of flowering plants is a highly reduced organism: at maturity, it often consists of as few as three cell types derived from a common precursor [1, 2]. However, because of its inaccessibility and small size, we know little about the molecular basis of cell specification and differentiation in the female gametophyte. Here we report expression profiles of all cell types in the mature Arabidopsis female gametophyte. Differentially expressed posttranscriptional regulatory modules and metabolic pathways characterize the distinct cell types. Several transcription factor families are overrepresented in the female gametophyte in comparison to other plant tissues, e.g., type I MADS domain, RWP-RK, and reproductive meristem transcription factors. PAZ/Piwi-domain encoding genes are upregulated in the egg, indicating a role of epigenetic regulation through small RNA pathways a feature paralleled in the germline of animals [3]. A comparison of human and Arabidopsis egg cells for enrichment of functional groups identified several similarities that may represent a consequence of coevolution or ancestral gametic features.", 
"Background: Ant societies comprise individuals belonging to different castes characterized by specialized morphologies and behaviors. Because ant embryos can follow different developmental trajectories, epigenetic mechanisms must play a role in caste determination. Ants have a full set of DNA methyltransferases and their genomes contain methylcytosine. To determine the relationship between DNA methylation and phenotypic plasticity in ants, we obtained and compared the genome-wide methylonnes of different castes and developmental stages of Camponotus floridanus and Harpegnathos saltator. Results: In the ant genomes, methylcytosines are found both in symmetric CG dinucleotides (CpG) and non-CpG contexts and are strongly enriched at exons of active genes. Changes in exonic DNA methylation correlate with alternative splicing events such as exon skipping and alternative splice site selection. Several genes exhibit caste-specific and developmental changes in DNA methylation that are conserved between the two species, including genes involved in reproduction, telomere maintenance, and noncoding RNA metabolism. Several loci are methylated and expressed monoallelically, and in some cases, the choice of methylated allele depends on the caste. Conclusions: These first ant methylomes and their intra- and interspecies comparison reveal an exonic methylation pattern that points to a connection between DNA methylation and splicing. The presence of monoallelic DNA methylation and the methylation of non-CpG sites in all samples suggest roles in genome regulation in these social insects, including the intriguing possibility of parental or caste-specific genomic imprinting.", 
"Polyphenism is the phenomenon where two or more distinct phenotypes are produced by the same genotype. Examples of polyphenism provide some of the most compelling systems for the study of epigenetics. Polyphenisms are a major reason for the success of the insects, allowing them to partition life history stages (with larvae dedicated to feeding and growth, and adults dedicated to reproduction and dispersal), to adopt different phenotypes that best suit predictable environmental changes (seasonal morphs), to cope with temporally heterogeneous environments (dispersal morphs), and to partition labour within social groups (the castes of eusocial insects). We survey the status of research on some of the best known examples of insect polyphenism, in each case considering the environmental cues that trigger shifts in phenotype, the neurochemical and hormonal pathways that mediate the transformation, the molecular genetic and epigenetic mechanisms involved in initiating and maintaining the polyphenism, and the adaptive and life-history significance of the phenomenon. We conclude by highlighting some of the common features of these examples and consider future avenues for research on polyphenism.", 
"New DNA sequencing technologies have provided novel insights into eukaryotic genomes, epigenomes, and the transcriptome, including the identification of new non-coding RNA (ncRNA) classes such as promoter-associated RNAs and long RNAs. Moreover, it is now clear that up to 90% of eukaryotic genomes are transcribed, generating an extraordinary range of RNAs with no coding capacity. Taken together, these new discoveries are modifying the status quo in genomic science by demonstrating that the eukaryotic gene pool is divided into two distinct categories of transcripts: protein-coding and non-coding. The function of the majority of ncRNAs produced by the transcriptome is largely unknown; however, it is probable that many are associated with epigenetic mechanisms. The purpose of this review is to describe the most recent discoveries in the ncRNA field that implicate these molecules as key players in the epigenome.", 
"Literature on maternal exposures and the risk of epigenetic changes or diseases in the offspring is growing. Paternal contributions are often not considered. However, some animal and epidemiologic studies on various contaminants, nutrition, and lifestyle-related conditions suggest a paternal influence on the offspring's future health. The phenotypic outcomes may have been attributed to DNA damage or mutations, but increasing evidence shows that the inheritance of environmentally induced functional changes of the genome, and related disorders, are (also) driven by epigenetic components. In this essay we suggest the existence of epigenetic windows of susceptibility to environmental insults during sperm development. Changes in DNA methylation, histone modification, and non-coding RNAs are viable mechanistic candidates for a non-genetic transfer of paternal environmental information, from maturing germ cell to zygote. Inclusion of paternal factors in future research will ultimately improve the understanding of transgenerational epigenetic plasticity and health-related effects in future generations.", 
"DNA methylation, especially CpG methylation at promoter regions, has been generally considered as a potent epigenetic modification that prohibits transcription factor (TF) recruitment, resulting in transcription suppression. Here, we used a protein microarray-based approach to systematically survey the entire human TF family and found numerous purified TFs with methylated CpG (mCpG)-dependent DNA-binding activities. Interestingly, some TFs exhibit specific binding activity to methylated and unmethylated DNA motifs of distinct sequences. To elucidate the underlying mechanism, we focused on Kruppel-like factor 4 (KLF4), and decoupled its mCpG- and CpG-binding activities via site-directed mutagenesis. Furthermore, KLF4 binds specific methylated or unmethylated motifs in human embryonic stem cells in vivo. Our study suggests that mCpG-dependent TF binding activity is a widespread phenomenon and provides a new framework to understand the role and mechanism of TFs in epigenetic regulation of gene transcription.", 
"Genetically identical cells grown in the same culture display striking cell-to-cell heterogeneity in gene expression and other traits. A crucial challenge is to understand how much of this heterogeneity reflects the noise tolerance of a robust system and how much serves a biological function. In bacteria, stochastic gene expression results in cell-to-cell heterogeneity that might serve as a bet-hedging mechanism, allowing a few cells to survive through an antimicrobial treatment while others perish. Despite its clinical importance, the molecular mechanisms underlying bet hedging remain unclear. Here, we investigate the mechanisms of bet hedging in Saccharomyces cerevisiae using a new high-throughput microscopy assay that monitors variable protein expression, morphology, growth rate, and survival outcomes of tens of thousands of yeast microcolonies simultaneously. We find that clonal populations display broad distributions of growth rates and that slow growth predicts resistance to heat killing in a probabalistic manner. We identify several gene products that are likely to play a role in bet hedging and confirm that Tsl1, a trehalose-synthesis regulator, is an important component of this resistance. Tsl1 abundance correlates with growth rate and replicative age and predicts survival. Our results suggest that yeast bet hedging results from multiple epigenetic growth states determined by a combination of stochastic and deterministic factors.", 
"Epigenetics, the transgenerational transfer of phenotypic characters without modification of gene sequence, is a burgeoning area of study in many disciplines of biology. However, the potential impact of this phenomenon on the physiology of animals is not yet broadly appreciated, in part because the phenomenon of epigenetics is not typically part of the design of physiological investigations. Still enigmatic and somewhat ill defined is the relationship between the overarching concept of epigenetics and interesting transgenerational phenomena (e.g. 'maternal/parental effects') that alter the physiological phenotype of subsequent generations. The lingering effect on subsequent generations of an initial environmental disturbance in parent animals can be profound, with genes continuing to be variously silenced or expressed without an associated change in gene sequence for many generations. Known epigenetic mechanisms involved in this phenomenon include chromatin remodeling (DNA methylation and histone modification), RNA-mediated modifications (non-coding RNA and microRNA), as well as other less well studied mechanisms such as self-sustaining loops and structural inheritance. In this review we: (1) discuss how the concepts of epigenetics and maternal effects both overlap with, and are distinct from, each other; (2) analyze examples of existing animal physiological studies based on these concepts; and (3) offer a construct by which to integrate these concepts into the design of future investigations in animal physiology.", 
"Bisphenol-A (BPA) is a nonsteroidal estrogen that is ubiquitous in the environment. The homeobox gene Hoxa10 controls uterine organogenesis, and its expression is affected by in utero BPA exposure. We hypothesized that an epigenetic mechanism underlies BPA-mediated alterations in Hoxa10 expression. We analyzed the expression pattern and methylation profile of Hoxa10 after in utero BPA exposure. Pregnant CD-1 mice were treated with BPA (5 mg/kg IP) or vehicle control on d 9-16 of pregnancy. Hoxa10 mRNA and protein expression were increased by 25% in the reproductive tract of mice exposed in utero. Bisulfite sequencing revealed that cytosine-guanine dinucleotide methylation was decreased from 67 to 14% in the promoter and from 71 to 3% in the intron of Hoxa10 after in utero BPA exposure. Decreased DNA methylation led to an increase in binding of ER-alpha to the Hoxa10 ERE both in vitro as and in vivo as determined by EMSA and chromatin immunoprecipitation, respectively. Diminished methylation of the ERE-containing promoter sequence resulted in an increase in ERE-driven gene expression in reporter assays. We identify altered methylation as a novel mechanism of BPA-induced altered developmental programming. Permanent epigenetic alteration of ERE sensitivity to estrogen may be a general mechanism through which endocrine disruptors exert their action.-Bromer, J. G., Zhou, Y., Taylor, M. B., Doherty, L., Taylor, H. S.. Bisphenol-A exposure in utero leads to epigenetic alterations in the developmental programming of uterine estrogen response. FASEB J. 24, 2273-2280 (2010). www.fasebj.org", 
"Background: Subtelomeres, regions proximal to telomeres, exhibit characteristics unique to eukaryotic genomes. Genes residing in these loci are subject to epigenetic regulation and elevated rates of both meiotic and mitotic recombination. However, most genome sequences do not contain assembled subtelomeric sequences, and, as a result, subtelomeres are often overlooked in comparative genomics. Results: We studied the evolution and functional divergence of subtelomeric gene families in the yeast lineage. Our computational results show that subtelomeric families are evolving and expanding much faster than families that do not contain subtelomeric genes. Focusing on three related subtelomeric MAL gene families involved in disaccharide metabolism that show typical patterns of rapid expansion and evolution, we show experimentally how frequent duplication events followed by functional divergence yield novel alleles that allow the metabolism of different carbohydrates. Conclusions: Taken together, our computational and experimental analyses show that the extraordinary instability of eukaryotic subtelomeres supports rapid adaptation to novel niches by promoting gene recombination and duplication followed by functional divergence of the alleles.", 
"DNA methylation is dynamically remodelled during the mammalian life cycle through distinct phases of reprogramming and de novo methylation. These events enable the acquisition of cellular potential followed by the maintenance of lineage-restricted cell identity, respectively, a process that defines the life cycle through successive generations. DNA methylation contributes to the epigenetic regulation of many key developmental processes including genomic imprinting, X-inactivation, genome stability and gene regulation. Emerging sequencing technologies have led to recent insights into the dynamic distribution of DNA methylation during development and the role of this epigenetic mark within distinct genomic contexts, such as at promoters, exons or imprinted control regions. Additionally, there is a better understanding of the mechanistic basis of DNA demethylation during epigenetic reprogramming in primordial germ cells and during pre-implantation development. Here, we discuss our current understanding of the developmental roles and dynamics of this key epigenetic system.", 
"MicroRNAs (miRNAs), small noncoding RNAs, can control gene expression by binding to their target genes for degradation and/or translational repression. Epigenetic mechanisms are defined as heritable changes in gene expression that do not involve coding sequence modifications. Both mechanisms play an important role in maintaining physiological functions and are also related to disease development. However, few studies report that miRNA-mediated epigenetic regulations are involved in atherosclerosis. In the present study, oxidized low-density lipoprotein (oxLDL) significantly increased primary human aortic smooth muscle cell (HASMC) migration through MMP-2/MMP-9 up-regulation associated with decreased DNA methylation levels. Either mRNA or protein level of DNA methyltransferase 3b (DNMT3b) showed a dose-dependent down-regulation in oxLDL-mediated HASMCs. Knockdown DNMT3b expression enhanced oxLDL-induced DNA demethylation levels of MMP-2/MMP-9. The expression of miRNA-29b (miR-29b), directly targeting DNMT3b, was up-regulated by oxLDL treatment in a dose-dependent manner. OxLDL-mediated MMP-2/MMP-9 up-regulation, DNMT3b down-regulation, and DNA demethylation were all attenuated after knockdown miR-29b expression by antagomiR-29b. We find that oxLDL can up-regulate miR-29b expression, resulting in DNMT3b down-regulation in HASMCs and epigenetically regulated MMP2/MMP-9 genes involved in cell migration. These results show that miRNA-mediated epigenetic regulation may be a novel mechanism in atherosclerosis.-Chen, K.-C., Wang, Y.-S., Hu, C.-Y., Chang, W.-C., Liao, Y C., Dai, C.-Y., Juo, S.-H. H. OxLDL up-regulates microRNA-29b, leading to epigenetic modifications of MMP-2/MMP-9 genes: a novel mechanism for cardiovascular diseases. FASEB J. 25, 1718-1728 (2011). www.fasebj.org", 
"Somatic cells have been reprogrammed into pluripotent stem cells by introducing a combination of several transcription factors, such as Oct3/4, Sox2, Klf4 and c-Myc. Induced pluripotent stem ( iPS) cells from a patient's somatic cells could be a useful source for drug discovery and cell transplantation therapies. However, most human iPS cells are made by viral vectors, such as retrovirus and lentivirus, which integrate the reprogramming factors into the host genomes and may increase the risk of tumour formation. Several non-integration methods have been reported to overcome the safety concern associated with the generation of iPS cells, such as transient expression of the reprogramming factors using adenovirus vectors or plasmids, and direct delivery of reprogramming proteins. Although these transient expression methods could avoid genomic alteration of iPS cells, they are inefficient. Several studies of gene expression, epigenetic modification and differentiation revealed the insufficient reprogramming of iPS cells, thus suggesting the need for improvement of the reprogramming procedure not only in quantity but also in quality. This report will summarize the current knowledge of iPS generation and discuss future reprogramming methods for medical application.", 
"Polycomb group (PcG) proteins form conserved regulatory complexes that modify chromatin to repress the genes that are not required in a specific differentiation status [1]. In animals, the two best-characterized PcG complexes are PRC2 and PRC1, which respectively possess histone 3 lysine 27 (H3K27) trimethyltransferase [2-4] and histone 2A lysine 119 (H2AK119) E3 ubiquitin ligase activities [5-7]. In Arabidopsis, PRC2 activity is also required for the gene silencing mechanism [8]; however, the existence of PRC1 has been questioned, because plant genomes do not encode clear PRC1 components and H2A monoubiquitination has not been detected [6, 9]. Conversely, recent reports have unveiled the presence of homologs to PRC1 components that together with plant-specific proteins could be part of the long-sought PRC1-like complexes [10, 11]. Here we show that the PRC1 RING-finger homologs AtBMI1A and AtBMI1B are implicated in the repression of embryonic and stem cell regulators. Plants impaired in AtBMI1A and AtBMI1B show derepression of embryonic traits in somatic cells, displaying a phenotype similar to plants mutant in PRC2 components [12-14]. Our data demonstrate that the AtBMI1A/B proteins mediate H2A monoubiquitination in Arabidopsis and that this mark, together with PRC2-mediated H3K27 trimethylation, plays a key role in maintaining cell identity.", 
"Histone modifications are thought to control the regulation of genetic programs in normal physiology and cancer. Methylation (mono-, di-, and tri-methylation) on histone H3 lysine (K) 27 induces transcriptional repression, and thereby participates in controlling gene expression patterns. Enhancer of zeste (EZH) 2, a methyltransferase and component of the polycomb repressive complex 2 (PRC2), plays an essential role in the epigenetic maintenance of the H3K27me3 repressive chromatin mark. Abnormal EZH2 expression has been associated with various cancers including breast cancer. Here, we discuss the contribution of EZH2 and the PRC2 complex in controlling the H3K27 methylation status and subsequent consequences on genomic instability and the cell cycle in breast cancer cells. We also discuss distinct molecular mechanisms used by EZH2 to suppress BRCA1 functions.", 
"Waddington's epigenetic landscape is probably the most famous and most powerful metaphor in developmental biology. Cells, represented by balls, roll downhill through a landscape of bifurcating valleys. Each new valley represents a possible cell fate and the ridges between the valleys maintain the cell fate once it has been chosen. Here I examine models of two important developmental processes - cell-fate induction and lateral inhibition - and ask whether the landscapes for these models at least qualitatively resemble Waddington's picture. For cell-fate induction, the answer is no. The commitment of a cell to a new fate corresponds to the disappearance of a valley from the landscape, not the splitting of one valley into two, and it occurs through a type of bifurcation - a saddle-node bifurcation - that possesses an intrinsic irreversibility that is missing from Waddington's picture. Lateral inhibition, a symmetrical cell-cell competition process, corresponds better to Waddington's picture, with one valley reversibly splitting into two through a pitchfork bifurcation. I propose an alternative epigenetic landscape that has numerous valleys and ridges right from the start, with the process of cell-fate commitment corresponding to the irreversible disappearance of some of these valleys and ridges, via cell-fate induction, complemented by the creation of new valleys and ridges through processes like cell-cell competition.", 
"During gametogenesis and pre-implantation development, the mammalian epigenome is reprogrammed to establish pluripotency in the epiblast. Here we show that the histone 3 lysine 4 (H3K4) methyltransferase, MLL2, controls most of the promoter-specific chromatin modification, H3K4me3, during oogenesis and early development. Using conditional knockout mutagenesis and a hypomorph model, we show that Mll2 deficiency in oocytes results in anovulation and oocyte death, with increased transcription of p53, apoptotic factors, and lap elements. MLL2 is required for (1) bulk H3K4me3 but not H3K4me1, indicating that MLL2 controls most promoters but monomethylation is regulated by a different H3K4 methyltransferase; (2) the global transcriptional silencing that preceeds resumption of meiosis but not for the concomitant nuclear reorganization into the surrounded nucleolus (SN) chromatin configuration; (3) oocyte survival; and (4) normal zygotic genome activation. These results reveal that MLL2 is autonomously required in oocytes for fertility and imply that MLL2 contributes to the epigenetic reprogramming that takes place before fertilization. We propose that once this task has been accomplished, MLL2 is not required until gastrulation and that other methyltransferases are responsible for bulk H3K4me3, thereby revealing an unexpected epigenetic control switch amongst the H3K4 methyltransferases during development.", 
"The consequences of early developmental conditions for performance in later life are now subjected to convergent interest from many different biological sub-disciplines. However, striking data, largely from the biomedical literature, show that environmental effects experienced even before conception can be transmissible to subsequent generations. Here, we review the growing evidence from natural systems for these cross-generational effects of early life conditions, showing that they can be generated by diverse environmental stressors, affect offspring in many ways and can be transmitted directly or indirectly by both parental lines for several generations. In doing so, we emphasize why early life might be so sensitive to the transmission of environmentally induced effects across generations. We also summarize recent theoretical advancements within the field of developmental plasticity, and discuss how parents might assemble different 'internal' and 'external' cues, even from the earliest stages of life, to instruct their investment decisions in offspring. In doing so, we provide a preliminary framework within the context of adaptive plasticity for understanding inter-generational phenomena that arise from early life conditions.", 
"Here, we describe that lysine-specific demethylase 1 (Lsd1/KDM1a), which demethylates histone H3 on Lys4 or Lys9 (H3K4/K9), is an indispensible epigenetic governor of hematopoietic differentiation. Integrative genomic analysis, combining global occupancy of Lsd1, genome-wide analysis of its substrates H3K4 monomethylation and dimethylation, and gene expression profiling, reveals that Lsd1 represses hematopoietic stem and progenitor cell (HSPC) gene expression programs during hematopoietic differentiation. We found that Lsd1 acts at transcription start sites, as well as enhancer regions. Loss of Lsd1 was associated with increased H3K4me1 and H3K4me2 methylation on HSPC genes and gene derepression. Failure to fully silence HSPC genes compromised differentiation of hematopoietic stem cells as well as mature blood cell lineages. Collectively, our data indicate that Lsd1-mediated concurrent repression of enhancer and promoter activity of stem and progenitor cell genes is a pivotal epigenetic mechanism required for proper hematopoietic maturation.", 
"Epigenetics could help to explain individual differences in weight loss after an energy-restriction intervention. Here, we identify novel potential epigenetic biomarkers of weight loss, comparing DNA methylation patterns of high and low responders to a hypocaloric diet. Twenty-five overweight or obese men participated in an 8-wk caloric restriction intervention. DNA was isolated from peripheral blood mononuclear cells and treated with bisulfite. The basal and endpoint epigenetic differences between high and low responders were analyzed by methylation microarray, which was also useful in comparing epigenetic changes due to the nutrition intervention. Subsequently, MALDI-TOF mass spectrometry was used to validate several relevant CpGs and the surrounding regions. DNA methylation levels in several CpGs located in the ATP10A and CD44 genes showed statistical baseline differences depending on the weight-loss outcome. At the treatment endpoint, DNA methylation levels of several CpGs on the WT1 promoter were statistically more methylated in the high than in the low responders. Finally, different CpG sites from WT1 and ATP10A were significantly modified as a result of the intervention. In summary, hypocaloric-diet-induced weight loss in humans could alter DNA methylation status of specific genes. Moreover, baseline DNA methylation patterns may be used as epigenetic markers that could help to predict weight loss.-Milagro, F. I., Campion, J., Cordero, P., Goyenechea, E., Gomez-Uriz, A. M., Abete, I., Zulet, M. A., Martinez, J. A. A dual epigenomic approach for the search of obesity biomarkers: DNA methylation in relation to diet-induced weight loss. FASEB J. 25, 1378-1389 (2011). www.fasebj.org", 
"Common themes are emerging in the molecular mechanisms of long non-coding RNA-mediated gene repression. Long non-coding RNAs (lncRNAs) participate in targeted gene silencing through chromatin remodelling, nuclear reorganisation, formation of a silencing domain and precise control over the entry of genes into silent compartments. The similarities suggest that these are fundamental processes of transcription regulation governed by lncRNAs. These findings have paved the way for analogous investigations on other lncRNAs and chromatin remodelling enzymes. Here we discuss these common mechanisms and provide our view on other molecules that warrant similar investigations. We also present our concepts on the possible mechanisms that may facilitate the exit of genes from the silencing domains and their potential therapeutic applications. Finally, we point to future areas of research and put forward our recommendations for improvements in resources and applications of existing technologies towards targeted outcomes in this active area of research.", 
"The epigenetic modification of chromatin structure and its effect on complex neuronal processes like learning and memory is an emerging field in neuroscience. However, little is known about the writers'' of the neuronal epigenome and how they lay down the basis for proper cognition. Here, we have dissected the neuronal function of the Drosophila euchromatin histone methyltransferase (EHMT), a member of a conserved protein family that methylates histone 3 at lysine 9 (H3K9). EHMT is widely expressed in the nervous system and other tissues, yet EHMT mutant flies are viable. Neurodevelopmental and behavioral analyses identified EHMT as a regulator of peripheral dendrite development, larval locomotor behavior, non-associative learning, and courtship memory. The requirement for EHMT in memory was mapped to 7B-Gal4 positive cells, which are, in adult brains, predominantly mushroom body neurons. Moreover, memory was restored by EHMT re-expression during adulthood, indicating that cognitive defects are reversible in EHMT mutants. To uncover the underlying molecular mechanisms, we generated genome-wide H3K9 dimethylation profiles by ChIP-seq. Loss of H3K9 dimethylation in EHMT mutants occurs at 5% of the euchromatic genome and is enriched at the 5' and 3' ends of distinct classes of genes that control neuronal and behavioral processes that are corrupted in EHMT mutants. Our study identifies Drosophila EHMT as a key regulator of cognition that orchestrates an epigenetic program featuring classic learning and memory genes. Our findings are relevant to the pathophysiological mechanisms underlying Kleefstra Syndrome, a severe form of intellectual disability caused by mutations in human EHMT1, and have potential therapeutic implications. Our work thus provides novel insights into the epigenetic control of cognition in health and disease.", 
"Acquired drug resistance is a major limitation for the successful treatment of cancer. Resistance can emerge due to a variety of reasons including host environmental factors as well as genetic or epigenetic alterations in the cancer cells. Evolutionary theory has contributed to the understanding of the dynamics of resistance mutations in a cancer cell population, the risk of resistance pre-existing before the initiation of therapy, the composition of drug cocktails necessary to prevent the emergence of resistance, and optimum drug administration schedules for patient populations at risk of evolving acquired resistance. Here we review recent advances towards elucidating the evolutionary dynamics of acquired drug resistance and outline how evolutionary thinking can contribute to outstanding questions in the field. Published by Elsevier Ltd.", 
"DNA methylation is an epigenetic modification associated with gene silencing. In Arabidopsis, DNA methylation is established by DOMAINS REARRANGED METHYLTRANSFERASE 2 (DRM2), which is targeted by small interfering RNAs through a pathway termed RNA-directed DNA methylation (RdDM) [1, 2]. Recently, RdDM was shown to require intergenic noncoding (IGN) transcripts that are dependent on the Pol V polymerase. These transcripts are proposed to function as scaffolds for the recruitment of downstream RdDM proteins, including DRM2, to loci that produce both siRNAs and IGN transcripts [3]. However, the mechanism(s) through which Pol V is targeted to specific genomic loci remains largely unknown. Through affinity purification of two known RdDM components, DEFECTIVE IN RNA-DIRECTED DNA METHYLATION 1 (DRD1) [4] and DEFECTIVE IN MERISTEM SILENCING 3 (DMS3) [5, 6], we found that they copurify with each other and with a novel protein, RNA-DIRECTED DNA METHYLATION 1 (RDM1), forming a complex we term DDR. We also found that DRD1 copurified with Pol V subunits and that RDM1, like DRD1 [3] and DMS3 [7], is required for the production of Pol V-dependent transcripts. These results suggest that the DDR complex acts in RdDM at a step upstream of the recruitment or activation of Pol V.", 
"The Neo-Darwinian concept of natural selection is plausible when one assumes a straightforward causation of phenotype by genotype. However, such simple 1:1 mapping must now give place to the modern concepts of gene regulatory networks and gene expression noise. Both can, in the absence of genetic mutations, jointly generate a diversity of inheritable randomly occupied phenotypic states that could also serve as a substrate for natural selection. This form of epigenetic dynamics challenges Neo-Darwinism. It needs to incorporate the non-linear, stochastic dynamics of gene networks. A first step is to consider the mathematical correspondence between gene regulatory networks and Waddington's metaphoric epigenetic landscape, which actually represents the quasi-potential function of global network dynamics. It explains the coexistence of multiple stable phenotypes within one genotype. The landscape's topography with its attractors is shaped by evolution through mutational re-wiring of regulatory interactions offering a link between genetic mutation and sudden, broad evolutionary changes.", 
"Over two decades of research have demonstrated that the peptide hormone endothelin-1 (ET-1) plays multiple, complex roles in cardiovascular, neural, pulmonary, reproductive, and renal physiology. Differential and tissue-specific production of ET-1 must be tightly regulated in order to preserve these biologically diverse actions. The primary mechanism thought to control ET-1 bioavailability is the rate of transcription from the ET-1 gene (edn1). Studies conducted on a variety of cell types have identified key transcription factors that govern edn1 expression. With few exceptions, the cis-acting elements bound by these factors have been mapped in the edn1 regulatory region. Recent evidence has revealed new roles for some factors originally believed to regulate edn1 in a tissue or hormone-specific manner. In addition, other mechanisms involved in epigenetic regulation and mRNA stability have emerged as important processes for regulated edn1 expression. The goal of this review is to provide a comprehensive overview of the specific factors and signaling systems that govern edn1 activity at the molecular level.-Stow, L. R., Jacobs, M. E., Wingo, C. S., Cain, B. D. Endothelin-1 gene regulation. FASEB J. 25, 16-28 (2011). www.fasebj.org", 
"In most eukaryotes, the HISTONE 3 family comprises several variants distinguished by their amino acid sequence, localization, and correlation with transcriptional activity [1-3]. Transgenerational inheritance of epigenetic information carried by histones is still unclear [4, 5]. In addition to covalent histone modifications, the mosaic distribution of H3 variants onto chromatin has been proposed to provide a new level of epigenetic information [6]. To study the transmission of patterns of H3 variants through generations, we combined transcriptional profiling and live imaging of the 13 H3 variants encoded by the Arabidopsis plant genome [7]. In comparison with somatic cells, only a restricted number of H3 variants are present in male and female gametes. Upon fertilization, H3 variants contributed by both gametes are actively removed from the zygote chromatin. The somatic H3 composition is restored in the embryo by de novo synthesis of H3 variants. A survey of Arabidopsis homologs of animal H3 chaperones suggests that removal of parental H3 from the zygote nucleus relies on a new mechanism. Our results suggest that reprogramming of parental genomes in the zygote limits the inheritance of epigenetic information carried by H3 variants across generations.", 
"Fundamental to genomic imprinting in mammals is the acquisition of epigenetic marks that differ in male and female gametes at 'imprinting control regions' (ICRs). These marks mediate the allelic expression of imprinted genes in the offspring. Much has been learnt about the nature of imprint marks, the times during gametogenesis at which they are laid down and some of the factors responsible especially for DNA methylation. Recent work has revealed that transcription and histone modifications are critically involved in DNA methylation acquisition, and these findings allow us to propose rational models for methylation establishment. A completely novel perspective on gametic DNA methylation has emerged from epigenomic profiling. Far more differentially methylated loci have been identified in gametes than known imprinted genes, which leads us to revise the notion that methylation of ICRs is a specifically targeted process. Instead, it seems to obey default processes in germ cells, giving rise to distinct patterns of DNA methylation in sperm and oocytes. This new insight, together with the identification of proteins that preserve DNA methylation after fertilization, emphasizes the key role played by mechanisms that selectively retain differential methylation at imprinted loci during early development. Addressing these mechanisms will be essential to understanding the specificity and evolution of genomic imprinting.", 
"DNA methylation maintains genome stability and regulates gene expression [1]. In mammals, DNA methylation is reprogrammed in the germline from one generation to the next [2]. In plants, it was considered that patterns of DNA methylation are stably maintained through sexual reproduction [3-6]. However, a recent report showed discrete variations of DNA methylation profiles from mother to daughter plants [7]. The mechanisms that explain these variations have remained unknown. Here, we report that maintenance DNA methyltransferases are barely expressed during Arabidopsis female gametogenesis. In contrast, after fertilization both maintenance and de novo DNA methyltransferases are expressed strongly in the embryo. Embryogenesis is marked by increased de novo DNA methylation, reaching levels that are further maintained in the adult plant. The accumulation of these epigenetic marks after fertilization silences a methylation-sensitive fluorescent reporter. De novo DNA methylation in the embryo provides a mechanism that could account for the gradual remethylation of experimentally demethylated genomes [8, 9]. In conclusion, we uncover that DNA methylation activity fluctuates during sexual reproduction. This cycle likely explains variations of genome-wide patterns of DNA methylation across generations in Arabidopsis [7, 10] and enables a limited degree of reprogramming of the epigenome.", 
"We hypothesize that heritable epigenetic changes can affect rates of fitness increase as well as patterns of genotypic and phenotypic change during adaptation. In particular, we suggest that when natural selection acts on pure epigenetic variation in addition to genetic variation, populations adapt faster, and adaptive phenotypes can arise before any genetic changes. This may make it difficult to reconcile the timing of adaptive events detected using conventional population genetics tools based on DNA sequence data with environmental drivers of adaptation, such as changes in climate. Epigenetic modifications are frequently associated with somatic cell differentiation, but recently epigenetic changes have been found that can be transmitted over many generations. Here, we show how the interplay of these heritable epigenetic changes with genetic changes can affect adaptive evolution, and how epigenetic changes affect the signature of selection in the genetic record.", 
"The in utero availability of methyl donors, such as choline, may modify fetal epigenetic marks and lead to sustainable functional alterations throughout the life course. The hypothalamic-pituitary-adrenal (HPA) axis regulates cortisol production and is sensitive to perinatal epigenetic programming. As an extension of a 12-wk dose-response choline feeding study conducted in third-trimester pregnant women, we investigated the effect of maternal choline intake (930 vs. 480 mg/d) on the epigenetic state of cortisol-regulating genes, and their expression, in placenta and cord venous blood. The higher maternal choline intake yielded higher placental promoter methylation of the cortisol-regulating genes, corticotropin releasing hormone (CRH; P = 0.05) and glucocorticoid receptor (NR3C1; P = 0.002); lower placental CRH transcript abundance (P = 0.04); lower cord blood leukocyte promoter methylation of CRH (P = 0.05) and NR3C1 (P = 0.04); and 33% lower (P = 0.07) cord plasma cortisol. In addition, placental global DNA methylation and dimethylated histone H3 at lysine 9 (H3K9me2) were higher (P = 0.02) in the 930 mg choline/d group, as was the expression of select placental methyltransferases. These data collectively suggest that maternal choline intake in humans modulates the epigenetic state of genes that regulate fetal HPA axis reactivity as well as the epigenomic status of fetal derived tissues.-Jiang, X., Yan, J., West, A. A., Perry, C. A., Malysheva, O. V., Devapatla, S., Pressman, E., Vermeylen, F., Caudill, M. A. Maternal choline intake alters the epigenetic state of fetal cortisol-regulating genes in humans. FASEB J. 26, 3563-3574 (2012). www.fasebj.org", 
"Even though schizophrenia has a strong hereditary component, departures from simple genetic transmission are prominent. DNA methylation has emerged as an epigenetic explanatory candidate of schizophrenia's nonmendelian characteristics. To investigate this assumption, we examined genome-wide (global) and gene-specific DNA methylation levels, which are associated with genomic stability and gene expression activity, respectively. Analyses were conducted using DNA from leukocytes of patients with schizophrenia and controls. Global methylation results revealed a highly significant hypomethylation in patients with schizophrenia (P<2.0 x 10(-6)) and linear regression among patients generated a model in which antipsychotic treatment and disease onset explained 11% of the global methylation variance (adjusted R-2=0.11, ANOVA P<0.001). Specifically, haloperidol was associated with higher (control-like) methylation (P=0.001), and early onset (a putative marker of schizophrenia severity) was associated with lower methylation (P=0.002). With regard to the gene-specific methylation analyses, and in accordance with the dopamine hypothesis of psychosis, we found that the analyzed region of S-COMT was hypermethylated in patients with schizophrenia (P=0.004). In summary, these data support the notion of a dysregulated epigenome in schizophrenia, which, at least globally, is more pronounced in early-onset patients and can be partly rescued by antipsychotic medication. In addition, blood DNA-methylation signatures show promise of serving as a schizophrenia biomarker in the future.-Melas, P. A., Rogdaki, M., sby, U., Schalling, M., Lavebratt, C., Ekstrom, T. J. Epigenetic aberrations in leukocytes of patients with schizophrenia: association of global DNA methylation with antipsychotic drug treatment and disease onset. FASEB J. 26, 2712-2718 (2012). www.fasebj.org", 
"The centromere, responsible for chromosome segregation during mitosis, is epigenetically defined by CENP-A containing chromatin. The amount of centromeric CENP-A has direct implications for both the architecture and epigenetic inheritance of centromeres. Using complementary strategies, we determined that typical human centromeres contain similar to 400 molecules of CENP-A, which is controlled by a mass-action mechanism. This number, despite representing only similar to 4% of all centromeric nucleosomes, forms a similar to 50-fold enrichment to the overall genome. In addition, although pre-assembled CENP-A is randomly segregated during cell division, this amount of CENP-A is sufficient to prevent stochastic loss of centromere function and identity. Finally, we produced a statistical map of CENP-A occupancy at a human neocentromere and identified nucleosome positions that feature CENP-A in a majority of cells. In summary, we present a quantitative view of the centromere that provides a mechanistic framework for both robust epigenetic inheritance of centromeres and the paucity of neocentromere formation.", 
"Long non-coding RNAs (lncRNAs), which represent a new frontier in molecular biology, play important roles in regulating gene expression at epigenetic, transcriptional and post-transcriptional levels. More and more lncRNAs have been found to play important roles in normal cell physiological activities, and participate in the development of varieties of tumors and other diseases. Previously, we have only been able to determine the function of lncRNAs through multiple mechanisms, including genetic imprinting, chromatin remodeling, splicing regulation, mRNA decay, and translational regulation. Application of technological advances to research into the function of lncRNAs is extremely important. The major tools for exploring lncRNAs include microarrays, RNA sequencing (RNA-seq), Northern blotting, real-time quantitative reverse transcription-polymerase chain reaction (qRT-PCR), fluorescence in situ hybridization (FISH), RNA interference (RNAi), RNA-binding protein immunoprecipitation (RIP), chromatin isolation by RNA purification (ChIRP), crosslinking-immunopurification (CLIP), and bioinformatic prediction. In this review, we highlight the functions of lncRNAs, and advanced methods to research lncRNA-protein interactions.", 
"To explore the potential evolutionary relevance of heritable epigenetic variation, the National Evolutionary Synthesis Center recently hosted a catalysis meeting that brought together molecular epigeneticists, experimental evolutionary ecologists, and theoretical population and quantitative geneticists working across a wide variety of systems. The group discussed the methods available to investigate epigenetic variation and epigenetic inheritance, and how to evaluate their importance for phenotypic evolution. We found that understanding tire relevance of epigenetic effects in phenotypic evolution will require clearly delineating epigenetics within existing terminology and expanding research efforts into ecologically relevant circumstances across model and nonmodel organisms. In addition, a critical component of understanding epigenetics will be the development of new and current statistical approaches and expansion of quantitative and population genetic theory Although the importance of heritable epigenetic effects on evolution is still under discussion, investigating them in the context of a multidisciplinary approach could transform the field.", 
"Two-thirds of gene promoters in mammals are associated with regions of non-methylated DNA, called CpG islands (CGIs), which counteract the repressive effects of DNA methylation on chromatin. In cold-blooded vertebrates, computational CGI predictions often reside away from gene promoters, suggesting a major divergence in gene promoter architecture across vertebrates. By experimentally identifying non-methylated DNA in the genomes of seven diverse vertebrates, we instead reveal that non-methylated islands (NMIs) of DNA are a central feature of vertebrate gene promoters. Furthermore, NMIs are present at orthologous genes across vast evolutionary distances, revealing a surprising level of conservation in this epigenetic feature. By profiling NMIs in different tissues and developmental stages we uncover a unifying set of features that are central to the function of NMIs in vertebrates. Together these findings demonstrate an ancient logic for NMI usage at gene promoters and reveal an unprecedented level of epigenetic conservation across vertebrate evolution.", 
"The PRDM family has recently spawned considerable interest as it has been implicated in fundamental aspects of cellular differentiation and exhibits expanding ties to human diseases. The PRDMs belong to the SET domain family of histone methyltransferases, however, enzymatic activity has been determined for only few PRDMs suggesting that they act by recruiting co-factors or, more speculatively, confer methylation of non-histone targets. Several PRDM family members are deregulated in human diseases, most prominently in hematological malignancies and solid cancers, where they can act as both tumor suppressors or drivers of oncogenic processes. The molecular mechanisms have been delineated for only few PRDMs and little is known about functional redundancy within the family. Future studies should identify target genes of PRDM proteins and the protein complexes in which PRDM proteins reside to provide a more comprehensive understanding of the biological and biochemical functions of this important protein family.", 
"Interactions of cell-autonomous circadian oscillators with diurnal cycles govern the temporal compartmentalization of cell physiology in mammals. To understand the transcriptional and epigenetic basis of diurnal rhythms in mouse liver genome-wide, we generated temporal DNA occupancy profiles by RNA polymerase II (Pol II) as well as profiles of the histone modifications H3K4me3 and H3K36me3. We used these data to quantify the relationships of phases and amplitudes between different marks. We found that rhythmic Pol II recruitment at promoters rather than rhythmic transition from paused to productive elongation underlies diurnal gene transcription, a conclusion further supported by modeling. Moreover, Pol II occupancy preceded mRNA accumulation by 3 hours, consistent with mRNA half-lives. Both methylation marks showed that the epigenetic landscape is highly dynamic and globally remodeled during the 24-hour cycle. While promoters of transcribed genes had tri-methylated H3K4 even at their trough activity times, tri-methylation levels reached their peak, on average, 1 hour after Pol II. Meanwhile, rhythms in tri-methylation of H3K36 lagged transcription by 3 hours. Finally, modeling profiles of Pol II occupancy and mRNA accumulation identified three classes of genes: one showing rhythmicity both in transcriptional and mRNA accumulation, a second class with rhythmic transcription but flat mRNA levels, and a third with constant transcription but rhythmic mRNAs. The latter class emphasizes widespread temporally gated posttranscriptional regulation in the mouse liver.", 
"Telomeres protect the chromosome ends from unscheduled DNA repair and degradation. Telomeres are heterochromatic domains composed of repetitive DNA (TTAGGG repeats) bound to an array of specialized proteins. The length of telomere repeats and the integrity of telomere-binding proteins are both important for telomere protection. Furthermore, telomere length and integrity are regulated by a number of epigenetic modifications, thus pointing to higher order control of telomere function. In this regard, we have recently discovered that telomeres are transcribed generating long, non-coding RNAs, which remain associated with the telomeric chromatin and are likely to have important roles in telomere regulation. In the past, we showed that telomere length and the catalytic component of telomerase, Tert, are critical determinants for the mobilization of stem cells. These effects of telomerase and telomere length on stem cell behaviour anticipate the premature ageing and cancer phenotypes of telomerase mutant mice. Recently, we have demonstrated the anti-ageing activity of telomerase by forcing telomerase expression in mice with augmented cancer resistance. Shelterin is the major protein complex bound to mammalian telomeres; however, its potential relevance for cancer and ageing remained unaddressed to date. To this end, we have generated mice conditionally deleted for the shelterin proteins TRF1, TPP1 and Rap1. The study of these mice demonstrates that telomere dysfunction, even if telomeres are of a normal length, is sufficient to produce premature tissue degeneration, acquisition of chromosomal aberrations and initiation of neoplastic lesions. These new mouse models, together with the telomerase-deficient mouse model, are valuable tools for understanding human pathologies produced by telomere dysfunction.", 
"The basic unit of genome packaging is the nucleosome, and nucleosomes have long been proposed to restrict DNA accessibility both to damage and to transcription. Nucleosome number in cells was considered fixed, but recently aging yeast and mammalian cells were shown to contain fewer nucleosomes. We show here that mammalian cells lacking High Mobility Group Box 1 protein (HMGB1) contain a reduced amount of core, linker, and variant histones, and a correspondingly reduced number of nucleosomes, possibly because HMGB1 facilitates nucleosome assembly. Yeast nhp6 mutants lacking Nhp6a and -b proteins, which are related to HMGB1, also have a reduced amount of histones and fewer nucleosomes. Nucleosome limitation in both mammalian and yeast cells increases the sensitivity of DNA to damage, increases transcription globally, and affects the relative expression of about 10% of genes. In yeast nhp6 cells the loss of more than one nucleosome in four does not affect the location of nucleosomes and their spacing, but nucleosomal occupancy. The decrease in nucleosomal occupancy is non-uniform and can be modelled assuming that different nucleosomal sites compete for available histones. Sites with a high propensity to occupation are almost always packaged into nucleosomes both in wild type and nucleosome-depleted cells; nucleosomes on sites with low propensity to occupation are disproportionately lost in nucleosome-depleted cells. We suggest that variation in nucleosome number, by affecting nucleosomal occupancy both genomewide and gene-specifically, constitutes a novel layer of epigenetic regulation.", 
"The multifunctional zinc-finger protein CCCTC-binding factor (CTCF) is a very strong candidate for the role of coordinating the expression level of coding sequences with their three-dimensional position in the nucleus, apparently responding to a code in the DNA itself. Dynamic interactions between chromatin fibers in the context of nuclear architecture have been implicated in various aspects of genome functions. However, the molecular basis of these interactions still remains elusive and is a subject of intense debate. Here we discuss the nature of CTCF-DNA interactions, the CTCF-binding specificity to its binding sites and the relationship between CTCF and chromatin, and we examine data linking CTCF with gene regulation in the three-dimensional nuclear space. We discuss why these features render CTCF a very strong candidate for the role and propose a unifying model, the CTCF code, explaining the mechanistic basis of how the information encrypted in DNA may be interpreted by CTCF into diverse nuclear functions.", 
"Somatic cell nuclear transfer (SCNT) cloning is the sole reproductive engineering technology that endows the somatic cell genome with totipotency. Since the first report on the birth of a cloned sheep from adult somatic cells in 1997, many technical improvements in SCNT have been made by using different epigenetic approaches, including enhancement of the levels of histone acetylation in the chromatin of the reconstructed embryos. Although it will take a considerable time before we fully understand the nature of genomic programming and totipotency, we may expect that somatic cell cloning technology will soon become broadly applicable to practical purposes, including medicine, pharmaceutical manufacturing and agriculture. Here we review recent progress in somatic cell cloning, with a special emphasis on epigenetic studies using the laboratory mouse as a model.", 
"Long non-coding RNAs (lncRNAs) are transcripts with low protein-coding potential that represent a large proportion of the transcriptional output of the cell. Many lncRNAs exhibit features indicative of functionality including tissue-restricted expression, localization to distinct subcellular structures, regulated expression and evolutionary conservation. Some lncRNAs have been shown to associate with chromatin-modifying activities and transcription factors, suggesting that a common mode of action may be to guide protein complexes to target genomic loci. However, the functions (if any) of the vast majority of lncRNA transcripts are currently unknown, and the subject of investigation. Here, we consider the putative role(s) of lncRNAs in neurodevelopment and brain function with an emphasis on the epigenetic regulation of gene expression. Associations of lncRNAs with neurodevelopmental/neuropsychiatric disorders, neurodegeneration and brain cancers are also discussed.", 
"Compelling evidence suggests that the transcription factor Foxp3 acts as a master switch governing the development and function of CD4(+) regulatory T cells (Tregs). However, whether transcriptional control of Foxp3 expression itself contributes to the development of a stable Treg lineage has thus far not been investigated. We here identified an evolutionarily conserved region within the foxp3 locus upstream of exon-1 possessing transcriptional activity. Bisulphite sequencing and chromatin immunoprecipitation revealed complete demethylation of CpG motifs as well as histone modifications within the conserved region in ex vivo isolated Foxp3(+)CD25(+)CD4(+) Tregs, but not in naive CD25(-) CD4(+) T cells. Partial DNA demethylation is already found within developing Foxp3(+) thymocytes; however, Tregs induced by TGF-beta in vitro display only incomplete demethylation despite high Foxp3 expression. In contrast to natural Tregs, these TGF-beta-induced Foxp3(+) Tregs lose both Foxp3 expression and suppressive activity upon restimulation in the absence of TGF-beta. Our data suggest that expression of Foxp3 must be stabilized by epigenetic modification to allow the development of a permanent suppressor cell lineage, a finding of significant importance for therapeutic applications involving induction or transfer of Tregs and for the understanding of long-term cell lineage decisions.", 
"This review describes new developments in the study of transgenerational epigenetic inheritance, a component of epigenetics. We start by examining the basic concepts of the field and the mechanisms that underlie epigenetic inheritance. We present a comprehensive review of transgenerational cellular epigenetic inheritance among different taxa in the form of a table, and discuss the data contained therein. The analysis of these data shows that epigenetic inheritance is ubiquitous and suggests lines of research that go beyond present approaches to the subject. We conclude by exploring some of the consequences of epigenetic inheritance for the study of evolution, while also pointing to the importance of recognizing and understanding epigenetic inheritance for practical and theoretical issues in biology.", 
"Induced pluripotent stem (iPS) cells are generated from somatic cells by genetic manipulation. Reprogramming entails multiple transgene integrations and occurs apparently stochastically in rare cells over many days. Tissue stem cells may be subject to less-stringent epigenetic restrictions than other cells and might therefore be more amenable to deprogramming. We report that brain-derived neural stem (NS) cells acquire undifferentiated morphology rapidly and at high frequency after a single round of transduction with reprogramming factors. However, critical attributes of true pluripotency-including stable expression of endogenous Oct4 and Nanog, epigenetic erasure of X chromosome silencing in female cells, and ability to colonise chimaeras-were not attained. We therefore applied molecularly defined conditions for the derivation and propagation of authentic pluripotent stem cells from embryos. We combined dual inhibition (2i) of mitogen-activated protein kinase signalling and glycogen synthase kinase-3 (GSK3) with the self-renewal cytokine leukaemia inhibitory factor (LIF). The 2i/LIF condition induced stable up-regulation of Oct4 and Nanog, reactivation of the X chromosome, transgene silencing, and competence for somatic and germline chimaerism. Using 2i/LIF, NS cell reprogramming required only 1-2 integrations of each transgene. Furthermore, transduction with Sox2 and c-Myc is dispensable, and Oct4 and Klf4 are sufficient to convert NS cells into chimaera-forming iPS cells. These findings demonstrate that somatic cell state influences requirements for reprogramming and delineate two phases in the process. The ability to capture pre-pluripotent cells that can advance to ground state pluripotency simply and with high efficiency opens a door to molecular dissection of this remarkable phenomenon.", 
"Biomedical science has little considered the relevance of life history theory and evolutionary and ecological developmental biology to clinical medicine. However, the observations that early life influences can alter later disease risk-the developmental origins of health and disease (DOHaD) paradigm-have led to a recognition that these perspectives can inform our understanding of human biology. We propose that the DOHaD phenomenon can be considered as a subset of the broader processes of developmental plasticity by which organisms adapt to their environment during their life course. Such adaptive processes allow genotypic variation to be preserved through transient environmental changes. Cues for plasticity operate particularly during early development; they may affect a single organ or system, but generally they induce integrated adjustments in the mature phenotype, a process underpinned by epigenetic mechanisms and influenced by prediction of the mature environment. In mammals, an adverse intrauterine environment results in an integrated suite of responses, suggesting the involvement of a few key regulatory genes, that resets the developmental trajectory in expectation of poor postnatal conditions. Mismatch between the anticipated and the actual mature environment exposes the organism to risk of adverse consequences-the greater the mismatch, the greater the risk. For humans, prediction is inaccurate for many individuals because of changes in the postnatal environment toward energy-dense nutrition and low energy expenditure, contributing to the epidemic of chronic noncommunicable disease. This view of human disease from the perspectives of life history biology and evolutionary theory offers new approaches to prevention, diagnosis and intervention.", 
"Age-related defects in stem cells can limit proper tissue maintenance and hence contribute to a shortened lifespan. Using highly purified hematopoietic stem cells from mice aged 2 to 21 mo, we demonstrate a deficit in function yet an increase in stem cell number with advancing age. Expression analysis of more than 14,000 genes identified 1,500 that were age-induced and 1,600 that were age-repressed. Genes associated with the stress response, inflammation, and protein aggregation dominated the up-regulated expression profile, while the down-regulated profile was marked by genes involved in the preservation of genomic integrity and chromatin remodeling. Many chromosomal regions showed coordinate loss of transcriptional regulation; an overall increase in transcriptional activity with age and inappropriate expression of genes normally regulated by epigenetic mechanisms was also observed. Hematopoietic stem cells from early-aging mice expressing a mutant p53 allele reveal that aging of stem cells can be uncoupled from aging at an organismal level. These studies show that hematopoietic stem cells are not protected from aging. Instead, loss of epigenetic regulation at the chromatin level may drive both functional attenuation of cells, as well as other manifestations of aging, including the increased propensity for neoplastic transformation.", 
"Trimethylation of histone H3 lysine 27 (H3K27me3) plays critical roles in regulating animal development, and in several cases, H3K27me3 is also required for the proper expression of developmentally important genes in plants. However, the extent to which H3K27me3 regulates plant genes on a genome-wide scale remains unknown. In addition, it is not clear whether the establishment and spreading of H3K27me3 occur through the same mechanisms in plants and animals. We identified regions containing H3K27me3 in the genome of the flowering plant Arabidopsis thaliana using a high-density whole-genome tiling microarray. The results suggest that H3K27me3 is a major silencing mechanism in plants that regulates an unexpectedly large number of genes in Arabidopsis (similar to 4,400), and that the maintenance of H3K27me3 is largely independent of other epigenetic pathways, such as DNA methylation or RNA interference. Unlike in animals, where H3K27m3 occupies large genomic regions, in Arabidopsis, we found that H3K27m3 domains were largely restricted to the transcribed regions of single genes. Furthermore, unlike in animals systems, H3K27m3 domains were not preferentially associated with low-nucleosome density regions. The results suggest that different mechanisms may underlie the establishment and spreading of H3K27me3 in plants and animals.", 
"The relative contribution of genetic and environmental influences to the US black-white disparity in cardiovascular disease (CVD) is hotly debated within the public health, anthropology, and medical communities. In this article, we review evidence for developmental and epigenetic pathways linking early life environments with CVD, and critically evaluate their possible role in the origins of these racial health disparities. African Americans not only suffer from a disproportionate burden of CVD relative to whites, but also have higher rates of the perinatal health disparities now known to be the antecedents of these conditions. There is extensive evidence for a social origin to prematurity and low birth weight in African Americans, reflecting pathways such as the effects of discrimination on maternal stress physiology. In light of the inverse relationship between birth weight and adult CVD, there is now a strong rationale to consider developmental and epigenetic mechanisms as links between early life environmental factors like maternal stress during pregnancy and adult race-based health disparities in diseases like hypertension, diabetes, stroke, and coronary heart disease. The model outlined here builds upon social constructivist perspectives to highlight an important set of mechanisms by which social influences can become embodied, having durable and even transgenerational influences on the most pressing US health disparities. We conclude that environmentally responsive phenotypic plasticity, in combination with the better-studied acute and chronic effects of social-environmental exposures, provides a more parsimonious explanation than genetics for the persistence of CVD disparities between members of socially imposed racial categories. Am. J. Hum. Biol. 21:2-15, 2009. (C) 2008 Wiley-Liss, Inc.", 
"Cytosine methylation of repetitive sequences is widespread in plant genomes, occurring in both symmetric (CpG and CpNpG) as well as asymmetric sequence contexts. We used the methylation- dependent restriction enzyme McrBC to profile methylated DNA using tiling microarrays of Arabidopsis Chromosome 4 in two distinct ecotypes, Columbia and Landsberg erecta. We also used comparative genome hybridization to profile copy number polymorphisms. Repeated sequences and transposable elements (TEs), especially long terminal repeat retrotransposons, are densely methylated, but one third of genes also have low but detectable methylation in their transcribed regions. While TEs are almost always methylated, genic methylation is highly polymorphic, with half of all methylated genes being methylated in only one of the two ecotypes. A survey of loci in 96 Arabidopsis accessions revealed a similar degree of methylation polymorphism. Within-gene methylation is heritable, but is lost at a high frequency in segregating F-2 families. Promoter methylation is rare, and gene expression is not generally affected by differences in DNA methylation. Small interfering RNA are preferentially associated with methylated TEs, but not with methylated genes, indicating that most genic methylation is not guided by small interfering RNA. This may account for the instability of gene methylation, if occasional failure of maintenance methylation cannot be restored by other means.", 
"Polyploidy is produced by multiplication of a single genome (autopolyploid) or combination of two or more divergent genomes (allopolyploid). The available data obtained from the study of synthetic (newly created or human-made) plant allopolyploids have documented dynamic and stochastic changes in genomic organization and gene expression, including sequence elimination, inter-chromosomal exchanges, cytosine methylation, gene repression, novel activation, genetic dominance, subfunctionalization and transposon activation. The underlying mechanisms for these alterations are poorly understood. To promote a better understanding of genomic and gene expression changes in polyploidy, we briefly review origins and forms of polyploidy and summarize what has been learned from genome-wide gene expression analyses in newly synthesized auto- and allopolyploids. We show transcriptome divergence between the progenitors and in the newly formed allopolyploids. We propose models for transcriptional regulation, chromatin modification and RNA-mediated pathways in establishing locus-specific expression of orthologous and homoeologous genes during allopolyploid formation and evolution.", 
"In mammals, the X and Y chromosomes are subject to meiotic sex chromosome inactivation (MSCI) during prophase I in the male germline, but their status thereafter is currently unclear. An abundance of X-linked spermatogenesis genes has spawned the view that the X must be active [1-8]. On the other hand, the idea that the imprinted paternal X of the early embryo may be preinactivated by MSCI suggests that silencing may persist longer [9-12]. To clarify this issue, we establish a comprehensive X-expression profile during mouse spermatogenesis. Here, we discover that the X and Y occupy a novel compartment in the postmeiotic spermatid and adopt a non-Rabl configuration. We demonstrate that this postmeiotic sex chromatin (PMSC) persists throughout spermiogenesis into mature sperm and exhibits epigenetic similarity to the XY body. In the spermatid, 87% of X-linked genes remain suppressed postmeiotically, while autosomes are largely active. We conclude that chromosome-wide X silencing continues from meiosis to the end of spermiogenesis, and we discuss implications for proposed mechanisms of imprinted X-inactivation.", 
"Many genes are recruited to the nuclear periphery upon transcriptional activation. The mechanism and functional significance of this recruitment is unclear. We find that recruitment of the yeast INO1 and GAL1 genes to the nuclear periphery is rapid and independent of transcription. Surprisingly, these genes remain at the periphery for generations after they are repressed. Localization at the nuclear periphery serves as a form of memory of recent transcriptional activation, promoting reactivation. Previously expressed GAL1 at the nuclear periphery is activated much more rapidly than long- term repressed GAL1 in the nucleoplasm, even after six generations of repression. Localization of INO1 at the nuclear periphery is necessary and sufficient to promote more rapid activation. This form of transcriptional memory is chromatin based; the histone variant H2A. Z is incorporated into nucleosomes within the recently repressed INO1 promoter and is specifically required for rapid reactivation of both INO1 and GAL1. Furthermore, H2A. Z is required to retain INO1 at the nuclear periphery after repression. Therefore, H2A. Z-mediated localization of recently repressed genes at the nuclear periphery represents an epigenetic state that confers memory of transcriptional activation and promotes reactivation.", 
"There is increasing evidence that dynamic changes to chromatin, chromosomes and nuclear architecture are regulated by RNA signalling. Although the precise molecular mechanisms are not well understood, they appear to involve the differential recruitment of a hierarchy of generic chromatin modifying complexes and DNA methyltransferases to specific loci by RNAs during differentiation and development. A significant fraction of the genome-wide transcription of non-protein coding RNAs may be involved in this process, comprising a previously hidden layer of intermediary genetic information that underpins developmental ontogeny and the differences between species, ecotypes and individuals. It is also evident that RNA editing is a primary means by which hardwired genetic information in animals can be altered by environmental signals, especially in the brain, indicating a dynamic RNA-mediated interplay between the transcriptome, the environment and the epigenome. Moreover, RNA-directed regulatory processes may also transfer epigenetic information not only within cells but also between cells and organ systems, as well as across generations.", 
"Epigenetic gene silencing suppresses transposon activity and is critical for normal development [1, 2]. Two common epigenetic gene-silencing marks are DNA methylation and histone H3 lysine 9 dimethylation (H3K9me2) [1]. In Arabidopsis thaliana, H3K9me2, catalyzed by the methyltransferase KRYPTONITE (KYP/ SUVH4), is required for maintenance of DNA methylation outside of the standard CG sequence context [3, 4]. Additionally, loss of DNA methylation in the met1 mutant correlates with a loss of H3K9me2 [5, 6]. Here we show that KYP-dependent H3K9me2 is found at non-CG methylation sites in addition to those rich in CG methylation. Furthermore, we show that the SRA domain of KYP binds directly to methylated DNA, and SRA domains with missense mutations found in loss-of-function kyp mutants have reduced binding to methylated DNA in vitro. These data suggest that DNA methylation is required for the recruitment or activity of KYP and suggest a self-reinforcing loop between histone and DNA methylation. Lastly, we found that SRA domains from two Arabidopsis SRA-RING proteins also bind methylated DNA and that the SRA domains from KYP and SRA-RING proteins prefer methylcytosines in different sequence contexts. Hence, unlike the methyl-binding domain (MBD), which binds only methylated-CpG sequences, the SRA domain is a versatile new methyl-DNA-binding motif.", 
"PcG proteins mediate heritable transcriptional silencing by generating and recognizing covalent histone modifications. One conserved PcG complex, PRC2, is composed of several proteins including the histone methyltransferase (HMTase) Ezh2, the WD-repeat protein Eed, and the Zn-finger protein Suz12. Ezh2 methylates histone H3 on lysine 27 (H3K27) [1-4], which serves as an epigenetic mark mediating silencing. H3K27 can be mono-, di-, or trimethylated (1 mH3K27, 2mH3K27, and 3mH3K27, respectively) (5]. Hence, either PRC2 must be regulated so as to add one methyl group to certain nucleosomes but two or three to others, or distinct complexes must be responsible for 1m-, 2m-, and 3mH3K27. Consistent with the latter possibility, 2mH3K27 and 3mH3K27, but not 1mH3K27, are absent in Suz12(-/-) embryos, which lack both Suz12 and Ezh2 protein [6]. Mammalian proteins required for 1mH3K27 have not been identified. Here, we demonstrate that unlike Suz12 and Ezh2, Eed is required not only for 2m- and 3mH3K27 but also global 1mH3K27. These results provide a functionally important distinction between PRC2 complex components and implicate Eed in PRC2-independent histone methylation.", 
"Regulation of gene expression is considered a plausible mechanism of drug addiction, given the stability of behavioural abnormalities that define an addicted state. Among many transcription factors known to influence the addiction process, one of the best characterized is Delta FosB, which is induced in the brain's reward regions by chronic exposure to virtually all drugs of abuse and mediates sensitized responses to drug exposure. Since Delta FosB is a highly stable protein, it represents a mechanism by which drugs produce lasting changes in gene expression long after the cessation of drug use. Studies are underway to explore the detailed molecular mechanisms by which Delta FosB regulates target genes and produces its behavioural effects. We are approaching this question using DNA expression arrays coupled with the analysis of chromatin remodelling-changes in the posttranslational modifications of histones at drug-regulated gene promoters-to identify genes that are regulated by drugs of abuse via the induction of Delta FosB and to gain insight into the detailed molecular mechanisms involved. Our findings establish chromatin remodelling as an important regulatory mechanism underlying drug-induced behavioural plasticity, and promise to reveal fundamentally new insight into how Delta FosB contributes to addiction by regulating the expression of specific target genes in brain reward pathways.", 
"The centromere/kinetochore complex is indispensable for accurate segregation of chromosomes during cell divisions when it serves as the attachment site for spindle microtubules. Centromere identity in metazoans is believed to be governed by epigenetic mechanisms, because the highly repetitive centromeric DNA is neither sufficient nor required for specifying the assembly site of the kinetochore [1-4]. A candidate for an epigenetic mark is the centromere-specific histone H3 variant CENP-A that replaces H3 in alternating blocks of chromatin exclusively in active centromeres [1, 2, 5, 6]. CENP-A acts as an initiator of kinetochore assembly, but the detailed dynamics of the deposition of metazoan CENP-A and of other constitutive kinetochore components are largely unknown [1, 2, 7-10]. Here we show by quantitative fluorescence measurements in living early embryos that functional fluorescent fusion proteins of the Drosophila CENP-A and CENP-C homologs are rapidly incorporated into centromeres during anaphase. This incorporation is independent of ongoing DNA synthesis and pulling forces generated by the mitotic spindle, but strictly coupled to mitotic progression. Thus, our findings uncover a strikingly dynamic behavior of centromere components in anaphase.", 
"That there is a heritable or familial component of susceptibility to chronic non-communicable diseases such as type 2 diabetes, obesity and cardiovascular disease is well established, but there is increasing evidence that some elements of such heritability are transmitted non-genomically and that the processes whereby environmental influences act during early development to shape disease risk in later life can have effects beyond a single generation. Such heritability may operate through epigenetic mechanisms involving regulation of either imprinted or non-imprinted genes but also through broader mechanisms related to parental physiology or behaviour. We review evidence and potential mechanisms for non-genomic transgenerational inheritance of 'lifestyle' disease and propose that the 'developmental origins of disease' phenomenon is a maladaptive consequence of an ancestral mechanism of developmental plasticity that may have had adaptive value in the evolution of generalist species such as Homosapiens. (c) 2007 Wiley Periodicals, Inc.", 
"As is the case with any metaphor, parental effects mean different things to different biologists-from developmental induction of novel phenotypic variation to an evolved adaptation, and from epigenetic transference of essential developmental resources to a stage of inheritance and ecological succession. Such a diversity of perspectives illustrates the composite nature of parental effects that, depending on the stage of their expression and whether they are considered a pattern or a process, combine the elements of developmental induction, homeostasis, natural selection, epigenetic inheritance and historical persistence. Here, we suggest that by emphasizing the complexity of causes and influences in developmental systems and by making explicit the links between development, natural selection and inheritance, the study of parental effects enables deeper understanding of developmental dynamics of life cycles and provides a unique opportunity to explicitly integrate development and evolution. We highlight these perspectives by placing parental effects in a wider evolutionary framework and suggest that far from being only an evolved static outcome of natural selection, a distinct channel of transmission between parents and offspring, or a statistical abstraction, parental effects on development enable evolution by natural selection by reliably transferring developmental resources needed to reconstruct, maintain and modify genetically inherited components of the phenotype. The view of parental effects as an essential and dynamic part of an evolutionary continuum unifies mechanisms behind the origination, modification and historical persistence of organismal form and function, and thus brings us closer to a more realistic understanding of life's complexity and diversity.", 
"Over the past few years we have seen an odd change, or extension, in the use of the word 'epigenetic' when describing matters of gene regulation in eukaryotes. Although it may generally tie that it is not worth arguing over definitions, this is true only insofar as the participants in the discussion know what each other means. I believe the altered use of the term carries baggage from the standard definition that can have misleading implications. Here I wish to probe our use of language in this way, and to show how such a discussion leads to some more general considerations concerning gene regulation.", 
"Polyploidy, or the presence of two or more diploid parental genome sets within an organism, is found to an amazing degree in higher plants. In addition, many plant species traditionally considered to be diploid have recently been demonstrated to have undergone rounds of genome duplication in the past and are now referred to as paleopolyploids. Polyploidy and interspecific hybridisation (with which it is often associated) have long been thought to be important mechanisms of rapid species formation. The widespread occurrence of polyploids, which are frequently found in habitats different from that of their diploid progenitors, would seem to indicate that polyploidy is associated with evolutionary success in terms of the ability to colonise new environmental niches. A flurry of recent genomic studies has provided fresh insights into the potential basis of the phenotypic novelty of polyploid species. Here we review current knowledge of genetic, epigenetic, and transcriptional changes associated with polyploidy in plants and assess how these changes might contribute to the evolutionary success of polyploid plants. We conclude by stressing the need for field-based experiments to determine whether genetic changes associated with polyploidy are indeed adaptive.", 
"The human pathogen Candida albicans can assume either of two distinct cell types, designated ``white'' and ``opaque.'' Each cell type is maintained for many generations; switching between them is rare and stochastic, and occurs without any known changes in the nucleotide sequence of the genome. The two cell types differ dramatically in cell shape, colony appearance, mating competence, and virulence properties. In this work, we investigate the transcriptional circuitry that specifies the two cell types and controls the switching between them. First, we identify two new transcriptional regulators of white-opaque switching, Czf1 and white-opaque regulator 2 (Wor2). Analysis of a large set of double mutants and ectopic expression strains revealed genetic relationships between CZF1, WOR2, and two previously identified regulators of white-opaque switching, WOR1 and EFG1. Using chromatin immunoprecipitation, we show that Wor1 binds the intergenic regions upstream of the genes encoding three additional transcriptional regulators of white-opaque switching (CZF1, EFG1, and WOR2), and also occupies the promoters of numerous white-and opaque-enriched genes. Based on these interactions, we have placed these four genes in a circuit controlling white-opaque switching whose topology is a network of positive feedback loops, with the master regulator gene WOR1 occupying a central position. Our observations indicate that a key role of the interlocking feedback loop network is to stably maintain each epigenetic state through many cell divisions.", 
"Limitation of infarct size is a major goal of therapy for acute coronary syndromes, and research has focused on achieving rapid patency of infarct-related vessels. However, new understandings of epigenetic modifications during ischemia suggest additional targeted approaches that have not been extensively explored. Here, we show that ischemia induces histone deacetylase (HDAC) activity in the heart with deacetyladon of histones H3/4 in vitro and in vivo. We show, utilizing a standard murine model of ischemia-reperfusion, that chemical HDAC inhibitors significantly reduce infarct area, even when delivered 1 h after the ischemic insult. We demonstrate that HDAC inhibitors prevent ischemia-induced activation of gene programs that include hypoxia inducible factor-lot, cell death, and vascular permeability in vivo and in vitro, thus providing potential mechanisms to explain reduced vascular leak and myocardial injury. In vitro, siRNA knockdown experiments implicate HDAC4 as a mediator of the effects in ischemic cardiac myocytes. These results demonstrate that HDAC inhibitors alter the response to ischemic injury in the heart and reduce infarct size, suggesting novel therapeutic approaches for acute coronary syndromes.", 
"Heart valve anomalies are some of the most common congenital heart defects, yet neither the genetic nor the epigenetic forces guiding heart valve development are well understood. When functioning normally, mature heart valves prevent intracardiac retrograde blood flow; before valves develop, there is considerable regurgitation, resulting in reversing (or oscillatory) flows between the atrium and ventricle. As reversing flows are particularly strong stimuli to endothelial cells in culture, an attractive hypothesis is that heart valves form as a developmental response to retrograde blood flows through the maturing heart. Here, we exploit the relationship between oscillatory flow and heart rate to manipulate the amount of retrograde flow in the atrioventricular (AV) canal before and during valvulogenesis, and find that this leads to arrested valve growth. Using this manipulation, we determined that klf2a is normally expressed in the valve precursors in response to reversing flows, and is dramatically reduced by treatments that decrease such flows. Experimentally knocking down the expression of this shear-responsive gene with morpholine antisense oligonucleotides (MOs) results in dysfunctional valves. Thus, klf2a expression appears to be necessary for normal valve formation. This, together with its dependence on intracardiac hemodynamic forces, makes klf2a expression an early and reliable indicator of proper valve development. Together, these results demonstrate a critical role for reversing flows during valvulogenesis and show how relatively subtle perturbations of normal hemodynamic patterns can lead to both major alterations in gene expression and severe valve dysgenesis.", 
"The stunning possibility of reprogramming differentiated somatic cells to express a pluripotent stem cell phenotype (iPS, induced pluripotent stem cell) and the ground state character of pluripotency reveal fundamental features of cell fate regulation that lie beyond existing paradigms. The rarity of reprogramming events appears to contradict the robustness with which the unfathomably complex phenotype of stem cells can reliably be generated. This apparent paradox, however, is naturally explained by the rugged epigenetic landscape with valleys representing preprogrammed', attractor states that emerge from the dynamical constraints of the gene regulatory network. This article provides a pedagogical primer to the fundamental principles of gene regulatory networks as integrated dynamic systems and reviews recent insights in gene expression noise and fate determination, thereby offering a formal framework that may help us to understand why cell fate reprogramming events are inherently rare and yet so robust.", 
"Polycomb group ( PcG) proteins are able to maintain the memory of silent transcriptional states of homeotic genes throughout development. In Drosophila, they form multimeric complexes that bind to specific DNA regulatory elements named PcG response elements ( PREs). To date, few PREs have been identified and the chromosomal distribution of PcG proteins during development is unknown. We used chromatin immunoprecipitation ( ChIP) with genomic tiling path microarrays to analyze the binding profile of the PcG proteins Polycomb ( PC) and Polyhomeotic ( PH) across 10 Mb of euchromatin. We also analyzed the distribution of GAGA factor ( GAF), a sequence-specific DNA binding protein that is found at most previously identified PREs. Our data show that PC and PH often bind to clustered regions within large loci that encode transcription factors which play multiple roles in developmental patterning and in the regulation of cell proliferation. GAF co-localizes with PC and PH to a limited extent, suggesting that GAF is not a necessary component of chromatin at PREs. Finally, the chromosome-association profile of PC and PH changes during development, suggesting that the function of these proteins in the regulation of some of their target genes might be more dynamic than previously anticipated.", 
"Sexual reproduction involves epigenetic reprogramming [1] comprising DNA methylation [2] and histone modifications [3-6]. In addition, dynamics of HISTONE3 (H3) variant H3.3 upon fertilization are conserved in animals, suggesting an essential role [7-9]. In contrast to H3, H3.3 marks actively transcribed regions of the genome and can be deposited in a replication-independent manner [10, 11]. Although H3 variants are conserved in plants, their dynamics during fertilization have remained unexplored. We overcame technical limitations to live imaging of the fertilization process in Arabidopsis thaliana and studied dynamics of the male-gamete-specific H3.3 [12] and the centromeric Histone Three Related 12 (HTR12) [13]. The double-fertilization process in plants produces the zygote and the embryo-nourishing endosperm [14]. We show that the zygote is characterized by replication-independent removal of paternal H3.3 and homogeneous incorporation of parental chromatin complements. In the endosperm, the paternal H3.3 is passively diluted by replication while the paternal chromatin remains segregated apart from the maternal chromatin (gonomery). Hence epigenetic regulations distinguish the two products of fertilization in plants. H3.3-replication-independent dynamics and gonomery also mark the first zygotic divisions in animal species [5, 15]. We thus propose the convergent selection of parental epigenetic imbalance involving H3 variants in sexually reproducing organisms.", 
"Centromeres, the specialized chromatin structures that are responsible for equal segregation of chromosomes at mitosis, are epigenetically maintained by a centromere-specific histone H3 variant (CenH3). However, the mechanistic basis for centromere maintenance is unknown. We investigated biochemical properties of CenH3 nucleosomes from Drosophila melanogaster cells. Cross-linking of CenH3 nucleosomes identifies heterotypic tetramers containing one copy of CenH3, H2A, H2B, and H4 each. Interphase CenH3 particles display a stable association of approximately 120 DNA base pairs. Purified centromeric nucleosomal arrays have typical beads-on-a-string'' appearance by electron microscopy but appear to resist condensation under physiological conditions. Atomic force microscopy reveals that native CenH3-containing nucleosomes are only half as high as canonical octameric nucleosomes are, confirming that the tetrameric structure detected by cross-linking comprises the entire interphase nucleosome particle. This demonstration of stable half-nucleosomes in vivo provides a possible basis for the instability of centromeric nucleosomes that are deposited in euchromatic regions, which might help maintain centromere identity.", 
"It is widely suspected that gene regulatory networks are highly plastic. The rapid turnover of transcription factor binding sites has been predicted on theoretical grounds and has been experimentally demonstrated in closely related species. We combined experimental approaches with comparative genomics to focus on the role of combinatorial control in the evolution of a large transcriptional circuit in the fungal lineage. Our study centers on Mcm1, a transcriptional regulator that, in combination with five cofactors, binds roughly 4% of the genes in Saccharomyces cerevisiae and regulates processes ranging from the cell-cycle to mating. In Kluyveromyces lactis and Candida albicans, two other hemiascomycetes, we find that the Mcm1 combinatorial circuits are substantially different. This massive rewiring of the Mcm1 circuitry has involved both substantial gain and loss of targets in ancient combinatorial circuits as well as the formation of new combinatorial interactions. We have dissected the gains and losses on the global level into subsets of functionally and temporally related changes. One particularly dramatic change is the acquisition of Mcm1 binding sites in close proximity to Rap1 binding sites at 70 ribosomal protein genes in the K. lactis lineage. Another intriguing and very recent gain occurs in the C. albicans lineage, where Mcm1 is found to bind in combination with the regulator Wor1 at many genes that function in processes associated with adaptation to the human host, including the white-opaque epigenetic switch. The large turnover of Mcm1 binding sites and the evolution of new Mcm1-cofactor interactions illuminate in sharp detail the rapid evolution of combinatorial transcription networks.", 
"Background: Heterochromatin is chromosomal material that remains condensed throughout the cell division cycle and silences genes nearby. It is found in almost all eukaryotes, and although discovered (in plants) almost 100 years ago, the mechanism by which heterochromatin is inherited has remained obscure. Heterochromatic silencing and histone H3 lysine-9 methylation (H3K9me2) depend, paradoxically, on heterochromatic transcription and RNA interference (RNAi). Results: Here, we show that heterochromatin protein 1 in fission yeast (Swi6) is lost via phosphorylation of H3 serine 10 (H3S10) during mitosis, allowing heterochromatic transcripts to transiently accumulate in S phase. Rapid processing of these transcripts into small interfering RNA (siRNA) promotes restoration of H3K9me2 and Swi6 after replication when cohesin is recruited. We also show that RNAi in fission yeast is inhibited at high temperatures, providing a plausible mechanism for epigenetic phenomena that depend on replication and temperature, such as vernalization in plants and position effect variegation in animals. Conclusions: These results explain how silent heterochromatin can be transcribed and lead to a model for epigenetic inheritance during replication.", 
"Vernalization promotes flowering in Arabidopsis through epigenetic repression of the floral repressor, FLOWERING LOCUS C (FLC) [1-3]. Vernalization, like other polycomb-mediated repression events [4], occurs in two stages; FLCrepression is established at low temperatures, then maintained during subsequent growth at 22 degrees C [5, 6]. Low temperatures induce VIN3 activity, which is required for changes in histone modifications and the associated FLC repression [3]. Plant polycomb proteins FIE, VRN2, CLF, and SWIN, together with VIN3, form a complex that adds histone H3 lysine 27 methylation at FLC in vernalized plants [2,3,7]. VRN1 and LHP1 are required for maintenance of FLC repression [2, 3, 6, 8]. Tissue must be undergoing cell division during lowtemperature treatments for acceleration of flowering to occur [9, 10]. We show that low-temperature treatments repress FLC in cells that are not mitotically active, but this repression is notfully maintained. Trimethyl-lysine 27 (K27me3), is enriched at the start of the FLC gene during the cold, before spreading across the locus after vernalization. In the absence of DNA replication, K27me3 is added to chromatin at the start of FLC but is removed on return to 22 degrees C. This suggests that DNA replication is essential for maintenance of vernalization-induced repression of FLC.", 
"Ionizing radiation/Bystander effects/Signaling pathways/Plasma membrane/Radiotherapy. A rapidly growing body of experimental evidence indicates that ionizing radiation induces biological effects in non-irradiated bystander cells that have received signals from adjacent or distant irradiated cells. This phenomenon, which has been termed the ionizing radiation-induced bystander effect, challenges the long-standing paradigm that radiation traversal through the nucleus of a cell is a prerequisite to elicit genetic damage or a biological response. Bystander effects have been observed in a number of experimental systems, and cells whose nucleus or cytoplasm is irradiated exert bystander responses. Bystander cells manifest a multitude of biological consequences, such as genetic and epigenetic changes, alterations in gene expression, activation of signal transduction pathways, and delayed effects in their progeny. Several mediating mechanisms have been proposed. These involve gap junction-mediated intercellular communication, secreted soluble factors, oxidative metabolism, plasma membrane-bound lipid rafts, and calcium fluxes. This paper reviews briefly the current knowledge of the bystander effect with a focus on proposed mechanisms. The potential benefit of bystander effects to cancer radiotherapy will also be discussed.", 
"Vernalization, the acceleration of flowering by the prolonged cold of winter, ensures that plants flower in favorable spring conditions. During vernalization in Arabidopsis, cold temperatures repress FLOWERING LOCUS C (FLC) expression [1, 2] in a mechanism involving VERNALIZATION INSENSITIVE 3 (VIN3) [3], and this repression is epigenetically maintained by a Polycomb-like chromatin regulation involving VERNALIZATION 2 (VRN2), a Su(z)12 homolog, VERNALIZATION 1 (VRN1), and LIKE-HETEROCHROMATIN PROTEIN 1 [4-8]. In order to further elaborate how cold repression triggers epigenetic silencing, we have targeted mutations that result in FLC misexpression both at the end of the prolonged cold and after subsequent development. This identified VERNALIZATION 5 (VRN5), a PHD finger protein and homolog of VIN3. Our results suggest that during the prolonged cold, VRN5 and VIN3 form a heterodimer necessary for establishing the vernalization-induced chromatin modifications, histone deacetylation, and H3 lysine 27 trimethylation required for the epigenetic silencing of FLC. Double mutant and FLCmisexpression analyses reveal additional VRN5 functions, both FLC-dependent and -independent, and indicate a spatial complexity to FLC epigenetic silencing with VRN5 acting as a common component in multiple pathways.", 
"It is usually thought that the development of complex organisms is controlled by protein regulatory factors and morphogenetic signals exchanged between cells and differentiating tissues during ontogeny. However, it is now evident that the majority of all animal genomes is transcribed, apparently in a developmentally regulated manner, suggesting that these genomes largely encode RNA machines and that there may be a vast hidden layer of RNA regulatory transactions in the background. I propose that the epigenetic trajectories of differentiation and development are primarily programmed by feedforward RNA regulatory networks and that most of the information required for multicellular development is embedded in these networks, with cell-cell signalling required to provide important positional information and to correct stochastic errors in the endogenous RNA-directed program.", 
"Handedness and cerebral asymmetry are commonly assumed to be uniquely human, and even defining characteristics of our species. This is increasingly refuted by the evidence of behavioural asymmetries in non-human species. Although complex manual skill and language are indeed unique to our species and are represented asymmetrically in the brain, some non-human asymmetries appear to be precursors, and others are shared between humans and non-humans. In all behavioural and cerebral asymmetries so far investigated, a minority of individuals reverse or negate the dominant asymmetry, suggesting that such asymmetries are best understood in the context of the overriding bilateral symmetry of the brain and body, and a trade-off between the relative advantages and disadvantages of symmetry and asymmetry. Genetic models of handedness, for example, typically postulate a gene with two alleles, one disposing towards right-handedness and the other imposing no directional influence. There is as yet no convincing evidence as to the location of this putative gene, suggesting that several genes may be involved, or that the gene may be monomorphic with variations due to environmental or epigenetic influences. Nevertheless, it is suggested that, in behavioural, neurological and evolutionary terms, it may be more profitable to examine the degree rather than the direction of asymmetry.", 
"Background: Epigenetic regulation by diverse classes of small RNAs is mediated by the highly conserved Argonaute/Piwi family of proteins. Although Argonautes are broadly expressed, the Piwi subfamily primarily functions in the germ line. Piwi proteins are associated with germline-specific ribonucleoprotein (RNP) granules in Drosophila, zebrafish, and mouse. Depending on the species and on the specific family member, Piwis play important roles in spermatogenesis and/or in maintaining germ cell and stem cell totipotency. Piwis bind to a newly discovered class of small RNAs, called piRNAs. C. elegans contains a large set of Argonaute/Piwi-related proteins, including two closely related to piwi called prg-1 and prg-2. The function of prg-1 and prg-2 and whether piRNAs exist in C. elegans is unknown. Results: Here, we demonstrate that the Piwi-like protein PRG-1 is localized to P granules in germ cells entering spermatogenesis and is required for successful spermatogenesis. Loss of prg-1 causes a marked reduction in expression of a subset of mRNAs expressed during spermatogenesis, and prg-1 mutant sperm exhibit extensive defects in activation and fertilization. Moreover, prg-1 activity is required for the presence of the small RNAs called 21 U-RNAs. Conclusions: Our data suggest that PRG-1 promotes expression, processing, or stability of 21 U-RNAs, which, in turn or in concert with PRG-1, promote proper expression of spermatogenesis transcripts.", 
"Background: Adipose tissues serve not only as a store for energy in the form of lipid, but also as endocrine tissues that regulates metabolic activities of the organism by secreting various kinds of hormones. Peroxisome proliferator activated receptor. (PPAR gamma) is a key regulator of adipocyte differentiation that induces the expression of adipocyte-specific genes in preadipocytes and mediates their differentiation into adipocytes. Furthermore, PPAR gamma has an important role to maintain the physiological function of mature adipocyte by controlling expressions of various genes properly. Therefore, any reduction in amount and activity of PPAR gamma is linked to the pathogenesis of metabolic syndrome. Results: In this study, we investigated the contribution of epigenetic transcriptional regulatory mechanisms, such as DNA methylation, to the expression of the PPAR gamma gene, and further evaluated the contribution of such epigenetic regulatory mechanisms to the pathogenesis of metabolic syndrome. In 3T3-L1 preadipocytes, the promoter of the PPAR gamma 2 gene was hypermethylated, but was progressively demethylated upon induction of differentiation, which was accompanied by an increase of mRNA expression. Moreover, treatment of cells with 5'-aza-cytideine, an inhibitor of DNA methylation, increased expression of the PPAR gamma gene in a dose-dependent manner. Methylation in vitro of a PPAR gamma promoter-driven reporter construct also repressed the transcription of a downstream reporter gene. These results suggest that the expression of the PPAR gamma gene is inhibited by methylation of its promoter. We next compared the methylation status of the PPAR gamma promoters in adipocytes from wild-type (WT) mice with those from two diabetic mouse models: +Lepr(db)/+Lepr(db) and diet-induced obesity mice. Interestingly, we found increased methylation of the PPAR gamma promoter in visceral adipose tissues (VAT) of the mouse models of diabetes, compared to that observed in wild-type mice. We observed a concomitant decrease in the level of PPAR gamma mRNA in the diabetic mice compared to the WT mice. Conclusion: We conclude that the expression of PPAR gamma gene is regulated by DNA methylation of its promoter region and propose that reduced expression of PPAR gamma owing to DNA methylation in adipocytes of the VAT may contribute to the pathogenesis of metabolic syndrome.", 
"Developing embryos exhibit a robust capability to reduce phenotypic variations that occur naturally or as a result of experimental manipulation. This reduction in variation occurs by an epigenetic mechanism called canalization, a phenomenon which has resisted understanding because of a lack of necessary molecular data and of appropriate gene regulation models. In recent years, quantitative gene expression data have become available for the segment determination process in the Drosophila blastoderm, revealing a specific instance of canalization. These data show that the variation of the zygotic segmentation gene expression patterns is markedly reduced compared to earlier levels by the time gastrulation begins, and this variation is significantly lower than the variation of the maternal protein gradient Bicoid. We used a predictive dynamical model of gene regulation to study the effect of Bicoid variation on the downstream gap genes. The model correctly predicts the reduced variation of the gap gene expression patterns and allows the characterization of the canalizing mechanism. We show that the canalization is the result of specific regulatory interactions among the zygotic gap genes. We demonstrate the validity of this explanation by showing that variation is increased in embryos mutant for two gap genes, Kruppel and knirps, disproving competing proposals that canalization is due to an undiscovered morphogen, or that it does not take place at all. In an accompanying article in PLoS Computational Biology (doi:10.1371/journal:pcbi.1000303), we show that cross regulation between the gap genes causes their expression to approach dynamical attractors, reducing initial variation and providing a robust output. These results demonstrate that the Bicoid gradient is not sufficient to produce gap gene borders having the low variance observed, and instead this low variance is generated by gap gene cross regulation. More generally, we show that the complex multigenic phenomenon of canalization can be understood at a quantitative and predictive level by the application of a precise dynamical model.", 
"Fertilization in flowering plants initiates the development of the embryo and endosperm, which nurtures the embryo. A few genes subjected to imprinting are expressed in endosperm from their maternal allele, while their paternal allele remains silenced [1-3]. Imprinting of the FWA gene involves DNA methylation [4]. Mechanisms controlling imprinting of the Polycomb group (Pc-G) gene MEDEA (MEA) [5] are not yet fully understood [6-10]. Here we report that MEA imprinting is regulated by histone methylation. This epigenetic chromatin modification is mediated by several Pc-G activities during the entire plant life cycle. We show that Pc-G complexes maintain MEA transcription silenced throughout vegetative life and male gametogenesis. In endosperm, the maternal allele of MEA encodes an essential component of a Pc-G complex, which maintains silencing of the paternal MEA allele. Hence, we conclude that a feedback loop controls MEA imprinting. This feedback loop ensures a complete maternal control of MEA expression from both parental alleles and might have provided a template for evolution of imprinting in plants.", 
"Vertebrate hearts depend on highly specialized cardiomyocytes that form the cardiac conduction system (CCS) to coordinate chamber contraction and drive blood efficiently and unidirectionally throughout the organism. Defects in this specialized wiring system can lead to syncope and sudden cardiac death. Thus, a greater understanding of cardiac conduction development may help to prevent these devastating clinical outcomes. Utilizing a cardiac-specific fluorescent calcium indicator zebrafish transgenic line, Tg(cmlc2:gCaMP)(s878), that allows for in vivo optical mapping analysis in intact animals, we identified and analyzed four distinct stages of cardiac conduction development that correspond to cellular and anatomical changes of the developing heart. Additionally, we observed that epigenetic factors, such as hemodynamic flow and contraction, regulate the fast conduction network of this specialized electrical system. To identify novel regulators of the CCS, we designed and performed a new, physiology-based, forward genetic screen and identified for the first time, to our knowledge, 17 conduction-specific mutations. Positional cloning of hobgoblin(s634) revealed that tcf2, a homeobox transcription factor gene involved in mature onset diabetes of the young and familial glomerulocystic kidney disease, also regulates conduction between the atrium and the ventricle. The combination of the Tg(cmlc2:gCaMP)(s878) line/in vivo optical mapping technique and characterization of cardiac conduction mutants provides a novel multidisciplinary approach to further understand the molecular determinants of the vertebrate CCS.", 
"Bacterial LPS triggers dramatic changes in gene expression in macrophages. We show here that LPS regulated several members of the histone deacetylase (HDAC) family at the mRNA level in murine bone marrow-derived macrophages (BMM). LPS transiently repressed, then induced a number of HDACs (Hdac-4, 5, 7) in BMM, whereas Hdac-1 mRNA was induced more rapidly. Treatment of BMM with trichostatin A (TSA), an inhibitor of HDACs, enhanced LPS-induced expression of the Cox-2, Cxcl2, and Ifit2 genes. In the case of Cox-2, this effect was also apparent at the promoter level. Overexpression of Hdac-8 in RAW264 murine macrophages blocked the ability of LPS to induce Cox-2 mRNA. Another class of LPS-inducible genes, which included Ccl2, Ccl7, and Edn1, was suppressed by TSA, an effect most likely mediated by PU.1 degradation. Hence, HDACs act as potent and selective negative regulators of proinflammatory gene expression and act to prevent excessive inflammatory responses in macrophages.", 
"Cytoadherance of Plasmodium falciparum-infected erythrocytes in the brain, organs and peripheral microvasculature is linked to morbidity and mortality associated with severe malaria. Parasite-derived P. falciparum Erythrocyte Membrane Protein 1 (PfEMP1) molecules displayed on the erythrocyte surface are responsible for cytoadherance and undergo antigenic variation in the course of an infection. Antigenic variation of PfEMP1 is achieved by in situ switching and mutually exclusive transcription of the var gene family, a process that is controlled by epigenetic mechanisms. Here we report characterisation of the P. falciparum silent information regulator's A and B (PfSir2A and PfSir2B) and their involvement in mutual exclusion and silencing of the var gene repertoire. Analysis of P. falciparum parasites lacking either PfSir2A or PfSir2B shows that these NAD(+)-dependent histone deacetylases are required for silencing of different var gene subsets classified by their conserved promoter type. We also demonstrate that in the absence of either of these molecules mutually exclusive expression of var genes breaks down. We show that var gene silencing originates within the promoter and PfSir2 paralogues are involved in cis spreading of silenced chromatin into adjacent regions. Furthermore, parasites lacking PfSir2A but not PfSir2B have considerably longer telomeric repeats, demonstrating a role for this molecule in telomeric end protection. This work highlights the pivotal but distinct role for both PfSir2 paralogues in epigenetic silencing of P. falciparum virulence genes and the control of pathogenicity of malaria infection.", 
"Background: The telomere is a specialized heterochromatin conserved among eukaryotes. However, it remains unknown how heterochromatin protein 1 (HP1) is recruited to telomeres and how telomere heterochromatin is formed. In fission yeast, the RNAi (RNA interference)-RITS (RNA-induced initiation of transcriptional silencing) pathway initiates heterochromatin formation at the centromeres and the silent mat locus by using common DNA sequences, the dg and dh repeats, as the templates for small interfering RNA (siRNA). Results: We found that telomeric repeats are sufficient for the establishment of Swi6 (a fission-yeast HP1 homolog) heterochromatin, and the establishment requires Taz1, a telomere binding protein of the TRF family. Additionally, Swi6 heterochromatin is established by a part of the subtelomere that contains sequences highly homologous to that of the dh repeat, and it is strikingly destabilized by the deletion of both Taz1 and RNAi-RITS. Transcripts from the telomeric dh-homologous region were specifically associated with RITS, and deletion of the telomeric dh-homologous region showed the phenotype similar to that of the rnai mutant in terms of the telomeric silencing, indicating that the RNAi-RITS pathway acts at the telomeric dh-homologous region to establish Swi6 heterochromatin. Furthermore, we found that Taz1 establishes Swi6 heterochromatin independently of the telomeric repeats and the RNAi-RITS pathway at the subtelomeres. Conclusion: The telomere heterochromatin is regulated by at least two factors: One is Taz1, which is telomere specific, and the other is RNAi-RITS, which is commonly used at the constitutive heterochromatin regions.", 
"Imprinted genes are expressed either from the maternally or paternally inherited copy only, and they play a key role in regulating complex biological processes, including offspring development and mother-offspring interactions. There are several competing theories attempting to explain the evolutionary origin of this monoallelic pattern of gene expression, but a prevailing view has emerged that holds that genomic imprinting is a consequence of conflict between maternal and paternal gene copies over maternal investment. However, many imprinting patterns and the apparent overabundance of maternally expressed genes remain unexplained and may be incompatible with current theory. Here we demonstrate that sole expression of maternal gene copies is favored by natural selection because it increases the adaptive integration of offspring and maternal genomes, leading to higher offspring fitness. This novel coadaptation theory for the evolution of genomic imprinting is consistent with results of recent studies on epigenetic effects, and it provides a testable hypothesis for the origin of previously unexplained major imprinting patterns across different taxa. In conjunction with existing hypotheses, our results suggest that imprinting may have evolved due to different selective pressures at different loci.", 
"The origin of novel traits is what draws many to evolutionary biology, yet our understanding of the mechanisms that underlie the genesis of novelty remains limited. Here I review definitions of novelty including its relationship to homology. I then discuss how ontogenetic perspectives may allow us to move beyond current roadblocks in our understanding of the mechanics of innovation. Specifically, I explore the roles of canalization, plasticity and threshold responses during development in generating a reservoir of cryptic genetic variation free to drift and accumulate in natural populations. Environmental or genetic perturbations that exceed the buffering capacity of development can then release this variation, and, through evolution by genetic accommodation, result in rapid diversification, recurrence of lost phenotypes as well as the origins of novel features. I conclude that, in our quest to understand the nature of innovation, the nature of development deserves to take center stage.", 
"Many DNA hypermethylated and epigenetically silenced genes in adult cancers are Polycomb group (PcG) marked in embryonic stem (ES) cells. We show that a large region upstream (similar to 30 kb) of and extending similar to 60 kb around one such gene, GATA-4, is organized-in Tera-2 undifferentiated embryonic carcinoma (EC) cells-in a topologically complex multi-loop conformation that is formed by multiple internal long-range contact regions near areas enriched for EZH2, other PcG proteins, and the signature PcG histone mark, H3K27me3. Small interfering RNA (siRNA)-mediated depletion of EZH2 in undifferentiated Tera-2 cells leads to a significant reduction in the frequency of long-range associations at the GATA-4 locus, seemingly dependent on affecting the H3K27me3 enrichments around those chromatin regions, accompanied by a modest increase in GATA-4 transcription. The chromatin loops completely dissolve, accompanied by loss of PcG proteins and H3K27me3 marks, when Tera-2 cells receive differentiation signals which induce a similar to 60-fold increase in GATA-4 expression. In colon cancer cells, however, the frequency of the long-range interactions are increased in a setting where GATA-4 has no basal transcription and the loops encompass multiple, abnormally DNA hypermethylated CpG islands, and the methyl-cytosine binding protein MBD2 is localized to these CpG islands, including ones near the gene promoter. Removing DNA methylation through genetic disruption of DNA methyltransferases (DKO cells) leads to loss of MBD2 occupancy and to a decrease in the frequency of long-range contacts, such that these now more resemble those in undifferentiated Tera-2 cells. Our findings reveal unexpected similarities in higher order chromatin conformation between stem/precursor cells and adult cancers. We also provide novel insight that PcG-occupied and H3K27me3-enriched regions can form chromatin loops and physically interact in cis around a single gene in mammalian cells. The loops associate with a poised, low transcription state in EC cells and, with the addition of DNA methylation, completely repressed transcription in adult cancer cells.", 
"Background: Centromeres are cis-acting chromosomal domains that direct kinetochore formation, enabling faithful chromosome segregation. Centromeric regions of higher eukaryotes are structurally complex, consisting of various epigenetically modified chromatin types including specialized chromatin at the kinetochore itself, pericentromeric heterochromatin, and flanking euchromatin. Although the features necessary for the establishment and maintenance of discrete chromatin domains remain poorly understood, two models have been proposed based either on the passive convergence of competing activities involved in individual domain formation or, alternatively, on the action of specific genomic sequences and associated proteins to actively block the propagation of one chromatin type into another. Results: Functional analysis of centromeric sequences located at the intersection of Schizosaccharomyces pombe central core chromatin and outer repeat heterochromatin identified a chromatin barrier that contains a transfer RNA (tRNA) gene. Deletion or modification of the barrier sequences result in the propagation of pericentromeric heterochromatin beyond its normal boundary. The tRNA gene is transcriptionally active, and barrier activity requires sequences necessary for RNA polymerase III transcription. Moreover, absence of the barrier results in abnormal meiotic chromosome segregation. Conclusions: The identification of DNA sequences with chromatin barrier activity at the fission yeast centromere provides a model for establishment of centromeric chromatin domains in higher eukaryotes.", 
"Tandem repeat sequences are frequently associated with gene silencing phenomena. The Arabidopsis thaliana FWA gene contains two tandem repeats and is an efficient target for RNA-directed de novo DNA methylation when it is transformed into plants. We showed that the FWA tandem repeats are necessary and sufficient for de novo DNA methylation and that repeated character rather than intrinsic sequence is likely important. Endogenous FWA can adopt either of two stable epigenetic states: methylated and silenced or unmethylated and active. Surprisingly, we found small interfering RNAs ( siRNAs) associated with FWA in both states. Despite this, only the methylated form of endogenous FWA could recruit further RNA-directed DNA methylation or cause efficient de novo methylation of transgenic FWA. This suggests that RNA-directed DNA methylation occurs in two steps: first, the initial recruitment of the siRNA-producing machinery, and second, siRNA-directed DNA methylation either in cis or in trans. The efficiency of this second step varies depending on the nature of the siRNA-producing locus, and at some loci, it may require preexisting chromatin modifications such as DNA methylation itself. Enhancement of RNA-directed DNA methylation by pre-existing DNA methylation could create a self-reinforcing system to enhance the stability of silencing. Tandem repeats throughout the Arabidopsis genome produce siRNAs, suggesting that repeat acquisition may be a general mechanism for the evolution of gene silencing.", 
"Expression of imprinted genes is restricted to a single parental allele as a result of epigenetic regulation - DNA methylation and histone modifications. Igf2/H19 is a reciprocally imprinted locus exhibiting paternal Igf2 and maternal H19 expression. Their expression is regulated by a paternally methylated imprinting control region ( ICR) located between the two genes. Although the de novo DNA methyltransferases have been shown to be necessary for the establishment of ICR methylation, the mechanism by which they are targeted to the region remains unknown. We demonstrate that CTCFL/BORIS, a paralog of CTCF, is an ICR-binding protein expressed during embryonic male germ cell development, coinciding with the timing of ICR methylation. PRMT7, a protein arginine methyltransferase with which CTCFL interacts, is also expressed during embryonic testis development. Symmetrical dimethyl arginine 3 of histone H4, a modification catalyzed by PRMT7, accumulates in germ cells during this developmental period. This modified histone is also found enriched in both H19 ICR and Gtl2 differentially methylated region ( DMR) chromatin of testis by chromatin immunoprecipitation ( ChIP) analysis. In vitro studies demonstrate that CTCFL stimulates the histone-methyltransferase activity of PRMT7 via interactions with both histones and PRMT7. Finally, H19 ICR methylation is demonstrated by nuclear co-injection of expression vectors encoding CTCFL, PRMT7, and the de novo DNA methyltransferases, Dnmt3a, -b and -L, in Xenopus oocytes. These results suggest that CTCFL and PRMT7 may play a role in male germline imprinted gene methylation.", 
"Plant cells grown in culture exhibit genetic and epigenetic instability. Using a combination of chromatin immunoprecipitation and DNA methylation profiling on tiling microarrays, we have mapped the location and abundance of histone and DNA modifications in a continuously proliferating, dedifferentiated cell suspension culture of Arabidopsis. We have found that euchromatin becomes hypermethylated in culture and that a small percentage of the hypermethylated genes become associated with heterochromatic marks. In contrast, the heterochromatin undergoes dramatic and very precise DNA hypomethylation with transcriptional activation of specific transposable elements (TEs) in culture. High throughput sequencing of small interfering RNA (siRNA) revealed that TEs activated in culture have increased levels of 21-nucleotide (nt) siRNA, sometimes at the expense of the 24-nt siRNA class. In contrast, TEs that remain silent, which match the predominant 24-nt siRNA class, do not change significantly in their siRNA profiles. These results implicate RNA interference and chromatin modification in epigenetic restructuring of the genome following the activation of TEs in immortalized cell culture.", 
"The effects of nonmutagenic environmental exposures can sometimes be transmitted for several generations, suggesting transgenerational inheritance of induced epigenetic variation. Methyl donor supplementation of female mice during pregnancy induces CpG hypermethylation at the agouti viable yellow ( A(vy)) allele in A(vy)/a offspring. Epigenetic inheritance occurs at A(vy); when passed through the female germ line, A(vy) epigenotype is not completely reset. We therefore tested whether diet-induced epigenetic alterations at A(vy) are inherited transgenerationally. Female A(vy)/a mice were weaned onto either control ( n = 6) or a methyl-supplemented diet ( n = 5). These F0 dams were mated with a/a males. All F1 and F2 A(vy)/a females were weaned onto the same diet as their mothers, then mated with a/a males. F1, F2, and F3 A(vy)/a offspring were classified for coat color, an indicator of A(vy) methylation. In total, 62 F1, 98 F2, and 209 F3 A(vy)/a mice were studied. As expected, average A(vy)/a coat color was darker in the supplemented group ( P< 0.01). However, there was no cumulative effect of supplementation across successive generations. These results suggest that, in the female germ line, diet-induced A(vy) hypermethylation occurs in the absence of additional epigenetic modifications that normally confer transgenerational epigenetic inheritance at the locus. - Waterland, R. A., Travisano, M., Tahiliani, K. G. Diet-induced hypermethylation at agouti viable yellow is not inherited transgenerationally through the female.", 
"The enzyme-catalyzed acetylation of the N-terminal tail domains of core histones provides a rich potential source of epigenetic information. This may be used both to mediate transient changes in transcription, through modification of promoter-proximal nucleosomes, and for the longer-term maintenance and modulation of patterns of gene expression. The latter may be achieved by setting specific patterns of histone acetylation, perhaps involving acetylation of particular lysine residues, across relatively large chromatin domains. The histone acetylating and deacetylating enzymes (HATs and HDACs, respectively) can be targeted to specific regions of the genome and show varying degrees of substrate specificity, properties that are consistent with a role in maintaining a dynamic, acetylation-based epigenetic code. The code may be read (ie. exert a functional effect) either through non-histone proteins that bind in an acetylation-dependent manner, or through direct effects on chromatin structure. Recent evidence raises the interesting possibility that an acetylation-based code may operate through both mitosis and meiosis, providing a possible mechanism for germ-line transmission of epigenetic changes. BioEssays 22:836-845, 2000. (C) 2000 John Wiley & Sons, Inc.", 
"DNA methylation is essential for the control of a number of biological mechanisms in mammals [1]. Mammalian development is accompanied by two major waves of genome-wide demethylation and remethylation: one during germ-cell development and the other after fertilisation [2-7]. Most previous studies have suggested that the genome-wide demethylation observed after fertilisation occurs passively, that is, by the lack of maintenance methylation following DNA replication and cell division [6,7], although one other study has reported that replication-independent demethylation may also occur during early embryogenesis [8]. Here, we report that genes that are highly methylated in sperm are rapidly demethylated in the zygote only hours after fertilisation, before the first round of DNA replication commences. By contrast, the oocyte-derived maternal alleles are unaffected by this reprogramming. They either remain methylated after fertilisation or become further methylated de novo. These results provide the first direct evidence for active demethylation of single-copy genes in the mammalian zygote and, moreover, reveal a striking asymmetry in epigenetic methylation reprogramming. Whereas paternally (sperm)-derived sequences are exposed to putative active demethylases in the oocyte cytoplasm, maternally (oocyte)-derived sequences are protected from this reaction. These results, whose generality is supported by findings of Mayer et al. [9], have important implications for the establishment of biparental genetic totipotency after fertilisation, the establishment and maintenance of genomic imprinting, and the reprogramming of somatic cells during cloning.", 
"Proper DNA methylation patterning requires the complementary processes of de novo methylation (the initial methylation of unmethylated DNA sequences) and maintenance methylation (the faithful replication of preexisting methylation). Arabidopsis has two types of methyltransferases with demonstrated maintenance activity: MET1, which maintains CpG methylation [1-3] and is homologous to mammalian DNMT1, and CHROMOMETHYLASE 3 (CMT3), which maintains CpNpG (N = A, T, C, or G) methylation [3,4] and is unique to the plant kingdom. Here we describe loss-of-function mutations in the Arabidopsis DOMAINS REARRANGED METHYLASE (DRM) genes [5] and provide evidence that they encode de novo methyltransferases. drm1 drm2 double mutants retained preexisting CpG methylation at the endogenous FWA locus but blocked de novo CpG methylation that is normally associated with FWA transgene silencing. Furthermore, drm1 drm2 double mutants blocked de novo CpNpG and asymmetric methylation and gene silencing of the endogenous SUPERMAN (SUP) gene, which is normally triggered by an inverted SUP repeat. However, drm1 drm2 double mutants did not show reactivation of previously established SUPERMAN epigenetic silenced alleles. Thus, drm mutants prevent the establishment but not the maintenance of gene silencing at FWA and SUP, suggesting that the DRMs encode the major de novo methylation enzymes affecting these genes.", 
"The conserved histone variant H2A.Z functions in euchromatin to antagonize the spread of heterochromatin. The mechanism by which histone H2A is replaced by H2A.Z in the nucleosome is unknown. We identified a complex containing 13 different polypeptides associated with a soluble pool of H2A.Z in Saccharomyces cerevisiae. This complex was designated SWR1-Com in reference to the Swr1p subunit, a Swi2/Snf2-paralog. Swr1p and six other subunits were found only in SWR1-Com, whereas six other subunits were also found in the NuA4 histone acetyltransferase and/or the Ino80 chromatin remodeling complex. H2A.Z and SWR1 were essential for viability of cells lacking the EAF1 component of NuA4, pointing to a close functional connection between these two complexes. Strikingly, chromatin immunoprecipitation analysis of cells lacking Swr1p, the presumed ATPase of the complex, revealed a profound defect in the deposition of H2A.Z at euchromatic regions that flank the silent mating type cassette HMR and at 12 other chromosomal sites tested. Consistent with a specialized role for Swr1p in H2A.Z deposition, the majority of the genome-wide transcriptional defects seen in swr1Delta cells were also found in htz1Delta cells. These studies revealed a novel role for a member of the ATP-dependent chromatin remodeling enzyme family in determining the region-specific histone subunit composition of chromatin in vivo and controlling the epigenetic state of chromatin. Metazoan orthologs of Swr1p (Drosophila Domino; human SRCAP and p400) may have analogous functions.", 
"During differentiation, somatic nuclei acquire highly specialized DNA and chromatin modifications, which are thought to result in cellular memory of the differentiated state [1]. Upon somatic nuclear transfer into oocytes, the donor nucleus may have to undergo reprogramming of these epigenetic marks in order to achieve totipotency. This may involve changes in epigenetic features similar to those that occur in normal embryos during early development [2-6]. However, there is accumulating evidence that epigenetic reprogramming is severely deficient in cloned embryos [712]. Several reports reveal inefficient demethylation and inappropriate reestablishment of DNA methylation in quantitative and qualitative patterns on somatic nuclear transfer [7-12]. Here we examine histone H3 lysine 9 (H3-K9) methylation and acetylation in normal embryos and in those created by somatic nuclear transfer. We find that H3-K9 methylation is reprogrammed in parallel with DNA methylation in normal embryos. However, the majority of cloned embryos exhibit H3-K9 hypermethylation associated with DNA hypermethylation, suggesting a genome-wide failure of reprogramming. Strikingly, the precise epigenotype in cloned embryos depends on the donor cell type, and the proportion of embryos with normal epigenotypes correlates closely with the proportion developing to the blastocyst stage. These results suggest a mechanistic link between DNA and histone methylation in the mammalian embryo and reveal an association between epigenetic marks and developmental potential of cloned embryos.", 
"The central dogma of biology holds that genetic information normally flows from DNA to RNA to protein. As a consequence it has been generally assumed that genes generally code for proteins, and that proteins fulfil not only most structural and catalytic but also most regulatory functions, in all cells, from microbes to mammals. However, the latter may not be the case in complex organisms. A number of startling observations about the extent of non-protein-coding RNA (ncRNA) transcription in the higher eukaryotes and the range of genetic and epigenetic phenomena that are RNA-directed suggests that the traditional view of the structure of genetic regulatory systems in animals and plants may be incorrect. ncRNA dominates the genomic output of the higher organisms and has been shown to control chromosome architecture, mRNA turnover and the developmental timing of protein expression, and may also regulate transcription and alternative splicing. This paper re-examines the available evidence and suggests a new framework for considering and understanding the genomic programming of biological complexity, autopoletic development and phenotypic variation. BioEssays 25:930-939,2003. (C) 2003 Wiley Periodicals, Inc.", 
"Background: The heterochromatin of many eukaryotes is marked by both DNA methylation and histone H3 lysine 9 (H3-K9) methylation, though the exact relationship between these epigenetic modifications is unknown. In Neurospora, H3-K9 methylation is required for the maintenance of all known DNA methylation. In Arabidopsis, H3-K9 methylation directs some of the CpNpG and asymmetric methylation. However, it is not known in any organism whether DNA methylation may also direct histone H3 methylation. Results: Using chromatin immunoprecipitation (ChIP) assays, we show that Arabidopsis heterochromatin is associated with H3-K9 methylation. This histone methylation is dependent on the KRYPTONITE and DDM1 genes (SU[VAR]3-9 and SWI2/SNF2 homologs, respectively). We also find that a decrease in DNA methylation does not directly cause a loss of H3-K9 methylation. Instead, a decrease in H3-K9 methylation is only seen at loci where transcription is derepressed. Conclusions: We conclude that DNA methylation does not control the methylation of histone H3-K9. We propose that loss of H3-K9 methylation is due to transcriptional reactivation, coupled with deposition of unmethylated nucleosomes. These findings are consistent with recent observations of DNA replication-independent deposition of histone H3.3 in Drosophila. Our results also suggest that, in Arabidopsis, DNA methylation is sufficient for gene silencing, but H3-K9 methylation is not.", 
"Heritable, but reversible, changes in transposable element activity were first observed in maize by Barbara McClintock in the 1950s. More recently, transposon silencing has been associated with DNA methylation, histone H3 lysine-9 methylation (H3mK9), and RNA interference (RNAi). Using a genetic approach, we have investigated the role of these modifications in the epigenetic regulation and inheritance of six Arabidopsis transposons. Silencing of most of the transposons is relieved in DNA methyltransferase (met1), chromatin remodeling ATPase (ddm1), and histone modification (sil1) mutants. In contrast, only a small subset of the transposons require the H3mK9 methyltransferase KRYPTONITE, the RNAi gene ARGONAUTE1, and the CXG methyltransferase CHROMOMETHYLASE3. In crosses to wildtype plants, epigenetic inheritance of active transposons varied from mutant to mutant, indicating these genes differ in their ability to silence transposons. According to their pattern of transposon regulation, the mutants can be divided into two groups, which suggests that there are distinct, but interacting, complexes or pathways involved in transposon silencing. Furthermore, different transposons tend to be susceptible to different forms of epigenetic regulation.", 
"Background: The Ras-GTPase controls cell fate decisions through the binding of an array of effector molecules, such as Raf and PI 3-kinase, in a GTP-dependent manner. NORE1, a noncatalytic polypeptide, binds specifically to Ras-GTP and to several other Ras-like GTPases. NORE is homologous to the putative tumor suppressor RASSF1 and to the Caenorhabditis elegans polypeptide T24F1.3. Results: We find that all three NORE-related polypeptides bind selectively to the proapoptotic protein kinase MST1, a member of the Group 11 GC kinases. Endogenous NORE and MST1 occur in a constitutive complex in vivo that associates with endogenous Ras after serum stimulation. Targeting recombinant MST1 to the membrane, either through NORE or myristoylation, augments the apoptotic efficacy of MST1. Overexpression of constitutively active Ki-RasG12V promotes apoptosis in a variety of cell lines; Ha-RasG12V is a much less potent proapoptotic agent; however, a Ha-RasG12V effector loop mutant (E37G) that binds NORE, but not Raf or PI 3-kinase, exhibits proapoptotic efficacy approaching that of Ki-RasG12V. The apoptotic action of both Ki-RasG12V and Ha-RasG12V, E37G is suppressed by overexpression of the MST1 carboxy-terminal noncatalytic segment or by the NORE segment that binds MST1. Conclusions: MST1 is a phylogenetically conserved partner of the NORE/RASSF polypeptide family, and the NORE-MST1 complex is a novel Ras effector unit that mediates the apoptotic effect of Ki-RasG12V.", 
"Full-term development has now been achieved in several mammalian species by transfer of somatic nuclei into enucleated oocytes [1, 2]. Although a high proportion of such reconstructed embryos can evolve until the blastocyst stage, only a few percent develop into live offspring, which often exhibit developmental abnormalities [3, 4]. Regulatory epigenetic markers such as DNA methylation are imposed on embryonic cells as normal development proceeds, creating differentiated cell states. Cloned embryos require the erasure of their somatic epigenetic markers so as to regain a totipotent state [5]. Here we report on differences in the dynamics of chromosome methylation between cloned and normal bovine embryos before implantation. We show that cloned embryos fail to reproduce distinguishable parental-chromosome methylation patterns after fusion and maintain their somatic pattern during subsequent stages, mainly by a highly reduced efficiency of the passive demethylation process. Surprisingly, chromosomes appear constantly undermethylated on euchromatin in morulae and blastocysts, while centromeric heterochromatin remains more methylated than that of normal embryos. We propose that the abnormal time-dependent methylation events spanning the preimplantation development of clones may significantly interfere with the epigenetic reprogramming, contributing to the high incidence of physiological anomalies occurring later during pregnancy or after clone birth.", 
"Background: The association between DNA methylation and gene silencing has long been recognized; however, signals that initiate de novo methylation are largely unknown. In plants, recognition of RNAs that are inducers of posttranscriptional gene silencing (PTGS) can result in sequence-specific DNA methylation, and the aim of this work was to investigate whether heritable epigenetic changes can occur by this mechanism and if the Met1 methyltransferase is required. Results: RNA-directed DNA methylation (RdDM) was initiated in 35S-GFP transgenic plants following infection with plant RNA viruses modified to carry portions of either the 35S promoter or the GFP coding region. Targeting of the promoter sequence resulted in both methylation and transcriptional gene silencing (TGS) that was inherited independently of the RNA trigger. Targeting the coding region also resulted in methylation; however, this was not inherited, Expression of Met1 was suppressed in order to investigate its role in initiation and maintenance of RdDM. Initiation of RdDM was found to be Met1-independent, whereas maintenance of methylation and TGS in the subsequent generations in the absence of the RNA trigger was Met1-dependent. Maintenance of methylation associated with systemic PTGS was also found to be Met1-independent. Conclusions: RNA-triggered events can lead to heritable changes in gene expression, and it is possible that initiation of other epigenetic phenomena such as trans-silencing and paramutation may have an RNA component.", 
"In plants, the mechanism by which RNA can induce de novo cytosine methylation of homologous DNA is poorly understood. Cytosines in all sequence contexts become modified in response to RNA signals [1, 2]. Recent work has implicated the de novo DNA methyltransferases (DMTases), DRM1 and DRM2, in establishing RNA-directed methylation of the constitutive nopaline synthase promoter [3], as well as the DMTase MET1 [4] and the putative histone deacetylase HDA6 [5] in maintaining or enhancing CpG methylation induced by RNA. Despite the identification of enzymes that catalyze epigenetic modifications in response to RNA signals, it is unclear how RNA targets DNA for methylation. A screen for mutants defective in RNA-directed DNA methylation identified a novel putative chromatin-remodeling protein, DRD1. This protein belongs to a previously undefined, plant-specific subfamily of SW12/SNF2-like proteins most similar to the RAD54/ATRX subfamily. In drd1 mutants, RNA-induced non-CpG methylation is almost eliminated at a target promoter, resulting in reactivation, whereas methylation of centromeric and rDNA repeats is unaffected ' Thus, unlike the SNF2-like proteins DDM1/Lsh1 [6, 7] and ATRX [8, 9], which regulate methylation of repetitive sequences, DRD1 is not a global regulator of cytosine methylation. DRD1 is the first SNF2-like protein implicated in an RNA-guided, epigenetic modification of the genome.", 
"Methylation of cytosine residues in eukaryotic genomes is often associated with repeated sequences including transposons and their derivatives [1, 2]. Methylation has been implicated in control of two potential deleterious effects of these repeats: (1) uncontrolled transcription [2-4], which often disturbs proper expression of nearby host genes [5, 6], and (2) changes in genome structure by transposition and ectopic recombination [2, 7]. Arabidopsis thaliana provides a genetically tractable system to examine these possibilities, since viable mutants in DNA methyltransferases are available. Arabidopsis MET1 (METHYLTRANSFERASE 1, ortholog of mammalian DNA methyltransferase Dnmt1) is necessary for maintaining genomic cytosine methylation at 5'-CG-3' sites [8, 9]. Arabidopsis additionally methylates non-CG sites using CHROMO-METHYLASE3 (CMT3) [10, 11]. We examined the mobility of endogenous CACTA transposons in met1, cmt3, and cmt3-met1 mutants. High-frequency transposition of CACTA elements was detected in cmt3-met1 double mutants. Single mutants in either met1 or cmt3 were much less effective in mobilization, despite significant induction of CACTA transcript accumulation. These results lead us to conclude that CG and non-CG methylation systems redundantly function for immobilization of transposons. Non-CG methylation in plants may have evolved as an additional epigenetic tag dedicated to transposon control. This view is consistent with the recent finding that CMT3 preferentially methylates transposon-related sequences [12].", 
"Background: The genes of the trithorax (trxG) and Polycomb groups (PcG) are best known for their regulatory functions in Drosophila, where they control homeotic gene expression. Plants and animals are thought to have evolved multicellularity independently. Although homeotic genes control organ identity in both animals and plants, they are unrelated. Despite this fact, several plant homeotic genes are negatively regulated by plant genes similar to the repressors from the animal PcG. However, plant-activating regulators of the trxG have not been characterized. Results: We provide genetic, molecular, functional, and biochemical evidence that an Arabidopsis gene, ATK1, which is similar to the Drosophila trx, regulates floral organ development. The effects are specific: structurally and functionally related flower homeotic genes are under different control. We show that ATX1 is an epigenetic regulator with histone H3K4 methyltransferase activity. This is the first example of this kind of enzyme activity reported in plants, and, in contrast to the Drosophila and the yeast trithorax homologs, ATX1 can methylate in the absence of additional proteins. In its ability to methylate H3K4 as a recombinant protein, ATX1 is similar to the human homolog. Conclusions: ATX1 functions as an activator of homeotic genes, like Trithorax in animal systems. The histone methylating activity of the ATX1-SET domain argues that the molecular basis of these effects is the ability of ATX1 to modify chromatin structure. Our results suggest a conservation of trxG function between the animal and plant kingdoms despite the different structural nature of their targets.", 
"Background: Arabidopsis mutations causing genome-wide hypomethylation are viable but display a number of specific developmental abnormalities, including some that resemble known floral homeotic mutations. We previously showed that one of the developmental abnormalities present in an antisense-METHYLTRANSFERASEI (METI) transgenic line resulted from ectopic hypermethylation of the SUPERMAN gene. Results: Here, we investigate the extent to which hypermethylation of SUPERMAN occurs in several hypomethylation mutants, and describe methylation effects at a second gene, AGAMOUS. SUPERMAN gene hypermethylation occurred at a high frequency in several mutants that cause overall decreases in genomic DNA methylation. The hypermethylation pattern was largely similar in the different mutant backgrounds. Genetic analysis suggests that hypermethylation most likely arose either during meiosis or somatically in small sectors of the plant. A second floral development gene, AGAMOUS, also became hypermethylated and silenced in an Arabidopsis antisense-METI line. Conclusions: These results suggest that ectopic hypermethylation of specific genes in mutant backgrounds that show overall decreases in methylation may be a widespread phenomenon that could explain many of the developmental defects seen in Arabidopsis methylation mutants. This resembles a phenomenon seen in cancer cells, which can simultaneously show genome-wide hypomethylation and hypermethylation of specific genes. Comparison of the methylated sequences in SUPERMAN and AGAMOUS suggests that hypermethylation could involve DNA secondary structures formed by pyrimidine-rich sequences.", 
"Covalent modifications of chromatin have emerged as key determinants of the genome's transcriptional competence [1-3]. Histone H3 lysine 9(H3K9) methylation is an epigenetic signal that is recognized by HP1 [4, 5] and correlates with gene silencing in a variety of organisms [3]. Discovery of the enzymes that catalyze H3K9 methylation [6-8] has identified a second gene-specific function for this modification in transcriptional repression [9-11]. Whether H3K9 methylation is causative in the initiation and establishment of gene repression or is a byproduct of the process leading to the repressed state remains unknown. To investigate the role of HMTs and specifically H3K9 methylation in gene repression, we have employed engineered zinc-finger transcription factors (ZFPs) to target HMT activity to a specific endogenous gene. By utilizing ZFPs that recognize the promoter of the endogenous VEGF-A gene [12], and thus employing this chromosomal locus as an in vivo reporter, we show that ZFPs linked to a minimal catalytic HMT domain affect local methylation of histone H3K9 and the consequent repression of target gene expression. Furthermore, amino acid substitutions within the HMT that ablate its catalytic activity (6] effectively eliminate the ability of the UP fusions to repress transcription. Thus, H3K9 methylation is a primary signal that is sufficient for initiating a gene repression pathway in vivo.", 
"Function of the heart begins long before its formation is complete. Analyses in mouse and zebrafish have shown that myocardial function is not required for early steps of organogenesis, such as formation of the heart tube or chamber specification. However, whether myocardial function is required for later steps of cardiac development, such as endocardial cushion (EC) formation, has not been established. Recent technical advances and approaches have provided novel inroads toward the study of organogenesis, allowing us to examine the effects of both genetic and pharmacological perturbations of myocardial function on EC formation in zebrafish. To address whether myocardial function is required for EC formation, we examined silent heart (sih(-/-)) embryos, which lack a heartbeat due to mutation of cardiac troponin T (tnnt2), and observed that atrioventricular (AV) ECs do not form. Likewise, we determined that cushion formation is blocked in cardiofunk (cfk(-/-)) embryos, which exhibit cardiac dilation and no early blood flow. In order to further analyze the heart defects in cfk(-/-) embryos, we positionally cloned cfk and show that it encodes a novel sarcomeric actin expressed in the embryonic myocardium. The Cfk(s11) variant exhibits a change in a universally conserved residue (R177H). We show that in yeast this mutation negatively affects actin polymerization. Because the lack of cushion formation in sih- and cfk-mutant embryos could be due to reduced myocardial function and/or lack of blood flow, we approached this question pharmacologically and provide evidence that reduction in myocardial function is primarily responsible for the defect in cushion development. Our data demonstrate that early myocardial function is required for later steps of organogenesis and suggest that myocardial function, not endothelial shear stress, is the major epigenetic factor controlling late heart development. Based on these observations, we postulate that defects in cardiac morphogenesis may be secondary to mutations affecting early myocardial function, and that, in humans, mutations affecting embryonic myocardial function may be responsible for structural congenital heart disease.", 
"Many proteins can misfold into beta-sheet-rich, self-seeding polymers (amyloids). Prions are exceptional among such aggregates in that they are also infectious. In fungi, prions are not pathogenic but rather act as epigenetic regulators of cell physiology, providing a powerful model for studying the mechanism of prion replication. We used prion-forming domains from two budding yeast proteins (Sup35p and New1p) to examine the requirements for prion formation and inheritance. In both proteins, a glutamine/asparagine-rich (Q/N-rich) tract mediates sequence-specific aggregation, while an adjacent motif, the oligopeptide repeat, is required for the replication and stable inheritance of these aggregates. Our findings help to explain why although Q/N-rich proteins are relatively common, few form heritable aggregates: prion inheritance requires both an aggregation sequence responsible for self-seeded growth and an element that permits chaperone-dependent replication of the aggregate. Using this knowledge, we have designed novel artificial prions by fusing the replication element of Sup35p to aggregation-prone sequences from other proteins, including pathogenically expanded polyglutamine.", 
"In contrast to the biallelic expression of most genes, expression of genes subject to genomic imprinting is monoallelic and based on the sex of the transmitting parent. Possession of only a single active allele can lead to deleterious health consequences in humans. Aberrant expression of imprinted genes, through either genetic or epigenetic alterations, can result in,developmental failures, neurodevelopmental and neurobehavioral disorders and cancer. The evolutionary emergence of imprinting occurred in a common ancestor to viviparous mammals after divergence from the egg-laying monotremes. Current evidence indicates that imprinting regulation in metatherian mammals differs from that in eutherian mammals. This suggests that imprinting mechanisms are evolving from those that were established 150 million years ago. Therefore, comparing genomic sequence of imprinted domains from marsupials and eutherians with those of orthologous regions in monotremes offers a potentially powerful bioinformatics approach for identifying novel imprinted genes and their regulatory elements. Such comparative studies will also further our understanding of the molecular evolution and phylogenetic distribution of imprinted genes. BioEssays25:577-588,2003. (C) 2003 Wiley Periodicals, Inc.", 
"In female mammals, a single X chromosome is stably and heritably silenced early in embryogenesis [1]. The inactive X is characterized by asynchronous DNA replication [2, 3] and epigenetic chromatin modifications, including DNA methylation [4], histone H3/H4 hypoacetylation [5-7], and incorporation of a variant histone macroH2A [8-10]. X inactivation is initiated by a cis-acting RNA molecule, the X-inactive specific transcript (Xist), which coats the chromosome (11-17]. However, the mechanism by which Xist induces chromosome silencing is poorly understood. An important approach towards answering this question has been to determine the temporal order of epigenetic chromatin modifications in an in vitro model system, differentiating XX embryonic stem (ES) cells, and thereby to identify candidate targets for Xist RNA [18-21]. To date, these studies have demonstrated that, following accumulation of Xist RNA [18, 20], the transition to late replication of the X chromosome is the earliest detectable event [19]. H4 hypoacetylation [19], macroH2A1.2 incorporation [21], and DNA methylation [19] all occur subsequently. Recently, it has been shown that chromatin of the inactive X is also characterized by methylation of histone H3 at lysine 9 (H3-K9) [22, 23]. Here we show that H3-K9 methylation is a very early event in the process of X inactivation, which closely parallels the onset of Xist RNA accumulation.", 
"The information contained within the linear sequence of bases (the genome) must be faithfully replicated in each cell cycle, with a balance of constancy and variation taking place over the course of evolution. Recently, it has become clear that additional information important for genetic regulation is contained within the chromatin proteins associated with DNA (the epigenome). Epigenetic information also must be faithfully duplicated in each cell cycle, with a balance of constancy and variation taking place during the course of development to achieve differentiation while maintaining identity within cell lineages. Both the genome and the epigenome are synthesized at the replication fork, so the events occurring during S-phase provide a critical window of opportunity for eliciting change or maintaining existing epigenetic states. Cells discriminate between different states of chromatin through the activities of proteins that selectively modify the structure of chromatin. Several recent studies report the localization of certain chromatin-modifying proteins to replication forks at specific times during S-phase. Since transcriptionally active and inactive chromosome domains generally replicate at different times during S-phase, this spatiotemporal regulation of chromatin assembly proteins may be an integral part of epigenetic inheritance. (C) 2003 Wiley Periodicals, Inc.", 
"DNA methylation has been implicated in a variety of epigenetic processes, and abnormal methylation patterns have been seen in tumors [1-3]. Analysis of methylation patterns has traditionally been conducted either by using Southern analysis after cleavage with methyl-sensitive restriction endonucleases or by bisulfite sequencing [4]. However, neither method is practical for analyzing more than a few genes. Here, we describe a simple technique for genome-wide mapping of DNA methylation patterns. Fragmentation by a methyl-sensitive restriction endonuclease is followed by size fractionation and hybridization to microarrays. We demonstrate the utility of this method by characterizing methylation patterns in Arabidopsis methylation mutants. This analysis reveals that CHROMOMETHYLASE3 (CMT3) [5], which was previously shown to maintain CpXpG methylation [6,7], preferentially methylates transposons, even when they are present as single copies within the genome. Methylation profiling has potential applications in disease research and diagnostic screening.", 
"Nutritional regulatory responses were compared for the cryptic 'solitarious' and the conspicuously coloured, aggregating 'gregarious' phases of the desert locust Schistocerca gregaria. The desert locust has the genetic potential to exist in either phase, changing between them within a lifetime and epigenetically across generations. Our aim was to compare final-instar nymphs of the two phases with respect to key nutritional variables, including (i) points of regulated intake (the 'intake target') for protein and carbohydrate, (ii) the nature of trade-offs between over-eating nutrients in excess and under-eating those in deficit when fed nutritionally unbalanced foods, (iii) diet-related patterns of nutrient utilisation, and (iv) the performance consequences of eating nutritionally unbalanced diets. When provided with pairs of nutritionally unbalanced but complementary foods, both phases regulated their intake of protein and carbohydrate to a similar point. However, when confined to foods that were of unbalanced protein to carbohydrate ratio, gregarious nymphs ate more than solitarious insects. Both phases regulated protein growth, but gregarious insects did so to a lower adult body protein content and converted ingested protein to growth less efficiently. When fed a food high in carbohydrate and low in protein, gregarious nymphs deposited more body lipid and survived less well than did solitarious insects. Solitarious nymphs developed more quickly than gregarious nymphs except on the two most extremely unbalanced diets, on which development time was similar. The results are discussed with respect to the different nutritional ecologies of the two phases and used to develop the hypothesis that animals have evolved to trade-off the cost of eating excess of a nutritionally unbalanced diet against the probability of encountering foods of complementary composition in the future.", 
"Whether exposure to radiation emitted from cellular phones poses a health hazard is at the focus of current debate. We have examined whether in vitro exposure of human peripheral blood lymphocytes (PBL) to continuous 830 MHz electromagnetic fields causes losses and gains of chromosomes (aneuploidy), a major somatic mutation leading to genomic instability and thereby to cancer. PBL were irradiated at different average absorption rates (SAR) in the range of 1.6-8.8 W/kg for 72 hr in an exposure system based on a parallel plate resonator at temperatures ranging from 34.5-37.5 degreesC. The averaged SAR and its distribution in the exposed tissue culture flask were determined by combining measurements and numerical analysis based on a finite element simulation code. A linear increase in chromosome 17 aneuploidy was observed as a function of the SAR value, demonstrating that this radiation has a genotoxic effect. The SAR dependent aneuploidy was accompanied by an abnormal mode of replication of the chromosome 17 region engaged in segregation (repetitive DNA arrays associated with the centromere), suggesting that epigenetic alterations are involved in the SAR dependent genetic toxicity. Control experiments (i.e., without any RF radiation) carried out in the temperature range of 34.5-38.5 degreesC showed that elevated temperature is not associated with either the genetic or epigenetic alterations observed following RF radiation-the increased levels of aneuploidy and the modification in replication of the centromeric DNA arrays. These findings indicate that the genotoxic effect of the electromagnetic radiation is elicited via a non-thermal pathway. Moreover, the fact that aneuploidy is a phenomenon known to increase the risk for cancer, should be taken into consideration in future evaluation of exposure guidelines. (C) 2003 Wiley-Liss. Inc.", 
"One of the enigmas in tumor biology is that different types of cancers are prevalent in different age groups. One possible explanation is that the ability of a specific oncogene to cause tumorigenesis in a particular cell type depends on epigenetic parameters such as the developmental context. To address this hypothesis, we have used the tetracycline regulatory system to generate transgenic mice in which the expression of a c-MYC human transgene can be conditionally regulated in murine hepatocytes. MYC's ability to induce tumorigenesis was dependent upon developmental context. In embryonic and neonatal mice, MYC overexpression in the liver induced marked cell proliferation and immediate onset of neoplasia. In contrast, in adult mice MYC overexpression induced cell growth and DNA replication without mitotic cell division, and mice succumbed to neoplasia only after a prolonged latency. In adult hepatocytes, MYC activation failed to induce cell division, which was at least in part mediated through the activation of p53. Surprisingly, apoptosis is not a barrier to MYC inducing tumorigenesis. The ability of oncogenes to induce tumorigenesis may be generally restrained by developmentally specific mechanisms. Adult somatic cells have evolved mechanisms to prevent individual oncogenes from initiating cellular growth, DNA replication, and mitotic cellular division alone, thereby preventing any single genetic event from inducing tumorigenesis.", 
"We investigated the. randomness and uniqueness of human iris patterns by mathematically comparing 2.3 million different pairs of eve images. The phase structure of each iris pattern was extracted by demodulation with quadrature wavelets spanning several scales of analysis. The resulting distribution of phase sequence variation anion different eyes was precisely binomial, revealing 244 independent degrees of freedom. This amount of statistical variability corresponds to an entropy (information density) of about 3.2 bits mm(-2) over the iris. It implies that the probability of two different irides agreeing by chance in more than 70% of their phase sequence is about one in 7 billion. We also compared images of genetically identical irides, from the left and right eyes of 324 persons, and from monozygotic twins. Their relative phase sequence variation generated the same statistical distribution as did unrelated eyes. This indicates that apart from overall form and colour. iris patterns are determined epigenetically by random events in the morphogenesis of this tissue. The resulting diversity, and the combinatorial complexity created by so many dimensions of random variation, mean that the failure of a simple test of statistical independence performed on iris patterns can serve as a reliable rapid basis for automatic personal identification.", 
"The C. elegans proteins MES-2 and MES-6, orthologs of the Polycomb group (PcG) chromatin repressors E(Z) and ESC [1, 2], exist in a complex with their novel partner MES-3 [3]. The MES system participates in silencing the X chromosomes in the hermaphrodite germline [4, 5]. Loss of maternal MES function leads to germline degeneration and sterility [6]. We report here that the MES complex is responsible for di- and trimethylation of histone H3 Lys27 (H3-K27) in the adult germline and in early embryos and that MES-dependent H3-K27 marks are concentrated on the X's. Another H3-K27 HMT functions in adult somatic cells oocytes, and the PGCs of embryos. In PGCs, the ME; complex may specifically convert dimethyl to trimethyl H3-K27. The HMT activity of the MES complex appears to be dependent on the SET domain of MES-2. MES-2 thus joins its orthologs Drosophila E(Z) and human EZH2 among SET domain proteins known to function as HMTs (reviewed in [7]). Methylation of histones is important for long-term epigenetic regulation of chromatin and plays a key role in diverse processes such as X inactivation and oncogenesis [8]. Our results contribute to understanding the composition and roles of E(Z)/MES-2 complexes across species.", 
"Remodeling of microvascular networks in mammals is critical for physiological adaptations and therapeutic revascularization. Cellular behaviors such as proliferation, differentiation, and migration are coordinated in these remodeling events via combinations of biochemical and biomechanical signals. We developed a cellular automata (CA) computational simulation that integrates epigenetic stimuli, molecular signals, and cellular behaviors to predict microvascular network patterning events. Over 50 rules obtained from published experimental data govern independent behaviors (including proliferation, differentiation, and migration) of thousands of interacting cells and diffusible growth factors in their tissue environment. From initial network patterns of in vivo blood vessel networks, the model predicts emergent patterning responses to two stimuli: 1) network-wide changes in hemodynamic mechanical stresses, and 2) exogenous focal delivery of an angiogenic growth factor. The CA model predicts comparable increases in vascular density (370 +/- 29 mm/mm(3)) 14 days after treatment with exogenous growth factor to that in vivo (480 +/- 41 mm/mm(3)) and approximately a twofold increase in contractile vessel lengths 5-10 days after 10% increase in circumferential wall strain, consistent with in vivo results. The CA simulation was thus able to identify a functional patterning module capable of quantitatively predicting vessel network remodeling in response to two important epigenetic stimuli.", 
" ", 
"Allopolyploidy, the joining of two parental genomes in a polyploid organism with diploid meiosis, is an important mechanism of reticulate evolution. While many successful long-established allopolyploids are known, those formed recently undergo an instability phase whose basis is now being characterized. We describe observations made with the Arabidopsis system that include phenotypic instability, gene silencing and activation, and methylation changes. We present a model based on the epigenetic destabilization of genomic repeats, which in the parents are heterochromatinized and suppressed. We hypothesize that loss of epigenetic suppression of these sequences, here defined as the heterome, results in genomic instability including silencing of single-copy genes.", 
" ", 
"Reproductive isolation is considered to play a key part in evolution, and plants and animals have developed a range of strategies that minimize gene flow between species. In plants, these strategies involve either pre-zygotic barriers, such as differences in floral structure and pollen-stigma recognition, or post-zygotic barriers, which are less well understood and affect aspects of seed development ranging from fertilization to maturation. In most angiosperms, a double fertilization event gives rise to a zygote and the endosperm: a triploid tissue with an unequal parental genomic contribution, which, like the placenta of mammals, provides reserves to the developing embryo. Interestingly, many aspects of endosperm development, again like the placenta, are regulated by a range of epigenetic mechanisms that are globally termed imprinting. Imprinted genes are characterized by their uniparental expression, the other parental allele being silenced. Normal development of the endosperm thus requires a highly specific balance of gene expression, from either the maternal or paternal genomes. Any alteration of this balance resulting from changes in allelic copy number, sequence or epigenetic imprints can cause endosperm failure and eventual seed abortion. In its widest sense, the endosperm thus serves as an accurate 'sensor' of compatibility between parents. A first step in understanding this important, yet complex system must clearly be the isolation and characterization of as wide a range as possible of imprinted genes.", 
"Embryonic stem (ES) cells are permanent pluripotent stem cell lines established from pre-implantation mouse embryos. There is currently great interest in the potential therapeutic applications of analogous cells derived from human embryos. The isolation of ES cells is commonly presented as a straightforward transfer of cells in the early embryo into culture. In reality, however, continuous expansion of pluripotent cells does not occur in vivo, and in vitro is the exception rather than the norm. Both genetic and epigenetic factors influence the ability to derive ES cells. We have tracked the expression of a key marker and determinant of pluripotency, the transcription factor Oct-4, in primary cultures of mouse epiblasts and used this to assay the effect of experimental manipulations on the maintenance of a pluripotent cell compartment. We find that expression of Oct-4 is often lost prior to overt cytodifferentiation of the epiblast. The rate and extent of Oct-4 extinction varies with genetic background. We report that treatment with the MAP kinase/ERK kinase inhibitor PD98059, which suppresses activation of the mitogen-activated protein kinases Erk1 and Erk2, results in increased persistence of Oct-4-expressing cells. Oct-4 expression is also relatively sustained in cultures of diapause embryos and of isolated inner cell masses. Combination of all three conditions allowed the derivation of germline-competent ES cells from the normally refractory CBA mouse strain. These findings suggest that the genesis of an ES cell is a relatively complex process requiring epigenetic modulation of key gene expression over a brief time-window. Procedures that extend this time-window and/or directly regulate the critical genes should increase the efficiency of ES cell derivation.", 
"We suggest simulating evolution of complex organisms using a model constrained solely hv the requirement of robustness in its expression patterns. This scenario is illustrated by evolving discrete logical networks with epigenetic properties. Evidence for dynamical features in the evolved networks is found that carl be related to biological observables.", 
"Purpose: To investigate the associations between inflammation tests and radiation dose in A-bomb survivors. Subjects and methods: Subjects were A-bomb survivors who underwent inflammation tests of leukocyte counts, neutrophil counts, erythrocyte sedimentation rate, corrected erythrocyte sedimentation rate, alpha -1 globulin, alpha -2 globulin and sialic acid between 1988 and 1992. Associations with radiation dose (DS86) were analyzed by regression analysis and heterogeneity among inflammatory diseases, anaemia at examination, or history of cancer was also tested. Results: The associations with radiation dose were statistically significant for leukocyte counts (71.0 mm(-3) Gy(-1), p=0.015), erythrocyte sedimentation rate (1.58 mm h(-1) Gy(-1), p=0.0001), corrected erythrocyte sedimentation rate (1.14 mm h(-1) Gy(-1), p=0.0001), alpha -1 globulin (0.0057 g dl(-1) Gy(-1), p=0.0001), alpha -2 globulin (0.0128 g dl(-1) Gy(-1), p=0.0001), and sialic acid (1.2711 mg dl(-1) Gy(-1), p = 0.0001) but not for neutrophil counts (29.9 mm(-3) Gy(-1), p=0.17). Heterogeneity was not statistically significant. Among inflammatory diseases, associations were the strongest for chronic thyroiditis and chronic liver diseases. Conclusions: This study suggests statistically significant association between inflammation in A-bomb survivors and radiation dose of during 1988-1992. The association might contribute, as an epigenetic and/or bystander effect, to development of several radiation-induced disorders.", 
"Epigenetic asymmetry between parental genomes and embryonic lineages exists at the earliest stages of mammalian development. The maternal genome in the zygote is highly methylated in both its DNA and its histones and most imprinted genes have maternal germline methylation imprints. The paternal genome is rapidly remodelled with protamine removal, addition of acetylated histories, and rapid demethylation of DNA before replication. A minority of imprinted genes have paternal germline methylation imprints. Methylation and chromatin reprogramming continues during cleavage divisions, but at the blastocyst stage lineage commitment to inner cell mass (ICM) or trophectoderm (TE) fate is accompanied by a dramatic increase in DNA and histone methylation, predominantly in the ICM. This may set up major epigenetic differences between embryonic and extraembryonic tissues, including in X-chromosome inactivation and perhaps imprinting. Maintaining epigenetic asymmetry appears important for development as asymmetry is lost in cloned embryos, most of which have developmental defects, and in particular an imbalance between extraembryonic and embryonic tissue development.", 
"The polymerization of many amyloids is a two-stage process initiated by the formation of a seeding nucleus or protofibril. Soluble protein then assembles with these nuclei to form amyloid fibers. Whether fiber growth is bidirectional or unidirectional has been determined for two amyloids, In these cases, bidirectional growth was established by time lapse atomic-force microscopy, Here, we investigated the growth of amyloid fibers formed by NM, the prion-determining region of the yeast protein Sup35p, The conformational changes in NM that lead to amyloid formation in vitro serve as a model for the self-perpetuating conformational changes in Sup35p that allow this protein to serve as an epigenetic element of inheritance in vivo. To assess the directionality of fiber growth, we genetically engineered a mutant of NM so that it contained an accessible cysteine residue that was easily labeled after fiber formation. The mutant protein assembled in vitro with kinetics indistinguishable from those of the wild-type protein and propagated the heritable genetic trait [PSI+] with the same fidelity. In reactions nucleated with prelabeled fibers, unlabeled protein assembled at both ends. Thus, NM fiber growth is bidirectional. (C) 2001 Elsevier Science Ltd. All rights reserved.", 
"Background: Members of the GATA family of zinc finger transcription factors are genetically controlled master regulators of development in the hematopoietic and nervous systems. Whether GATA factors also serve to integrate epigenetic signals on target promoters is, however, unknown. The transforming growth factor-beta (TGF-beta) superfamily is a large group of phylogenetically conserved secreted factors controlling cell proliferation, differentiation, migration, and survival in multiple tissues. Results: GATA-3, a key regulator of T helper cell development, was found to directly interact with Smad3, an intracellular signal transducer of TGF-beta. Complex formation required a central region in GATA-3 and the N-terminal domain of Smad3. GATA-3 mediated recruitment of Smad3 to GATA binding sites independently of Smad3 binding to DNA, and the two factors cooperated synergistically to regulate transcription from the IL-5 promoter in a TGF-beta-dependent manner. Treatment of T helper cells with TGF-beta promoted the formation of an endogenous Smad3/GATA-3 nuclear complex and stimulated production of the Th2 cytokine IL-10 in a Smad3- and GATA-3-dependent manner. Conclusions: Although Smad proteins are known to interact with a number of general transcription factors, these are insufficient to explain the tissue-specific biology of TGF-beta proteins. Through its interaction with Smad3, GATA-3 is able to integrate a genetic program of cell differentiation with an extracellular signal, providing a molecular framework for the effects of TGF-beta on the development and function of specific subsets of immune cells and possibly other cell types.", 
"Whenever the state of a biological system is not determined solely by present conditions but depends on its past history, we can say that the system has memory. Bacteria and bacteriophage use a variety of memory mechanisms, some of which seem to convey adaptive value. A genetic type of heritable memory is the programmed inversion of specific DNA sequences, which causes switching between alternative patterns of gene expression. Heritable memory can also be based on epigenetic circuits, in which a system with two possible steady states is locked in one or the other state by a positive feedback loop. Epigenetic states have been observed in a variety of cellular processes, and are maintained by diverse mechanisms. Some of these involve alternative DNA methylation patterns that are stably transmitted to daughter molecules and can affect DNA-protein interactions (e.g., gene transcription). Other mechanisms exploit autocatalytic loops whereby proteins establish the proper conditions for their continued synthesis. Template polymers other than nucleic acids (e.g., components of the cell wall) may also propagate epigenetic states. Non-heritable memory is exemplified by parasitic organisms that bear a signature of their previous host, such as host-controlled modification of phage DNA or porin hitchhiking in predatory bacteria. The heterogeneous nature of the examples known may be indicative of widespread occurrence of memory mechanisms in bacteria and phage. However, the actual extent, variety and potential selective value of prokaryotic memory devices remain open questions, still to be addressed experimentally. (C) 2002 Wiley Periodicals, Inc.", 
"Recent inquiries have revealed a surprisingly large number (>2500) of naturally occurring antisense transcripts [1-4], but their function remains largely undiscovered. A better understanding of antisense mechanisms is clearly needed because of their potentially diverse roles in gene regulation and disease [5-8]. A well-documented case occurs in X inactivation, the mechanism by which X-linked gene expression is equalized between XX females and XY males [9]. The antisense gene Tsix [6] determines X chromosome choice and represses the noncoding silencer, Xist [10-12]. In principle, Tsix action may involve RNA, the act of transcription, or local chromatin. Here, we create novel Tsix alleles to distinguish transcription-versus RNA-based mechanisms. When Tsix transcription is terminated before Xist (Tsix(TRAP)), Tsix cannot block Xist upregulation, suggesting the importance of overlapping antisense transcription. To separate the act of transcription from RNA, we knocked in Tsix cDNA in the reverse orientation (Tsix(cDNA)) to restore RNA levels in cis without concurrent transcription across Xist. However, Tsix(cDNA) cannot complement Tsix(TRAP). Surprisingly, both mutations disrupt choice, indicating that this epigenetic step requires transcription. We conclude that the processed antisense RNA does not act alone and that Tsix function specifically requires antiparallel transcription through Xist. A mechanism of transcription-based feedback regulation is proposed.", 
"Malignant brain tumors consist of a number of distinct subclonal populations. Each of these subpopulations may be characterized by its own behaviors and properties. These subpopulations arise from the constant genetic and epigenetic alteration of existing cells in the rapidly growing tumor. However, since each single-cell mutation only leads to a small number of offspring initially, very few newly arisen subpopulations survive more than a short time. The present work quantifies emergence, i.e, the likelihood of an isolated subpopulation surviving for an extended period of time. Only competition between clones is considered; there are no cooperative effects included. The probability that a subpopulation emerges under these conditions is found to be a sigmoidal function of the degree of change in cell division rates. This function has a non-zero value for mutations which confer no advantage in growth rate, which represents the emergence of a distinct subpopulation with an advantage that has yet to be selected for, such as hypoxia tolerance or treatment resistance. A logarithmic dependence on the size of the mutated population is also observed. A significant probability of emergence is observed for subpopulations with any growth advantage that comprise even 0.1% of the proliferative cells in a tumor. The impact of even two clonal populations within a tumor is shown to be sufficient such that a prognosis based on the assumption of a monoclonal tumor can be markedly inaccurate. (C) 2000 Academic Press.", 
"Controversy has reigned for some time over the biological connection between DNA methylation and cancer. For this reason, the methylation mechanism responsible for increased cancer risk has received greater attention in recent years. Tumor suppressor genes are often hyper-methylated resulting in gene silencing. Although some have questioned this interpretation of the link between methylation and cancer, it appears that both hyper-methylation and hypomethylation events can create epigenetic changes that can contribute to cancer development. Recent studies have shown that the methyltransferases DNMT1 and DNMT3b cooperatively maintain DNA methylation and gene silencing in human cancer cells. Disruption of the human DNMT3b only slightly reduces the overall global DNA methylation; however, demethylation was markedly potentiated when both DNMT1 and DNMT3b were simultaneously deleted. The results to these experiments provide compelling evidence towards a role for DNA methylation in cancer. This review discusses the current understanding of cancer-epigenetic information and highlights recent studies that connect the methylation machinery and chromatin remodelling with cancer susceptibility. (C) 2003 Wiley Periodicals, Inc.", 
"The transplacental exposure of fetuses to maternal cigarette smoke may increase the risk of developmental impairments, congenital diseases, and childhood cancer. The whole-body exposure of Swiss mice to environmental cigarette smoke (ECS) during pregnancy decreased the number of fetuses per dam, placenta weight, and fetus weight. ECS increased DNA adducts, oxidative nucleotide alterations, and cytogenetic damage in fetus liver. Evaluation by cDNA array of 746 genes showed that 61 of them were expressed in fetus liver under basal conditions. The oral administration of N-acetylcysteine (NAC) during pregnancy enhanced the expression of three genes only, including two glutathione S-transferases and alpha1-antitrypsin precursor, whose deficiency plays a pathogenetic role in congenital emphysema. Transplacental ECS upregulated the expression of 116 genes involved in metabolism, response to oxidative stress, DNA and protein repair, and signal transduction. NAC inhibited the ECS-related genetic damage and upregulation of most genes. ECS stimulated pro-apoptotic genes and genes downregulating the cell cycle, which may justify growth impairments in the developing fetus. Thus, both genetic and epigenetic mechanisms were modulated by ECS. Moreover, hypoxia-related genes and several oncogenes and receptors involved in proliferation and differentiation of leukocytes were induced in the fetal liver, which also bears hematopoietic functions.", 
" ", 
"The purpose of the present study was to investigate the effect of thermal conditioning, (through exposure to heat stress), during pre-hatch development on some physiological responses of post-hatch broilers to a post-natal heat stress challenge. Exposure to heat stress at this stage, we hope, may possibly induce epigenetic heat adaptation. Incubating eggs were exposed to temperature of 39.0degreesC for 2 h from Day 13 to 17 of incubation. At 33, 35, 37, 39, 41 and 43 d of age, the broilers hatched from these eggs were housed individually in open-circuit respiration cells. The climatic chambers were set to 22degreesC and increased to 30degreesC for 4 h. O-2 consumption and CO2 production of each chicken was monitored continuously in order to calculate the heat production. Blood samples were obtained before and during the 4 h heat stress. Thermal conditioning during incubation did not affect the plasma T-4, corticosterone, glucose, uric acid and CK concentrations. Temperature challenge, decreased plasma T-3 of broilers of both groups but the decrease was greater in pre-conditioned broilers compared with controls. A similar trend was observed for triglycerides. These changes did not affect total heat production. Since decreased T3 and triglyceride levels are part of the mechanisms for thermoregulation, these suggest that thermal conditioning during incubation can improve the broiler chicken capability for thermotolerance at later post-hatch age. (C) 2002 Elsevier Science Ltd. All rights reserved.", 
"Autosomal genes that are subject to random allelic Inactivation (RAI), like imprinted genes [1] and genes subject to X-inactivation Gal, require mechanisms that dictate the differential transcriptional regulation of two sequence-identical alleles. RAI genes include olfactory receptor genes [3], and the various genes encoding antigen receptors on lymphocytes (immunoglobulin genes, T cell receptor genes and NK receptor genes [4-7]). These observations raise the possibility that other genes might be similarly regulated. Moreover, an interesting possibility is that certain genes might be monoallelically expressed in some cells and biallelically expressed in others. Recently, reports of monoallelic expression of interleukin-2 (IL-2) [8,9] and IL-4 [10,11] have raised the possibility that the cytokine gene family may be subject to monoallelic expression. Another report suggests that the gene encoding the transcription factor Pax-5, which is involved in B-cell (and cerebellar) development [12,13], is also subject to monoallelic expression [14]. Using a novel single-cell reverse transcription-polymerase chain reaction (RT PCR) approach, we have analyzed the IL-2 and Pax-5 genes in mice. We found that IL-2 is monoallelically transcribed in some T cells and biallelically transcribed in others, raising interesting questions regarding cytokine gene regulation. Additionally, our analyses suggest that Pax-5 is consistently biallelically transcribed. Thus, the IL-2 gene and other cytokine genes may be regulated in a stochastic manner that results in 0, 1 or 2 alleles of a given cytokine gene expressed in each T cell. This type of regulation could account for the wide variety of different combinations of cytokine genes expressed in individual T cells and therefore plays a role in the generation of T cells with a range of different effector functions. (C) 2000 Elsevier Science Ltd. All rights reserved.", 
"Recent studies on the transcriptional regulation of two linked, imprinted genes, Igf2 and H19, have provided evidence for a novel mechanism of epigenetic control. DNA methylation controls the activity of an insulator element located between the two linked genes by regulating the binding of the zinc-finger protein CTCF. (C) 2000 Elsevier Science Ltd. All rights reserved.", 
"Reversible acetylation of histone tails plays an important role in chromatin remodelling and regulation of gene activity [1, 2]. While modification by histone acetyltransferase (HAT) is usually linked to transcriptional activation, we provide here evidence for HAT function in several types of epigenetic repression. Chameau (Chm), a new Drosophila member of the MYST HAT family, dominantly suppresses position effect variegation (PEV), is required for the maintenance of Hox gene silencing by Polycomb group (PcG) proteins, and can partially substitute for the MYST Sas2 HAT in yeast telomeric position effect (TPE). Finally, we provide in vivo evidence that the acetyltransferase activity of Chm is required in these processes, since a variant protein mutated in the catalytic domain no longer rescues PEV modification, telomeric silencing of SAS2-deficient yeast cells, nor lethality of chm mutant flies. These findings emphasize the role of an acetyltransferase in gene silencing, which supports, according to the histone code hypothesis [2-4], that transcription at a particular locus is determined by a precise combination of histone tail modifications rather than by overall acetylation levels.", 
"Many scientists and philosophers of science are troubled by the relative isolation of developmental from evolutionary biology. Reconciling the science of development with the science of heredity preoccupied a minority of biologists for much of the twentieth century, but these efforts were not corporately successful. Mainly in the past fifteen years, however, these previously dispersed integrating programmes have been themselves synthesized and so reinvigorated. Two of these more recent synthesizing endeavours are evolutionary developmental biology (EDB, or evo-devo) and developmental systems theory (DST). While the former is a bourgeoning and scientifically well-respected biological discipline, the same cannot be said of DST, which is virtually unknown among biologists. In this review, we provide overviews of DST and EDB, summarize their key tenets, examine how they relate to one another and to the study of epigenetics, and survey the impact that DST and EDB have had (and in future should have) on biological theory and practice. (C) 2001 John Wiley & Sons, Inc.", 
"Background: SIR2 was originally identified in S. cerevisiae for its role in epigenetic silencing through the creation of specialized chromatin domains. It is the most evolutionarily conserved protein deacetylase, with homologs in all kingdoms. SIR2 orthologs in multicellular eukaryotes have been implicated in lifespan determination and regulation of the activities of transcription factors and other proteins. Although SIR2 has not been widely implicated in epigenetic silencing outside yeast, Drosophila SIR2 mutations were recently shown to perturb position effect variegation, suggesting that the role of SIR2 in epigenetic silencing may not be restricted to yeast. Results: Evidence is presented that Drosophila SIR2 is also involved in epigenetic silencing by the Polycomb group proteins. Sir2 mutations enhance the phenotypes of Polycomb group mutants and disrupt silencing of a mini-white reporter transgene mediated by a Polycomb response element. Consistent with this, SIR2 is physically associated with components of an E(Z) histone methyltransferase complex. SIR2 binds to many euchromatic sites on polytene chromosomes and colocalizes with E(Z) at most sites. Conclusions: SIR2 is involved in the epigenetic inheritance of silent chromatin states mediated by the Drosophila Polycomb group proteins and is physically associated with a complex containing the E(Z) histone methyltransferase.", 
"'Viable yellow' (A(vy)/alpha) mice are larger, obese, hyperinsulinemic, more susceptible to cancer, and, on average, shorter lived than their non-yellow siblings. They are epigenetic mosaics ranging from a yellow phenotype with maximum ectopic agouti overexpression, through a continuum of mottled agouti/yellow phenotypes with partial agouti overexpression, to a pseudoagouti phenotype with minimal ectopic expression. Pseudoagouti A(vy)/a mice are lean, healthy, and longer lived than their yellow siblings. Here we report that feeding pregnant black a/a dams methyl-supplemented diets alters epigenetic regulation of agouti expression in their offspring, as indicated by increased agouti/black mottling in the direction of the pseudoagouti phenotype, We also present confirmatory evidence that epigenetic phenotypes are maternally heritable. Thus A(vy) expression, already known to be modulated by imprinting, strain-specific modification, and maternal epigenetic inheritance, is also modulated by maternal diet. These observations suggest, at least in this special case, that maternal dietary supplementation may positively affect health and longevity of the offspring, Therefore, this experimental system should be useful for identifying maternal factors that modulate epigenetic mechanisms, especially DNA methylation, in developing embryos.", 
"Persistence of individual differences in animal behavior in reactions to various environmental challenges could reflect basic divergences in temperament, which might be used to predict details of adaptive response. Although studies have been carried out on fear and anxiety in various species, including laboratory, domestic and wild animals, no consistent definition of fearfulness as a basic trait of temperament has emerged. After a classification of the events that may produce a state of fear, this article describes the great variability in behavior and in physiological patterns generally associated with emotional reactivity. The difficulties of proposing fearfulness - the general capacity to react to a variety of potentially threatening situations - as a valid basic internal variable are then discussed. Although there are many studies showing covariation among the psychobiological responses to different environmental challenges, other studies find no such correlations and raise doubts about the interpretation of fearfulness as a basic personality trait. After a critical assessment of methodologies used in fear and anxiety studies, it is suggested that discrepancies among results are mainly due to the modulation of emotional responses in animals, which depend on numerous genetic and epigenetic factors. It is difficult to compare results obtained by different methods from animals reared under various conditions and with different genetic origins. The concept of fearfulness as an inner trait is best supported by two kinds of investigations. First, an experimental approach combining ethology and experimental psychology produces undeniable indicators of emotional reactivity. Second, genetic lines selected for psychobiological traits prove useful in establishing relationships between behavioral and neuroendocrine aspects of emotional reactivity. It is suggested that fearfulness could be considered a basic feature of the temperament of each individual, one that predisposes it to respond similarly to a variety of potentially alarming challenges, but is nevertheless continually modulated during development by the interaction of genetic traits of reactivity with environmental factors, particularly in the juvenile period. Such interaction may explain much of the interindividual variability observed in adaptive responses.", 
"Genomic instability is characterized by the increased rate of acquisition of alterations in the mammalian genome. These changes encompass a diverse set of biological end points including karyotypic abnormalities, gene mutation and amplification, cellular transformation, clonal heterogeneity and delayed reproductive cell death, The loss of stability of the genome is becoming accepted as one of the most important aspects of carcinogenesis, and the numerous genetic changes associated with the cancer cell implicate genomic stability as contributing to the neoplastic phenotype, Multiple metabolic pathways govern the accurate duplication and distribution of DNA to progeny cells; other pathways maintain the integrity of the information encoded by DNA and regulate the expression of genes during growth and development. For each of these functions, there is a normal baseline frequency at which errors occur, leading to spontaneous mutations and other genomic anomalies. This review summarizes the current status of knowledge about radiation-induced genomic instability. Those events and processes likely to be involved in the initiation and perpetuation of the unstable phenotype, the potential role of epigenetic factors in influencing the onset of genomic instability, and the delayed effects of cellular exposure to ionizing radiation are discussed. (C) 1996 by Radiation Research Society", 
"The trilaminar kinetochore directs the segregation of chromosomes in mitosis and meiosis. Despite its importance, the molecular architecture of this structure remains poorly understood [1]. The best known component of the kinetochore plates is CENP-C, a protein that is required for kinetochore assembly [2], but whose molecular role in kinetochore structure and function is unknown. Here we have raised for the first time monospecific antisera to CENP-A [3], a 17 kD centromere-specific histone variant that is 62% identical to the carboxy-terminal domain of histone H3 [4,5] and that resembles the yeast centromeric component CSE4 [6]. We have found by simultaneous immunofluorescence with centromere antigens of known ultrastructural location that CENP-A is concentrated in the region of the inner kinetochore plate at active centromeres. Because CENP A was previously shown to co-purify with nucleosomes [7], our data suggest a specific nucleosomal substructure for the kinetochore. In human cells, these kinetochore-specific nucleosomes are enriched in alpha-satellite DNA [8]. However, the association of CENP-A with neocentromeres lacking detectable alpha-satellite DNA, and the lack of CENP-A association with alpha-satellite-rich inactive centromeres of dicentric chromosomes together suggest that CENP-A association with kinetochores is unlikely to be determined solely by DNA sequence recognition. We speculate that CENP-A binding could be a consequence of epigenetic tagging of mammalian centromeres.", 
"A programme of research into phase change in the desert locust, Schistocerca gregaria, is described. The ability to change phase between solitarious and gregarious forms in response to population density is a key feature of locusts and is central to their occasional yet catastrophic impact on humans. Phase polymorphism is an extreme form of phenotypic plasticity. The most labile phase characteristic is behaviour. It is argued that a fully integrated study of behavioural phase change provides a powerful tool for understanding both the mechanisms of phase change and locust population dynamics, both of which offer possibilities for improved management and control. of desert locust plagues. An assay for measuring behavioural phase-state in individual locusts was derived, based on logistic regression analysis. Experiments are described that used the assay to quantify the time-course of behavioural change, both within the life of individual locusts and across generations. The locust-related stimuli that provoke behavioural gregarization were investigated. Complex interactions were found between tactile, visual and olfactory stimuli, with the former exerting the strongest effect. Behavioural analysis also directed a study of the mechanisms whereby adult females exert an epigenetic influence over the phase-state of their developing offspring. Female locusts use their experience of the extent and recency of being crowded to predict the probability that their offspring will emerge into a high-density population, and alter the development of their embryos accordingly through a gregarizing agent added to the foam that surrounds the eggs at laying. There is also a less pronounced paternal influence on hatchling phase-state. An understanding of the time-course of behavioural phase change led to a study of the effect of the fine-scale distribution of resources in the environment on interactions between individual locusts, and hence on phase change. This, in turn, stimulated an exploration of the implications of individual behavioural phase change for population dynamics. Cellular automata models were derived that explore the relationships between population density, density of food resources and the distribution of resources in the environment. The results of the simulation showed how the extent of gregarization within a population increases with rising population size relative to food abundance and increasing concentration of food resources. Of particular interest was the emergence of critical zones across particular combinations of resource abundance, resource distribution and population size, where a solitarious population would rapidly gregarize. The model provided the basis for further laboratory and field experiments, which are described.", 
"Acquired epigenetic modifications, such as DNA methylation or stable chromatin structures, are not normally thought to be inherited through the germline to future generations in mammals [1,2]. Studies in the mouse have shown that specific manipulations of early embryos, such as nuclear transplantation, can result in altered patterns of gene expression and induce phenotypic alterations at later stages of development [3-5]. These effects are consistent with acquired epigenetic modifications that are somatically heritable, such as DNA methylation. Repression and DNA methylation of genes encoding major urinary proteins, repression of the gene encoding olfactory marker protein, and reduced body weight can be experimentally induced by nuclear transplantation in early embryos M. Strikingly, we now report that these acquired phenotypes are transmitted to most of the offspring of manipulated parent mice. This is the first demonstration of epigenetic inheritance of specific alterations of gene expression through the germline, These observations establish a mammalian model for transgenerational effects that are important for human health, and also raise the question of the evolutionary importance of epigenetic inheritance. (C) Current Biology Ltd.", 
"The pathway of molecular interactions leading to kinetochore assembly on mammalian chromosomes is unknown. Kinetochores could be specified by structural features of centromeric satellite DNA [1-3] or by specific DNA sequences, analogous to budding yeast centromeres, interspersed in centromeric satellite DNA arrays [4,5]. Alternatively, kinetochores could be epigenetic structures that replicate without strict dependence on DNA sequence [6-8]. We purified kinetochore-associated chromatin from human chromosomes by immunoprecipitation of CENP-A, a centromere-specific histone H3 homologue located in the inner plate of the kinetochore [6,9,10]. Hybridization and DNA sequence analyses of cloned kinetochore DNA fragments revealed alpha-satellite as the predominant sequence associated with CENP-A. A major site of micrococcal nuclease digestion was identified by mapping the termini of alpha-satellite clones, suggesting that the inner kinetochore plate contains phased arrays of CENP-A-alpha-satellite nucleosomes. These experiments demonstrate for the first time that complex satellite DNA is a structural component of the kinetochore. Further, because complex satellite DNA is evolutionarily unconserved, these results suggest that molecular recognition events necessary for kinetochore formation take place at the level of DNA conformation or epigenetic mechanisms rather than DNA sequence per se.", 
"Purpose: This report reviews the data obtained in the author's laboratory over the past two decades, which support the hypothesis that radiation can induce a genome-wide process of instability in mammalian cells that is transmitted over many generations of cell replication leading to an enhanced frequency of genetic changes occurring among the progeny of the original irradiated cell. Methods: Endpoints examined are malignant transformation, delayed reproductive failure, chromosomal aberrations and mutagenesis; the relationship between these endpoints is not clear. Results: The instability phenotype was induced in approximately 10-20% of the irradiated cell population with doses of 4-12 Gy, and may be induced by much smaller doses. Mutational instability can be induced by either X-rays or ct-particles; the spectrum of DNA structural changes in mutations arising in such unstable populations differed significantly from that for direct radiation-induced mutations. Evidence is also presented for the occurrence of coincident mutations, further supporting the existence of a genome-wide process of hypermutability. Finally, data suggest that epigenetic factors acting in the host may significantly influence the development of genomic instability in vivo. Conclusions: It appears that radiation can induce a type of genomic instability in mammalian cells that is transmissible to their progeny. There remain, however, many unanswered questions concerning this phenomenon, in particular how it is initiated and how it is maintained over many generations of cell replication.", 
"There have been allegations in the media and in the courts that cell phones and other types of hand-held transceivers are a cause of cancer. There have also been numerous public objections to the siting of TV, radio and cell phone transmission facilities because of a fear of cancer induction. A recent publication in Radiation Research by Repacholi et al, (147, 631-640, 1997) which suggests that exposure to radiofrequency (RF) radiation may increase lymphoma incidence in mice has contributed to this controversy. The goal of this review is to provide biomedical researchers a brief overview of the existing RF radiation-cancer studies. This article begins with a brief review of the physics and technology of cell phones, It then reviews the existing epidemiological studies of RF radiation, identifying gaps in our knowledge. Finally, the review discusses the cytogenetics literature on RF radiation and the whole-animal RE-radiation carcinogenesis studies, The epidemiological evidence for an association between RF radiation and cancer is found to be weak and inconsistent, the laboratory studies generally do not suggest that cell phone RF radiation has genotoxic or epigenetic activity, and a cell phone RF radiation-cancer connection is found to be physically implausible. Overall, the existing evidence for a causal relationship between RF radiation from cell phones and cancer is found to be weak to nonexistent. (C) 1999 by Radiation Research Society.", 
"Recent work with plants has demonstrated that genome instability can be triggered by a change in chromosome number arising from either whole genome duplications (polyploidy) or loss/gain of individual chromosomes (aneuploidy). This genome instability is manifested as rapid structural and epigenetic alterations that can occur somatically or meiotically within a few generations after heteroploid formation. The intrinsic instability of newly formed polyploid and aneuploid genomes has relevance for genome evolution and human carcinogenesis, and points toward recombinational and epigenetic mechanisms that sense and respond to chromosome numerical changes. (C) 1999 John Wiley & Sons, Inc.", 
"Purpose: To review studies of radiation-induced genomic instability in haemopoietic cells. Major findings: Studies have demonstrated a high frequency of non-clonal, cytogenetic abnormalities in the clonal descendants of alpha-particle-irradiated (approximately one track per cell) primary murine and human haemopoietic stem cells in vitro. The induction of this phenomenon has a strong dependence on the genetic characteristics of the cells and is transmissible in vivo following transplantation of alpha-irradiated mouse bone marrow. In clonogenic cell cultures of alpha-irradiated haemopoietic cells, there is also an increased incidence of hprt mutations and an increased incidence of apoptosis. These effects may be regarded as the consequences of a destabilization of the genome collectively termed radiation-induced genomic instability. Instability is induced at very high frequencies suggesting that epigenetic changes may be a common underlying mechanism. Consistent with this suggestion is the finding of an enhanced and persisting oxy-radical activity in the descendants of irradiated stem cells, which is consistent with oxidative stress being an important feature of the mechanism(s) underlying the persistence of instability in haemopoietic cells. Recent studies have revealed that more clonogenic cells than are actually traversed by an alpha-particle are able to express the instability phenotype. These data are consistent with unexpected interactions between irradiated and nonirradiated cells but the mechanism of initiation of instability is not understood.", 
"Epigenetic inheritance systems enable the environmentally induced phenotypes to be transmitted between generations. Jablonka and Lamb (1991, 1995) proposed that these systems have a substantial role during speciation. They argued that divergence of isolated populations may be first triggered by the accumulation of (heritable) phenotypic differences that are later followed and strengthened by genetic changes. The plausibility of this idea is examined in this paper. At first, we discuss the exploratory behaviour of an epigenetic inheritance system on a one peak adaptive landscape. If a quantitative trait is far from the optimum, then it is advantageous to induce heritable phenotypic variation. Conversely, if the genotypes get closer to the peak, it is more favorable to canalize the phenotypic expression of the character. This process would lead to genetic assimilation. Next we show that the divergence of heritable epigenetic marks acts to reduce or to eliminate the genetic barrier between two adaptive peaks. Therefore, an epigenetic inheritance system can increase the probability of transition from one adaptive state to another. Peak shift might be initiated by (i) slight changes in the inducing environment or by (ii) genetic drift of the genes controlling epigenetic variability. Remarkably, drift-induced transition is facilitated even if phenotypic variation is not heritable. A corollary of our thesis is that evolution can proceed through suboptimal phenotypic states, without passing through a deep adaptive valley of the genotype. We also consider the consequences of this finding on the dynamics and mode of reproductive isolation. (C) 1999 Academic Press.", 
"An isologous diversification theory for cell differentiation is proposed, based on simulations of interacting cells with biochemical networks and the cell division process following consumption of some chemicals. According to the simulations of the interaction-based dynamical systems model, the following scenario of the cell differentiation is proposed. (1) Up to some threshold number, divisions bring about almost identical cells with synchronized biochemical oscillations. (2) As the number is increased, the oscillations lose synchrony, leading to groups of cells with different phases of oscillations. (3) Amplitudes of oscillation and averaged chemical compositions start to differ by groups of cells. The differentiated behavior of states is transmitted to daughter cells. (4) Recursivity is formed so that the daughter cells keep the identical chemical character. This ''memory'' is made possible through the transfer of initial conditions. (5) Successive differentiation proceeds. The mechanism of tumor cell formation, origin of stem cells, anomalous differentiation by transplantations, apoptosis and other features of cell differentiation process are also discussed, with some novel predictions. (C) 1997 Society for Mathematical Biology", 
"A mutation that disrupts post-transcriptional gene silencing in Neurospora crassa has been found to affect the homologue of a plant-encoded RNA-dependent RNA polymerase. This enzyme may produce a specificity determinant of gene silencing and mediate an epigenetic conversion at the RNA level.", 
"Isologous diversification, proposed for cell differentiation, is shown to be stable against molecular and other external fluctuations, where amplification of noise-induced slight difference between cells leads to a noise-tolerant society with differentiated cell types. It is a general consequence of interacting cells with biochemical networks and cell divisions, as is confirmed by several model simulations. According to the theory, differentiation proceeds first by loss of synchrony of intracellular oscillations as the number of cells increases. Then the chemical composition of the cells is differentiated. The differentiated compositions become inherited by the next generation, and lead to determined cell types. As a result of successive occurrence of the cell differentiation, the cell society will be composed of different cell types. The whole developmental process is robust not only against molecular fluctuations but also against the removal of a cluster of cells. This robustness is a remarkable feature of isologous diversification, in contrast to the conventional threshold-type mechanism for development. As a testable consequence of the theory, we also discuss interaction-dependent tumor formation and negative correlation between growth speed and chemical diversity. (C) 1999 Academic Press.", 
" ", 
"Four cases of paramutation and gene silencing in plants differ in their genetic properties, but each may Involve epigenetic interactions between transposable elements.", 
"The delayed expression of cell death in progeny of irradiated survivors was christened 'lethal mutations' by Tikvah Alper in 1984. The effect occurs when clones, or populations of cells grown up from irradiated progenitor cells, are replated and reassessed for cloning efficiency or population doubling time. The effect has been shown to be associated with the low dose shoulder region of the survival curve and is due to events occurring in the first two hours post irradiation, i.e. the 'fast repair' period. In this review the lethal mutation data accumulated over the past ten years is discussed in relation to our modern understanding of cellular and molecular events in radiation carcinogenesis and genomic instability research. It is suggested that lethal mutations are associated with a general epigenetic or field effect occurring in all irradiated cells, which makes them more prone to mutations, some of which are lethal. The implications of this for our current approach to risk estimates and therapeutic dose calculation, need to be addressed.", 
"The adaptive value of epigenetic inheritance systems is investigated in a simple mathematical framework. These systems enable the environmentally induced phenotypes to be transmitted between generations. The frequencies of the different epigenetic variants are determined by the plasticity and the efficiency of transmission (called memory). Plasticity and memory are genetically determined. This paper studies the evolution of a quantitative character, its plasticity and memory, on the adaptive landscape. Due to the dual inheritance of the character, selection acts on two revels: on the phenotypes of the same genotype, and on the different genotypes. Plasticity generates the raw material, and memory increases the strength of phenotypic selection. If the character is far from the peak of the landscape, then dual inheritance of the character can be advantageous for the genotype. Near the peak it is more favourable to suppress phenotypic variation. This would lead to genetic assimilation.", 
"Epigenetic regulation of transcription can lead to a stable differential expression of identical genetic information in the same cell or cell population. There is increasing evidence that higher order chromatin structures, involving specific multiprotein complexes, constitute one device to establish and maintain epigenetic marks. In addition, defined chromosomal elements conferring epigenetic inheritance of transcriptional expression states have recently been identified. During the period where the difference in expression of identical genes is established, these sequences appear to be used as switch elements by both negative and positive regulators. Once the epigenetic mark is set, the elements maintain either the silenced or the activated expression state over many cell generations. Here we review recent data obtained from analyzing epigenetic gene regulation in different organisms and show that similarities in the underlying mechanisms appear to exist. BioEssays 21:824-832, 1999. (C) 1999 John Wiley & Sons, Inc.", 
"Expression and transposition of the Suppressor-mutator (Spm) transposon of maize are controlled by interacting epigenetic and autoregulatory mechanisms. Methylation of critical element sequences prevents both transcription and transposition, heritably inactivating the element. The promoter, comprising the terminal 0.2 kb of the element, and a 0.35-kb, highly GC-rich, downstream sequence are the methylation target sequences. The element encodes two proteins necessary for transposition, TnpA and TnpD. There are multiple TnpA binding sites, both in the 5' terminal promoter region and at the element's 3' end. In addition to its role in transposition, TnpA is both a positive and a negative regulator of transcription. TnpA represses the element's promoter when it is not methylated. When the element is inactive and its promoter methylated, TnpA activates the methylated promoter and facilitates both its transient and heritable demethylation.", 
"Intracytoplasmic sperm injection (ICSI) is the latest, and by far the most efficient, variant of micromanipulation-assisted fertilization, whereby a single spermatozoon is selected, aspirated into a microinjection needle and injected to the oocyte cytoplasm. The development of this technique is mainly linked to application in human assisted reproduction for which it enables fertilization with defective spermatozoa that would not otherwise be able to penetrate an oocyte by their proper means. Because ICSI by-passes many steps of the natural fertilization process, it offers an extremely interesting model for the study of basic mechanisms underlying fertilization. This is particularly true for oocyte activation, whose mechanism needs to be revisited in light of the current ICSI research. The massive application of ICSI in human infertility treatment also represents a huge laboratory in which the impact of different genetic and epigenetic anomalies of the male gamete on fertilization and embryonic development can be studied. (C) 1999 John Wiley & Sons, Inc.", 
"The major paradigm for mathematically describing the carcinogenic process has been through the use of multistage models. Multistage models are made up of numerous compartments representing cells in various stages on the way to malignancy and where movement from one cell class to another is assumed to have exponential waiting time. Once a cell is in a particular class, clonal expansion through a linear birth-death process increases the size of the compartment, These models are characterized by movement of single cells from one compartment to another rather than clonal colonies of cells. However, there is some evidence to suggest that, in certain organs for certain types of agents, preneoplastic lesions with different phenotypes arise directly from an entire clonal colony rather than from a single cell within that colony. This manuscript describes a simple mathematical model of carcinogenesis using both persistent changes of single or several cells (to start the process) and shifting of colonies to describe the stages of the model. Likelihoods for the use of the model with data on colonies of preneoplastic lesions are described and applied to real data. (C) 1998 Elsevier Science Inc. All rights reserved.", 
"Cortical maps often contain global spatial structure; however, theoretical accounts for their development have generally concentrated on reproducing only local structure. We show that the elastic net model of cortical map formation can closely approximate the global structure of the ocular dominance column map observed in macaque primary visual cortex. A key component is the assumption of spatially non-uniform and anisotropic correlations in the retina. This work shows how genetic and epigenetic effects could combine to establish characteristic global structure in cortical maps.", 
"The formation of silent epialleles is accompanied by local hypermethylation of the DNA template, and genetically altered, methylation-deficient Arabidopsis mutants generate an increased number of epimutations. But what came first - methylation or epigenetic change? (C) Current Biology Ltd ISSN 0960-9822.", 
"Genomic imprinting is an epigenetic mechanism resulting in the preferential expression of the maternal or paternal alleles of a specific subset of genes in the mammalian genome. A key but relatively unexplored question is how imprints are established in the germline. New observations((1)) on two classical imprinting disorders, the Prader-Willi (PWS) and Angelman (AS) syndromes, offer the first genetic insight into this process. Molecular analysis of imprinting mutations that interfere with the appropriate establishment of the maternal and paternal epigenotypes has led to the identification of imprinted transcripts that could be involved in switching imprints in the germlines.", 
"In the mammalian brain dopamine systems play a central role in the control of movement, hormone release, emotional balance and reward. Alteration of dopaminergic neurotransmission is involved in Parkinson's disease and other movement disorders, as well as in some psychotic syndromes. This review summarises recent findings, which shed some light on signals and cellular interactions involved in the specification and maturation of the dopaminergic function during neurogenesis. In particular we will focus on three major issues: (1) the differentiation of dopaminergic neurones triggered by direct contact with the midbrain floor plate cells through the action of sonic hedgehog; (2) the neurotrophic factors acting on dopaminergic neurones; and (3) the role of target striatal cells on the survival and the axonal growth of developing or grafted dopaminergic neurones.", 
"Studies of regeneration research has a very profound historical background, longer than genetics and embryology per se. In this article, I have tried to make clear the importance of the discovery of Niazi on the effect of vitamin A on amphibian limb regeneration in the long tradition of regeneration study. His discovery is truly one of the main milestones in recent era of the field. It revitalized the study of regeneration, in particular the problem of replication of pattern, A more recent discovery of heteromorphic regeneration in tadpoles elicited by vitamin A by Mohanty-Hejmadi is also of a great historical significance. It has opened the way to investigate the most mysterious phenomena known from early 20th in modern eyes, most probably in terms of the epigenetic switch of homeotic genes.", 
"We have previously shown that primary explant cultures of human urothelium exposed to low doses of gamma-radiation subsequently accumulate a high level of stable p53 but it was not clear from those studies whether this protein stabilization occurred through an event in another gene involved in p53 protein control or possibly an epigenetic event. In these experiments, primary urothelial cultures from five different patients were exposed to either 0.5 or 5 Gy gamma-radiation from a (60)Cobalt source and allowed to grow for 7-10 division cycles to allow development of any radiation-induced, non-lethal changes in the cells. C-myc, Bcl-2 and stable p53 proteins were found to be elevated in cultures following both radiation doses. PCR-SSCPE analysis of the p53 gene was performed on cultures in order to determine whether genetic mutations could be the underlying basis for persistent increased stable p53 expression. Following 0.5 Gy exposure, the cultures also developed multiple distinct 'foci' of rapidly dividing cells which strongly overexpressed p53. These grew on a background of morphologically normal cells. When such foci were selectively analysed for their p53 mutation status by PCR-SSCPE, there was evidence that they contained cells which had developed changes to the p53 gene postirradiation. These changes appeared to occur more frequently in focal cells than in cells of normal morphological appearance in the same culture. These results may have mechanistic importance given the controversy regarding low-dose radiation effects and p53-related genomic instability.", 
"Purpose: The purpose of this review is to assess the effect of radiation-induced mutation on genes subject to genomic imprinting, and the consequences of this on the understanding of genomic instability. Genomic imprinting is the phenomenon in which one of the two alleles of a gene is expressed or suppressed depending on the gamete from which it was inherited, thus effectively rendering a cell hemizygous for the expression of certain key genes. The consequence of this is that such loci are potentially more likely targets for mutagenesis since one allele is normally inactive. This is not only important in the recognition of a subgroup of target genes for radiation-induced damage, but also raises the possibility of mutations affecting the epigenotype of key tumour suppressor or tumour promoting genes. Such mutations may in principle affect the stability of imprinting and may fall into a novel class of 'epimutation', where the DNA sequence is not affected, but post-transcriptional mechanisms of epigenotype maintenance are stably altered. These novel mechanisms are discussed in relation with radiation-induced genomic instability and the heritability of tumour predisposition from radiation-exposed parents. Conclusions: As yet there is only circumstantial evidence that the targets for radiation-induced DNA and epigenetic damage are imprinted genes, or genes involved in the maintenance of the epigenotype. However, the potential consequences of such genes being important targets for the generation of genomic instability or other forms of damage are serious and could affect the interpretation of the risks of low dose radiation exposure and of epidemiological data.", 
"The cell property of processing the environmental information is called sensitivity and referred to a ''sensitive'' cell system in which physical signals and specialized molecules work in relation to peculiar cell structure. It is suggested that, in addition to its role in cell behaviour, the sensitive system performs a basic role in morphogenesis in all cells. It interacts with the genetic system in gene expression and regulation directly or indirectly through epigenetic processes. Epigenetic inheritance, adaptive mutations and genetic assimilation allow environmental information to be embodied in new traits, thereby becoming relevant in evolution. This is consistent with an evolutionary mechanism with two alternate and recurrent phases: the first, epigenetic and Lamarckian at cell and organism levels and the second, selective and Darwinian, at population level.", 
"Genomic imprinting is an epigenetic marking process that confers parent-of-origin-dependent expression on certain genes. These imprinted genes are sometimes found in clusters, suggesting a possible involvement of higher order regulatory elements controlling expression and imprinting of genes organised in such clusters. In the distal chromosome 7 there are at least four imprinted genes: Mash2, lns2, lgf2 and H19. Recent evidence((1)) suggests that imprinting and expression of at least lgf2 and H19 may be mechanistically linked.", 
"The Gaia theory states that the whole surface layer of the globe, including organisms, atmosphere, hydrosphere and lithosphere, can be viewed as a self-evolving homeostatic system. Bacteria play a principal role in this formative process. This line of thinking should answer several questions: Is the biosphere simply a set of essentially selfish individuals, each testing its fitness? Is memory in the biosphere confined to genes? Is the composition of ecosystems and their behaviour only kind of dynamic equilibrium? Or is there a huge amount of epigenetic information that enables organisms to interpret properly their situation and behave accordingly? It is suggested here that it is the information flow, rather than pure thermodynamics and natural selection, that enables living beings not only to ''read'' their environment, interpret their situation and behave accordingly to this understanding, but also to establish geophysiology, to become part of Gaia. This flow comprises genes as well as signals, diffusible as well as permanent. From an analogy with the situation in the multicellular organism, three types of information mediators inside the body of Gaia are suggested: ultrastructure, diffusible signals and gene flux. With a shared gene pool, intricate ultrastructure and quickly diffusible signals, it is legitimate to consider Gaia a superorganism with an ontogeny. (C) 1995 Academic Press Limited", 
"The methylation of 5'CpG 3' dinucleotides within genes creates potential targets for protein complexes that bind to methylated DMA sequences and to histone deacetylases (MBD-HDAC). This can lead to transcriptional repression by modification of chromatin. To test the importance of this repression in vivo and to determine when during development these epigenetic controls are placed on genes, two novel genes have been engineered by directed mutagenesis of the CpG-rich LacZ gene that are depleted of (LagZ) or completely lacking (LagoZ) CpG sequences. We report that the expression (transcriptional and translational) of the three genes is indistinguishable in transient assays in cleaving mouse embryos. Therefore, the complete absence of CpG sequences within three kilobases of coding sequence is compatible with its maintenance in the nucleus and with its expression. These molecules can now be used to study the ontogenesis of the CpG-dependent repressive system in intact organisms. (C) 1999 Academie des sciences/Editions scientifiques et medicales Elsevier SAS.", 
"Multiple steady-state levels of autoregulatory transcription factors constitute alternative epigenetic states, and could help specify the heritable phenotype of an organism. In this work, the inheritance of autoregulatory transcription factors through sexual reproduction is modeled to determine the conditions under which different alternative epigenetic states become fixed in a population. The influence of transient environmental perturbations on the fixation of states is discussed in the context of speciation. (C) 1995 Academic Press Limited", 
"Odorant receptor genes reveal a new case oi monoallelic expression: just one allele of one oi the large number oi receptor genes appears to be transcribed in a given sensory neuron.", 
"Inappropriate estrogen exposure during critical periods of development will cause numerous abnormalities in the female reproductive tract. Epigenetic effects on the expression of estrogen-regulated genes is proposed to be one of the mechanisms by which neonatal estrogen elicits teratogenic and carcinogenic effects. Of note is the existence of an integral relationship between the regulation of members of the epidermal growth factor (EGF) gene family and estrogen effects on the growth and differentiation of the reproductive tract. To determine whether the EGF gene family plays a critical role in mediating the pathogenic effects of estrogen, we have used transforming growth factor-alpha (TGF alpha) transgenic mice to investigate the effects of constitutive TGF alpha expression in the reproductive tract and whether TGF alpha would potentiate carcinogenesis induced by the potent estrogen, diethylstilbestrol (DES), and by the carcinogen 7,12-dimethylbenz[a]anthracene (DMBA). The animals were homozygous TGF alpha transgenic female mice from the MT42 line and the parental CD-1 outbred mice. Constitutive TGF alpha expression was found to augment the effects of both DES and DMBA in eliciting hyperplastic and differentiation changes in the reproductive tract. The presence of the TGF alpha transgene significantly increased the incidence of DES-induced vaginal adenosis, uterine endometrial hyperplasia, hypospadia, and benign ovarian cysts. In addtion, TGF alpha potentiated the effects of DMBA in eliciting uterine polyps and benign ovarian cysts, and in the retention of Wolffian Duct remnants. However, the incidence of reproductive tract neoplasia was not promoted by the presence of the TGF alpha. transgene. This study indicates that TGF alpha plays a role in the developmental and morphogenic events of both the Mullerian duct and urogenital sinus, and that deregulation is associated with pathogenesis of these tissues. Furthermore, the fact that constitutive expression of the TGF alpha did not substitute for DES as a reproductive tract carcinogen or act as a promoter of DES-induced uterine neoplasia suggest that DES carcinogenesis involves more than aberrant expression of this single growth factor.", 
"The course and outcome of biological evolution is strongly influenced by constraints. They, flanking the work of natural selection, operate not only by limiting but also by changing the evolutionary processes. The following constraint categories are discussed and analyzed: structural developmental, phylogenetic, genetic, epigenetic, behavioural, ecological (as ecosystemic) and selective. Epigenetic, behavioural and ecosystemic constraints may be important ways through which an environmental information, supplementary to that conveyed by natural selection, can flow in the evolutionary process influencing the phenotype construction. The different constraint categories depend on three basic evolutionary forces: structure, environment and history or contingency. They form a network of reciprocal dependences and interact with natural selection. In the newly proposed two-step evolutionary model (Sara [1993]), in which two phases alternate at each generation, the constructive, organismic phase is represented by constraints (except the selective ones) while the selective, populational phase is represented by natural selection and selective constraints as drift.", 
"The meaning of hereditary information is not simple. It includes not only what a system receives and transmits but particularly what it makes. The syntactic basis to hereditary information is also not straightforward. For example, is DNA instructions, or data, or both? The answer to this question requires an appreciation of the meaning of the information yet there are a number of possible semantic systems for describing hereditary information including proteins and development. The descriptive boundaries of hereditary information are examined by locating some general organising themes including hierarchy; ecology, regulation, epigenetic systems and talkback. Though metaphors have limits in terms of their explanatory power, a number have influenced the development of biological thinking and biosystems have variously been represented as chemical laboratories, computers, electromechanical machines and societies. In this article a further metaphor is discussed, that of life-as-a-play or dance in which the trio of script (genome), cast (metabolism) and stage (cellular structure) co-exist and pre-exist the phenotypic life history which inherits them. A fuller examination of this trio provides an important perspective on the study of the organisations of information processing in hereditary systems. Copyright (C) 1997 Elsevier Science Ireland.", 
" ", 
"The combination of hierarchical constraints and compartmentalization provides a material basis for cells to 1) modulate the magnitude and duration of genome self-modification and 2) to restrict induced variation to defined regions of the genome. The hypothesis is presented that long-range coherence between genomic self-modification events and cytoplasmic activities can regulate and even divert genomic reorganizations. Finally it is proposed that two formal causes acr in phylogeny, one being the genome which encodes an ontogeny and the second being the organismal epigenetic attractors which act as boundary conditions on permissible genomic change. Genome self-modification can thus be viewed as a means for early epigenesis to modify genomic programs.", 
"In two Artemia wild-type sibling species originating from the Old and New Worlds, the processes underlying the construction of the naupliar body during embryonic development and the construction of the adult body during postembryonic development are disrupted by specific nutritional deficiencies and/or the administration of metabolic inhibitors. The species-specific phenotypic outcomes of these experimental disruptions on the construction of segments and the establishment of their identity, permit us to outline a model in which Hox genes would act as intermediary cogwheels fastened to a mechanism put in gear upstream by purine-mediated processes which would trigger downstream folic acid-mediated processes. The prevalent view that Hox genes can select for different developmental programmes, is challenged by this model whose relevancy is analysed in the context of our present knowledge on the master functions ascribed to Hox genes in developmental and evolutionary processes, ((C) Academie des sciences / Elsevier, Paris.)", 
"The objective of this study was to gather insights and compare the mode of action of the non phorbol, diterpene mezerein with the phorbol ester, phorbol-12-myristate-13 acetate, in normal and transformed cells. Both phorbol-12-myristate-13 acetate and mezerein are shown to activate the signal transduction pathways involving post translational modification of proteins by poly ADP-ribosylation and by protein kinase C, but to varying extents and showed different time kinetics and cell type differences, Multiple nuclear proteins, especially histones H3d, A24 and H1 served as accepters of poly ADP-ribose in response to PMA in both NIH 3T3 and HDCS cells whereas H1 and H2B were the major accepters in case of mezerein treatment, similarly in both NIH 3T3 and HDCS cells. The results suggest an epigenetic mechanism(s) in tumour promotion by mezerein.", 
"The molecular cell sciences have had a great impact in the analysis of the genetic and epigenetic events of esophageal and gastric tumorigenesis. In other regions of the alimentary tract such as the colon, the serial identification of the molecular events in the corresponding morphological lesions is perhaps most advanced. This is, in part, due to the relative ease of the histological characterisation of the premalignant lesions. In this regard the analysis of morphological and molecular adaptation in the alimentary tract is inextricable. This review aims, therefore, to judiciously assess the relative applications of contemporary techniques in investigative histopathology. (C) 1995 Wiley-Liss, Inc.", 
" ", 
" ", 
" ", 
" ", 
" ", 
"In order to understand the complex ontogenetical processes, the development of Streptomyces (S.) griseus was applied as a model. The developmental cycle of S. griseus starts and ends as a conidium. In between, coenocytic mycelium develops which, if studied by cytomorphological of biochemical methods, exhibits conspicious heterogeneity. The hyphae develop into young, transient and old vegetative hyphae and different stages of reproductive forms. In developmentally blocked mutants these sequences of events appear mixed in all possible associations. It seems as if the program of development could be divided into several subprograms. The quantitative evaluation of the results show that the individual morphological markers exhibit certain independence from each other realized with a given probability. The conidia of S. griseus are also heterogeneous concerning all morphological and physiological traits examined so far (shape, size, light refraction, staining and shape of nucleoids with Feulgen, methyl green - pyronine, intensity and form of polysaccharide distribution, heat resistance, etc.). Kinetics of the survival curves of two S. griseus strains - a well-sporulating and its developmentally blocked nutant /24/ - are different from each other, one has many more heat resistant conidia than the other but the kinetics of the survival curves of the two S. griseus strains indicate that spore populations of both react differently to heat treatment and heat resistance can be modeled by assuming the presence of two independent subpopulations of spores with different heat sensitivity. The emergence of two distinct subpopulations with (possibly) the same genetic make-up is designated: phenotypic segregation. Heat resistance is first of all species specific (genetically determined) but the epigenetic segregation seems to be characteristic of the developmental process. This process can in certain mutants be affected by environmental conditions and more importantly by the so-called autoregulators (A-factor and factor C). Factor C and A-factor are needed to normal development, if their quantity or the time of addition to the culture was not optimal, the quantity of spares decreased.", 
" ", 
"Massive cell migration, proliferation, phenotypic differentiation, and enhanced biosynthetic activities characterize the sites of wound healing and fibrosis. Regulation of cellular functions by extracellular matrix, which consists of a dynamic assemblage of a variety of interacting molecules capable of reorganization in response to endogenous and exogenous stimuli, represents a fundamental epigenetic mechanism regulating cellular behavior and phenotype. Interactions of the individual components of extracellular matrix with specific cell surface molecules, integrin receptors, and proteoglycans initiate a cascade of signal transduction leading to varied short-term or persistent cellular responses. Extracellular matrix also serves as an important reservoir of cytokines and growth factors, thus modulating the action of a host of potent biological response modifiers by their selective, local accumulation and release. Currently known mechanisms by which extracellular matrix modulates different facets of the process of tissue remodeling after injury, which culminate either in normal wound repair or fibrosis, are discussed.", 
"Fluctuating asymmetry is an epigenetic measure of the ability of individuals to undergo identical development of a usually bilaterally symmetrical character on both sides of the body. Horns of beetles and spurs of birds are elaborate structures usually restricted to or enlarged in individuals of the limited sex and frequently used in intrasexual combat. Individuals with large horns usually win fights against individuals with small horns which suggests that horn size may reliable signal male fighting ability. A comparative study of the patterns of fluctuating asymmetry in horns of beetles and spurs of birds showed that the degree of asymmetry in these secondary sex traits was considerably larger than in elytra and wing length. which are apparently not subjected to sexual selection. There was a negative relationship between the degree of asymmetry and size of the secondary sex trait while the relationship for elytra and wing length was flat or U-shaped. This demonstrates that horn and spur size may reliable reflect male quality as determined by the degree of fluctuating asymmetry.", 
" ", 
" ", 
"Fundamental, yet unresolved, issues in developmental neurobiology concern the relative influence of genetic vs. epigenetic factors in determining cell phenotype and establishing positional relationships among clonally related cells in the developing and mature vertebrate central nervous system (CNS). Advances in our understanding of how cells acquire their identity awaited a means to introduce lineage tracers into dividing cells of the developing CNS where the progenitor cells, which are situated in a neuroepithelial layer adjacent to the ventricles, generally are small and inaccessible. The technique of retroviral-mediated gene transfer, whereby a heritable, easily detectable marker such as the gene for lacZ is integrated into the DNA of individual progenitor cells, has been used to analyze the lineage relationships of cells in the CNS and to derive the types of progenitor cells in the proliferative zones at different developmental stages. Collectively, these studies indicate that the CNS uses more than one strategy to achieve cell diversity. The analysis of the phenotypic composition of the cells within a clone indicates that there are predominantly separate progenitor cells for each of the main cell types comprising the cerebral cortex by the onset of cortical neurogenesis, although in other systems mostly multipotential progenitor cells persist throughout neurogenesis. Here I will compare and contrast the inferred role of cell lineage in the developing cerebral cortex and olfactory bulb, where postmitotic neurons migrate relatively long distances from their site of generation to their final destination, with that in other regions of the CNS in which the displacement of postmitotic neurons from their birthplace is significantly less.", 
"Evolutionary transitions to new levels of individuality have usually been treated as a part of the ''units of selection'' problem. It has previously been assumed that the unit of transmission and heritable variation at each level is the same-that it is the DNA base sequence and its variations. It is suggested here that considering the nature and the role of hereditary variations produced by non-DNA inheritance systems is essential for understanding some evolutionary transitions. The argument is illustrated by considering the role of epigenetic inheritance systems (EISs), the systems responsible for cellular memory, in the transition from unicellularity to multicellularity. It is argued that EISs played a vital role in the transition to multicellularity and the evolution of complex ontogenies, as well as having an important effect on the evolution of developmental strategies which protect the multicellular individual from disintegrating into its component parts. An analogy between the transition to multicellularity and the transition to a cultural unit integrated by language is also suggested.", 
" ", 
"Germinally transmissible variants that arise by the anomalous imposition of developmental information on the genome are not uncommon in plant genetics, although they are often ignored. Better understanding of such variants is believed to be important because they appear to reflect basic features of developmental control processes. This paper briefly reviews classical genetic studies of such variants in plants, then discusses recent work on the genetic behaviour of plant transgenes, the results of which parallel and extend the classical genetic studies of these phenomena. An attempt is made to explain the molecular basis of these phenomena in terms of modern hypotheses on the dynamic organization of chromatin.", 
"Variation in the branching morphologies of clonal plants, fungi, and sessile marine invertebrates is frequently correlated with a suite of life-history traits (e.g. 'phalanx' or 'guerilla'). These correlations have been interpreted to be the causal product of selection. A tacit assumption of selection on a trait is that development is canalized in the manner Waddington originally suggested for aclonal taxa, i.e. small perturbations in development result in a return to an equilibrial morphology. We tested this assumption by manipulating developing colonies of the hydroid Hydractinia echinata. The growth trajectory of these colonies follows a clone-specific schedule of production of three structures: feeding polyps, stolonal mat, and peripheral stolons. Isogeneic manipulations of the relative frequency of these structures show that developing colonies can regulate the rate of production of these three structures, but that regulation does not result in rapid convergence on a common growth trajectory.", 
"In an earlier work a model of the autocrine and paracrine pathways of tumor growth control was developed (Michelson and Leith. 1991. Autocrine and paracrine growth factors in tumor growth. Bull. math. Biol. 53, 639-656). The target population, a generic tumor, was modeled as a single, homogeneous population using the standard Verhulst equation of logistic growth. Mitogenic signals were represented by modifications to the Malthusian growth parameter and adaptational signals were represented by modifications to the carrying capacity. Three growth scenarios were described: (1) normal tissue wound healing, (2) unperturbed tumor growth, and (3) tumor growth in a radiation damaged environment, a phenomenon termed the Tumor Bed Effect (TBE). In this paper, we extend those results to include a ''triad'' of growth factor controls (autocrine, paracrine and endocrine) and heterogeneity of the target population. The heterogeneous factors in the model represent either intrinsic, epigenetic or environmental differences in both normally differentiating tissues and tumors. Three types of growth are modeled: (1) normal tissue differentiation or wound healing, assuming no communication between differentiated and undifferentiated cell compartments; (2) normal wound healing with feedback inhibition, due to signalling from the differentiated compartment; and (3) the development of hypoxia in a spherical tumor. The signal processing within the triad is discussed for each model and biologically reasonable constraints are defined for limits on growth control.", 
" ", 
"Epigenetic information, not DNA sequence, distinguishes different cell types within a multicellular organism. The molecular nature of epigenetic information specifying differentiated cell types is unknown. However, it must be stable over time and mitotically heritable. In this paper, steady-state levels of autoregulatory transcription factors are explored as a potential form of epigenetic information. In particular, a model autoregulatory transcription factor having two alternative stable steady-state levels is presented. Each steady-state level specifies a phenotypically distinguishable epigenetic state, analogous to a differentiated cell type. These states are mitotically stable if the transcription factor is inherited during cell division.", 
"Parental genomes in mammals are programmed in the germline with heritable epigenetic modifications that exert control on the expression of specific (imprinted) genes. DNA methylation is one form of epigenetic modification which shows marked genome-wide variations in the germline and during early development. Certain transgene loci also demonstrate (reversible) germline-specific methylation imprints that are heritable in somatic tissues during development. Recently, four endogenous genes have been identified whose expression is dependent on their parental origin. The mechanism of genomic imprinting and the role of imprinted genes during development is beginning to be analysed. Three of these genes map to the mouse chromosome 7. Human chromosomes 11p13, 11p15, and 15q11-13 are associated with disorders exhibiting parental origin effects in their patterns of inheritance. These regions share syntenic homology with mouse chromosome 7. The relationship between parental imprints, germline-dependent epigenetic inheritance and totipotency is also under investigation using embryonic stem cells derived from the epiblast. These cells are pluripotent or totipotent and evidence indicates the presence of at least the primary parental imprints. However, imprints inherited from the paternal germline in androgenetic cells are apparently more stable than those from the female germline in parthenogenetic cells.", 
"In Armadillidium vulgare, genetic variability at the mt DNA level has been studied to explain the nature of the mixed populations composed of genetic females (broods with sex ratio 1:1) and neo-females (female-biased broods). As a general rule, within a population, the restriction fragment patterns are well separated between the two categories of females. However, among populations, the same mitotype may correspond to both categories.", 
" ", 
"Halictine bees dig their nests into the earth. These bees include a few genus and numerous species which differ by ethological characteristics, some of them having reached a social way of life. They were studied both in the field and in large laboratory cages. In this first part, a summary of the biological cycle, the main problems treated and the results obtained are given for each of the studied species, classified from solitary to social ones. Evylaeus pallens is a solitary species with one annual exit of both sexes. Lasioglossum leucozonium is a solitary species with only one generation but two annual exits of females. Evylaeus villosulus is a solitary species with two generations and three annual exits of females. The following points are tackled: polymorphism of overwintering females; comparison, using eight characteristics, of the females in the two successive generations; sex ratio; longevity of overwintering females; cohabitation of several females in the same nest; biology of the summer generation. The size difference between the females of the two generations (7,4 per 100) and the low percentage of males produced in the first generation (21 per 100) might be regarded as preadaptations to a separation into castes. Halictus scabiosae is an often polygynous eusocial species with a poorly defined social polymorphism. Evylaeus calceatus, E. nigripes and E. albipes are three sibling eusocial species with two distinct broods; the first one is composed of workers (with more or less males), the second one of sexuals: future foundresses and big males. In E. calceatus, the following problems were dealt with : a) addition of pollen into the first brood cells: the volume of the pollen ball influences the sex of the egg laid on it: a weight threshold of sex determination can be brought out; b) large female individuals with foundress size but worker function are thus obtained: a dissociation between; size and caste is observed but no weight threshold of caste determination can be found; c) influence of temperature on the size of workers: the higher the temperature, the smaller the workers; d) polygny; e) replacement of the queen, before of after the emergence of the workers. E. nigripes produces less males in the first brood and a dissociation takes place between the provisioning of the cells and the egg-laying by the queen. According to populations, E. albipes shows a solitary form or a social form; the social level, in the latter, looks lower than in E. calceatus. Three grouped worker pupae evolved in a trigynous society, composed of an egg-layer and two pollen collectors and produced a lot of female offspring (73 per 100). Evylaeus marginatus is, so far, the only perenial known social species. Each foundress produces, year after year, in the same season, more and more workers; then, on the sixth year, she gives birth to sexuals: males and future foundresses in large numbers (> 500). The main problems dealt with in that species are social polymorphism, epigenetical determinism of the castes, analysis of the factors of socialization. The study of the biological features of these various species leaves us under the impression of a great diversity.", 
"A non-linear reaction diffusion model of a negative feedback epigenetic control system is presented. The model involves synthesis of the mitotic inducing and inhibiting proteins, simultaneously with intercellular self-diffusion and cross-diffusion of the latter only. The importance of negative cross-diffusion for creating a regular dissipative structure is shown. A bifurcation analysis of the non-linear diffusive system has been performed and it is concluded that bifurcation is supercritical. Lastly, using Liapunov's direct method, it is shown that the pattern evolved by the system is globally asymptotically stable.", 
" ", 
"Plant reproduction comprises an interlocking array of developmental pathways which include the formation of the sexual organs, the generation of germ lines de novo, and the operation of the mechanisms which regulate epigenetic imprinting and the system of self-incompatibility found in many angiosperms. Little is known of how these processes are regulated at a molecular level, with the exception of the floral organs which are determined by families of homeotic genes operating in a heterochronic fashion. In dioecious and monoecious plants the expression of these 'floral' genes must be modulated by sex-determination sequences, situated in some circumstances on sex chromosomes. Older, physiological data indicate that sex can be determined by growth regulators, particularly gibberellic acid (GA) and cytokinins, and it is possible that sex-determination genes establish local concentrations of growth regulators at the apex, which in turn influence the expression of the homeotic floral genes. Evidence from anther development indicates genes involved in differentiation of the male and female germ lines to be regulated by defined promoter, enhancer, and silencer regions, but few data are available on the sequences directing the initiation and regulation of meiosis; certainly parallels can be drawn with similar events in microorganisms, and useful complementation strategies may be devised, but significant differences do exist between yeasts and higher plants suggesting that more appropriate parallels should be drawn with multicellular eukaryotes such as nematodes. The loci involved in epigenetic imprinting and self-incompatibility are important because they affect both male and female developmental pathways. Nothing is known of the regulatory sequences which direct the epigenetic imprinting of the sperm and central cell genomes, but information is becoming available on the promoter regions of the S(incompatibility)-locus. Interestingly, sequences directing expression in male and female tissues are contained within a single 5' stretch within the locus, and these promoters also induce expression in different cell types in the anther and pistil depending on the type of self-incompatibility involved. Regulation of reproductive development in plants is apparently not very stringent, for there are examples in both male and female germ lines of reversion to an embryonic condition (apomixis and microspore embryogenesis); whether this reflects the highly dedifferentiated state of these cells or differences in the regulation of somatic and reproductive development remains to be determined.", 
"A non-linear reaction-diffusion model of the epigenetic system involving the synthesis and simultaneous intercellular self and cross-diffusion of the activator and inhibitor during embryogenesis has been proposed. It has been observed that the system spontaneously emerges into a stationary dissipative (Turing) structure only in the presence of positive cross-diffusion. That the stationary structure, thus envolved, has been shown to be globally asymptotically stable by using a suitable Liapunov's function.", 
"Teratocarcinomas are malignant tumors derived from germ cells or early embryonic cells that have undergone malignant transformation. To understand better the histogenesis of teratocarcinomas and related germ cell tumors, the process of tumorigenesis and the nature of tumor stem cells were studied in murine tumor models. Teratocarcinomas were produced by grafting early pregastrulation mouse embryos to extrauterine sites of syngeneic adult hosts. In the adult host many transplanted cells retained their embryonic phenotype. The adult organism could not control proliferation of these ostensibly normal embryonic cells which assumed the growth properties of malignant tumor cells. This malignancy developed without viral or chemical carcinogenesis and did not entail any genetic transformation. Retinoic acid and related morphogens induced differentiation and inhibited proliferation of teratocarcinoma stem cells. Likewise, teratocarcinoma stem cells inserted into the embryo were regulated by the developmental fields in control of proliferation of normal embryonic cells. These data show that in these cells malignancy can develop without irreversible genetic changes, that the malignant phenotype is reversible, and that proliferation of teratocarcinoma stem cells can be controlled by developmental fields regulating differentiation of equivalent normal cells. Murine teratocarcinoma derived from embryos is a prototype model for studies of epigenetic control of malignancy.", 
"Three types of phenotypic characters are recognized, which evolve without direct action of the selection but, however, could not be treated as selectively neutral (in commonly used meaning of the term). The first are characters with wide norm of reaction, which phenotypic expressions reproducibility depends on the information forthcoming from the environment. This includes ecological and ethological interactions between individuals: for instance, characters of the social component of the inheritance. The second are characters unreproducible at epigenetic level but having their own mechanisms of genetic reproduction. Self-replicating organelles serve as an example of such characters. The third, and commonest, set contains characters, which are reproducible only due to interaction of other phenotypic characters. Such are, for instance, morphogenetic interrelations causing complication of the spatial organization. Episelective evolution occurs as an indirect effect of independent selection of the characters, which are able to interact but are not united by either functional of morphogenetic correlations. It is supposed that essential portion of evolutionary phenomena usually attributed to the direct or correlative selective effects, emerges factually at the episelective level. The very possibility of episelective evolution is caused by relative independency of genetic and ontogenetic mechanisms of the reproduction of phenotypic characters.", 
"An intrathymic (i.t.) injection assay on B10.Thy-1 congenic mice was used to demonstrate that thymic prelymphoma cells developed first within mouse thymus 4 to 8 days after split-dose irradiation and were present in more than 63% of the test donor mice thymuses examined 21 and 31 days after irradiation. For the characterization of these thymic prelymphoma cells, thymocytes from BIO.Thy-1.1 mice sampled 1 mo after irradiation were stained with J11d mAb and mAb against TL-2 (thymus-leukemia) antigen which is not expressed on normal thymocytes of the BIO.Thy 1.2 and BIO.Thy 1.1 strains but does appear on thymocytes of split-dose irradiated mice. These cells were sorted into subpopulations, samples of which were injected into recipient thymuses to determine which subpopulations contained thymic prelymphoma cells. Results showed that thc prelymphoma cells were located in the J11d+TL-2+ cells. These prelymphoma cells were further characterized phenotypically as to their expression of the CD4 and CD8 antigens, which demonstrated that the thymic prelymphoma cells were present in the CD4-CD8- and CD4-CD8+ thymocyte subpopulations mainly and in the CD4+CD8+ subpopulation. The experiments on i.t. injection of a graded quantity of TL-2+ thymocytes from individual mice suggest that not all TL-2+ cells undergo neoplastic initiation and that prelymphoma cells may develop infrequently from one or more TL-2+ cells by genetic or epigenetic changes.", 
"Dose-response curves for chronic leukemia in A-bomb survivors and liver tumors in patients given Thorotrast (colloidal thorium dioxide) show large threshold effects. The existence of these threshold effects can be explained by the following hypothesis. A high dose of radiation causes a persistent wound in a cell-renewable tissue. Disorder of the injured cell society partly frees the component cells from territorial restraints on their proliferation, enabling them to continue development of their cellular functions toward advanced autonomy. This progression might be achieved by continued epigenetic and genetic changes as a result of occasional errors in the otherwise concerted healing action of various endogenous factors recruited for tissue repair. Carcinogenesis is not simply a single-cell problem but a cell-society problem. Therefore, it is not warranted to estimate risk at low doses by linear extrapolation from cancer data at high doses without knowledge of the mechanism of radiation carcinogenesis.", 
" ", 
"Several experimental results led to the hypothesis that odontoblast's and ameloblast's genetic programmes may determine a minimal number of cell cycles before overt differentiation can be triggered by specific, epigenetic, matrix-mediated interactions. The cell kinetic dependent maturation of the dental cells could imply transcriptional modifications of matrix molecules and paracrine factors involved in the control of the respective cytodifferentiations and of specific receptors confering competence to the responding cells.", 
" ", 
"The strategy of the evolutionary synthesis outlined in the works of I.I.Schmalhausen distinctly differs from the strategy adopted by the Synthetic Theory of Evolution. Schmalhausen's interpretation is mostly connected with the transitions from genetical to epigenetical levels, with the role of morphogenesis, with stabilizing selection and with mobility and stability in evolution. These problems are considered from the angle of the dynamic stability principle, which may be regarded as a universal evolutionary principle (VOROBYEVA, 1987; 1988; 1989).",
]


In [35]:
###df = ["Individual animals frequently exhibit repeatable differences from other members of their population, differences now commonly referred to as 'animal personality'. Personality differences can arise, for example, from differences in permanent environmental effects-including parental and epigenetic contributors- and the effect of additive genetic variation. Although several studies have evaluated the heritability of behaviour, less is known about general patterns of heritability and additive genetic variation in animal personality. As overall variation in behaviour includes both the among-individual differences that reflect different personalities and temporary environmental effects, it is possible for personality to be largely genetically influenced even when heritability of behaviour per se is quite low. The relative contribution of additive genetic variation to personality variation can be estimated whenever both repeatability and heritability are estimated for the same data. Using published estimates to address this issue, we found that approximately 52% of animal personality variation was attributable to additive genetic variation. Thus, while the heritability of behaviour is often moderate or low, the heritability of personality is much higher. Our results therefore (i) demonstrate that genetic differences are likely to be a major contributor to variation in animal personality and (ii) support the phenotypic gambit: that evolutionary inferences drawn from repeatability estimates may often be justified.", "Environment has a critical role in the natural selection process for Darwinian evolution. The primary molecular component currently considered for neo-Darwinian evolution involves genetic alterations and random mutations that generate the phenotypic variation required for natural selection to act. The vast majority of environmental factors cannot directly alter DNA sequence. Epigenetic mechanisms directly regulate genetic processes and can be dramatically altered by environmental factors. Therefore, environmental epigenetics provides a molecular mechanism to directly alter phenotypic variation generationally. Lamarck proposed in 1802 the concept that environment can directly alter phenotype in a heritable manner. Environmental epigenetics and epigenetic transgenerational inheritance provide molecular mechanisms for this process. Therefore, environment can on a molecular level influence the phenotypic variation directly. The ability of environmental epigenetics to alter phenotypic and genotypic variation directly can significantly impact natural selection. Neo-Lamarckian concept can facilitate neo-Darwinian evolution. A unified theory of evolution is presented to describe the integration of environmental epigenetic and genetic aspects of evolution.", "DNA methylation contributes to gene and transcriptional regulation in eukaryotes, and therefore has been hypothesized to facilitate the evolution of plastic traits such as sociality in insects. However, DNA methylation is sparsely studied in insects. Therefore, we documented patterns of DNA methylation across a wide diversity of insects. We predicted that underlying enzymatic machinery is concordant with patterns of DNA methylation. Finally, given the suggestion that DNA methylation facilitated social evolution in Hymenoptera, we tested the hypothesis that the DNA methylation system will be associated with presence/absence of sociality among other insect orders. We found DNA methylation to be widespread, detected in all orders examined except Diptera (flies). Whole genome bisulfite sequencing showed that orders differed in levels of DNA methylation. Hymenopteran (ants, bees, wasps and sawflies) had some of the lowest levels, including several potential losses. Blattodea (cockroaches and termites) show all possible patterns, including a potential loss of DNA methylation in a eusocial species whereas solitary species had the highest levels. Species with DNA methylation do not always possess the typical enzymatic machinery. We identified a gene duplication event in the maintenance DNA methyltransferase 1 (DNMT1) that is shared by some Hymenoptera, and paralogs have experienced divergent, nonneutral evolution. This diversity and nonneutral evolution of underlying machinery suggests alternative DNA methylation pathways may exist. Phylogenetically corrected comparisons revealed no evidence that supports evolutionary association between sociality and DNA methylation. Future functional studies will be required to advance our understanding of DNA methylation in insects.""", "Phenotypic characteristics of animals can change independently from changes in the genetic code. These plastic phenotypic responses are important for population persistence in changing environments. Plasticity can be induced during early development, with persistent effects on adult phenotypes, and it can occur reversibly throughout life (acclimation). These manifestations of plasticity have been viewed as separate processes. Here we argue that developmental conditions not only change mean trait values but also modify the capacity for acclimation. Acclimation counteracts the potentially negative effects of phenotype-environment mismatches resulting from epigenetic modifications during early development. Developmental plasticity is therefore also beneficial when environmental conditions change within generations. Hence, the evolution of reversible acclimation can no longer be viewed as independent from developmental processes.", "An organism's phenotype can be influenced by maternal cues and directly perceived environmental cues, as well as by its genotype at polymorphic loci, which can be interpreted as a genetic cue. In fluctuating environments, natural selection favors organisms that efficiently integrate different sources of information about the likely success of phenotypic alternatives. In such situations, it can be beneficial to pass on maternal cues that offspring can respond to. A maternal cue could be based on environmental cues directly perceived by the mother but also partly on cues that were passed on by the grandmother. We have used a mathematical model to investigate how the passing of maternal cues and the integration of different sources of information evolve in response to qualitatively different kinds of temporal and spatial environmental fluctuations. The model shows that the passing of maternal cues and the transgenerational integration of sources of information readily evolve. Factors such as the degree of temporal autocorrelation, the predictive accuracy of different environmental cues, and the level of gene flow strongly influence the expression of adaptive maternal cues and the relative weights given to different sources of information. We outline the main features of the relation between the characteristics of environmental fluctuations and the adaptive systems of phenotype determination and compare these predictions with empirical studies on cue integration.", "Understanding how organisms cope with global change is a major scientific challenge. The molecular pathways underlying rapid adaptive phenotypic responses to global change remain poorly understood. Here, we highlight the relevance of two environment-sensitive molecular elements: transposable elements (TEs) and epigenetic components (ECs). We first outline the sensitivity of these elements to global change stressors and review how they interact with each other. We then propose an integrative molecular engine coupling TEs and ECs and allowing organisms to fine-tune phenotypes in a real-time fashion, adjust the production of phenotypic and genetic variation, and produce heritable phenotypes with different levels of transmission fidelity. We finally discuss the implications of this molecular engine in the context of global change.", "Existing insight suggests that maternal effects have a substantial impact on evolution, yet these predictions assume that maternal effects themselves are evolutionarily constant. Hence, it is poorly understood how natural selection shapes maternal effects in different ecological circumstances. To overcome this, the current study derives an evolutionary model of maternal effects in a quantitative genetics context. In constant environments, we show that maternal effects evolve to slight negative values that result in a reduction of the phenotypic variance (canalization). By contrast, in populations experiencing abrupt change, maternal effects transiently evolve to positive values for many generations, facilitating the transmission of beneficial maternal phenotypes to offspring. In periodically fluctuating environments, maternal effects evolve according to the autocorrelation between maternal and offspring environments, favoring positive maternal effects when change is slow, and negative maternal effects when change is rapid. Generally, the strongest maternal effects occur for traits that experience very strong selection and for which plasticity is severely constrained. By contrast, for traits experiencing weak selection, phenotypic plasticity enhances the evolutionary scope of maternal effects, although maternal effects attain much smaller values throughout. As weak selection is common, finding substantial maternal influences on offspring phenotypes may be more challenging than anticipated.", "As climate change challenges organismal fitness by creating a phenotype-environment mismatch, phenotypic plasticity generated by epigenetic mechanisms (e.g., DNA methylation) can provide a temporal buffer for genetic adaptation. Epigenetic mechanisms may be crucial for sessile benthic marine organisms, such as reef-building corals, where ocean acidification (OA) and warming reflect in strong negative responses. We tested the potential for scleractinian corals to exhibit phenotypic plasticity associated with a change in DNA methylation in response to OA. Clonal coral fragments of the environmentally sensitive Pocillopora damicornis and more environmentally robust Montipora capitata were exposed to fluctuating ambient pH (7.9-7.65) and low pH (7.6-7.35) conditions in common garden tanks for similar to 6 weeks. M. capitata responded weakly, or acclimated more quickly, to OA, with no difference in calcification, minimal separation of metabolomic profiles, and no change in DNA methylation between treatments. Conversely, P. damicornis exhibited diminished calcification at low pH, stronger separation in metabolomic profiles, and responsiveness of DNA methylation to treatment. Our data suggest corals differ in their temporal dynamics and sensitivity for environmentally triggered real-time epigenetic reprogramming. The generation of potentially heritable plasticity via environmental induction of DNA methylation provides an avenue for assisted evolution applications in corals under rapid climate change.", "Migration is essential for the reproduction and survival of many animals, yet little is understood about its underlying molecular mechanisms. We used the salmonid Oncorhynchus mykiss to gain mechanistic insight into smoltification, which is a morphological, physiological and behavioural transition undertaken by juveniles in preparation for seaward migration. O. mykiss is experimentally tractable and displays intra- and interpopulation variation in migration propensity. Migratory individuals can produce nonmigratory progeny and vice versa, indicating a high degree of phenotypic plasticity. One potential way that phenotypic plasticity might be linked to variation in migration-related life history tactics is through epigenetic regulation of gene expression. To explore this, we quantitatively measured genome-scale DNA methylation in fin tissue using reduced representation bisulphite sequencing of F-2 siblings produced from a cross between steelhead (migratory) and rainbow trout (nonmigratory) lines. We identified 57 differentially methylated regions (DMRs) between smolt and resident O. mykiss juveniles. DMRs were high in magnitude, with up to 62% differential methylation between life history types, and over half of the gene-associated DMRs were in transcriptional regulatory regions. Many of the DMRs encode proteins with activity relevant to migration-related transitions (e.g. circadian rhythm pathway, nervous system development, protein kinase activity). This study provides the first evidence of a relationship between epigenetic variation and life history divergence associated with migration-related traits in any species.", "While traits and trait plasticity are partly genetically based, investigating epigenetic mechanisms may provide more nuanced understanding of the mechanisms underlying response to environment. Using AFLP and methylation-sensitive AFLP, we tested the hypothesis that differentiation to habitats along natural salt marsh environmental gradients occurs at epigenetic, but not genetic loci in two salt marsh perennials. We detected significant genetic and epigenetic structure among populations and among subpopulations, but we found multilocus patterns of differentiation to habitat type only in epigenetic variation for both species. In addition, more epigenetic than genetic loci were correlated with habitat in both species. When we analysed genetic and epigenetic variation simultaneously with partial Mantel, we found no correlation between genetic variation and habitat and a significant correlation between epigenetic variation and habitat in Spartina alterniflora. In Borrichia frutescens, we found significant correlations between epigenetic and/or genetic variation and habitat in four of five populations when populations were analysed individually, but there was no significant correlation between genetic or epigenetic variation and habitat when analysed jointly across the five populations. These analyses suggest that epigenetic mechanisms are involved in the response to salt marsh habitats, but also that the relationships among genetic and epigenetic variation and habitat vary by species. Site-specific conditions may also cloud our ability to detect response in replicate populations with similar environmental gradients. Future studies analysing sequence data and the correlation between genetic variation and DNA methylation will be powerful to identify the contributions of genetic and epigenetic response to environmental gradients.", "Epigenetic variation is being integrated into our understanding of adaptation, yet we lack models on how epigenetic mutations affect evolution that includes de novo genetic change. We model the effects of epigenetic mutations on the dynamics and endpoints of adaptive walks-a process where a series of beneficial mutations move a population towards a fitness optimum. We use an individual-based model of an asexual population, where mutational effects are drawn from Fisher's geometric model. We find cases where epigenetic mutations speed adaptation or result in populations with higher fitness. However, we also find cases where they slow adaptation or result in populations with lower fitness. The effect of epigenetic mutations on adaptive walks depends crucially on their stability and fitness effects relative to genetic mutations, with small-effect epigenetic mutations generally speeding adaptation, and epigenetic mutations with the same fitness effects as genetic mutations slowing adaptation. Our work reveals a complex relationship between epigenetic mutations and natural selection and highlights the need for empirical data.", "Environmental stresses experienced by individual parents can influence offspring phenotypes in ways that enhance survival under similar conditions. Although such adaptive transgenerational plasticity is well documented, its transmission mechanisms are generally unknown. One possible mechanism is environmentally induced DNA methylation changes. We tested this hypothesis in the annual plant Polygonum persicaria, a species known to express adaptive transgenerational plasticity in response to parental drought stress. Replicate plants of 12 genetic lines (sampled from natural populations) were grown in dry versus moist soil. Their offspring were exposed to the demethylating agent zebularine or to control conditions during germination and then grown in dry soil. Under control germination conditions, the offspring of drought-stressed parents grew longer root systems and attained greater biomass compared with offspring of well-watered parents of the same genetic lines. Demethylation removed these adaptive developmental effects of parental drought, but did not significantly alter phenotypic expression in offspring of well-watered parents. The effect of demethylation on the expression of the parental drought effect varied among genetic lines. Differential seed provisioning did not contribute to the effect of parental drought on offspring phenotypes. These results demonstrate that DNA methylation can mediate adaptive, genotype-specific effects of parental stress on offspring phenotypes.", "The role of natural selection in the evolution of adaptive phenotypes has undergone constant probing by evolutionary biologists, employing both theoretical and empirical approaches. As Darwin noted, natural selection can act together with other processes, including random changes in the frequencies of phenotypic differences that are not under strong selection, and changes in the environment, which may reflect evolutionary changes in the organisms themselves. As understanding of genetics developed after 1900, the new genetic discoveries were incorporated into evolutionary biology. The resulting general principles were summarized by Julian Huxley in his 1942 book Evolution: the modern synthesis. Here, we examine how recent advances in genetics, developmental biology and molecular biology, including epigenetics, relate to today's understanding of the evolution of adaptations. We illustrate how careful genetic studies have repeatedly shown that apparently puzzling results in a wide diversity of organisms involve processes that are consistent with neo-Darwinism. They do not support important roles in adaptation for processes such as directed mutation or the inheritance of acquired characters, and therefore no radical revision of our understanding of the mechanism of adaptive evolution is needed.", "The process of biological invasion exposes a species to novel pressures, in terms of both the environments it encounters and the evolutionary consequences of range expansion. Several invaders have been shown to exhibit rapid evolutionary changes in response to those pressures, thus providing robust opportunities to clarify the processes at work during rapid phenotypic transitions. The accelerating pace of invasion of cane toads (Rhinella marina) in tropical Australia during its 80-year history has been well characterized at the phenotypic level, including common-garden experiments that demonstrate heritability of several dispersal-relevant traits. Individuals from the invasion front (and their progeny) show distinctive changes in morphology, physiology and behaviour that, in combination, result in far more rapid dispersal than is true of conspecifics from long-colonized areas. The extensive body of work on cane toad ecology enables us to place into context studies of the genetic basis of these traits. Our analyses of differential gene expression from toads from both ends of this invasion-history transect reveal substantial upregulation of many genes, notably those involved in metabolism and cellular repair. Clearly, then, the dramatically rapid phenotypic evolution of cane toads in Australia has been accompanied by substantial shifts in gene expression, suggesting that this system is well suited to investigating the genetic underpinnings of invasiveness.", "DNA methylation in plants affects transposon silencing, transcriptional regulation and thus phenotypic variation. One unanswered question is whether DNA methylation could be involved in local adaptation of plant populations to their environments. If methylation alters phenotypes to improve plant response to the environment, then methylation sites or the genes that affect them could be a target of natural selection. Using reduced-representation bisulphite sequencing (RRBS) data, we assessed whether climate is associated with variation in DNA methylation levels among 58 naturally occurring, and species-wide samples of valley oak (Quercus lobata) collected across climate gradients. We identified the genomic context of these variants referencing a new draft valley oak genome sequence. Methylation data were obtained for 341 107 cytosines, of which we deemed 57 488 as single-methylation variants (SMVs), found in the CG, CHG and CHH sequence contexts. Environmental association analyses revealed 43 specific SMVs that are significantly associated with any of four climate variables, the majority of which are associated with mean maximum temperature. The 43 climate-associated SMVs tend to occur in or near genes, several of which have known involvement in plant response to environment. Multivariate analyses show that climate and spatial variables explain more overall variance in CG-SMVs among individuals than in SNPs, CHG-SMVs or CHH-SMVs. Together, these results from natural oak populations provide initial evidence for a role of CG methylation in locally adaptive evolution or plasticity in plant response.", "Comparative genomics of social insects has been intensely pursued in recent years with the goal of providing insights into the evolution of social behaviour and its underlying genomic and epigenomic basis. However, the comparative approach has been hampered by a paucity of data on some of the most informative social forms (e.g. incipiently and primitively social) and taxa (especially members of the wasp family Vespidae) for studying social evolution. Here, we provide a draft genome of the primitively eusocial model insect Polistes dominula, accompanied by analysis of caste-related transcriptome and methylome sequence data for adult queens and workers. Polistes dominula possesses a fairly typical hymenopteran genome, but shows very low genomewide GC content and some evidence of reduced genome size. We found numerous caste-related differences in gene expression, with evidence that both conserved and novel genes are related to caste differences. Most strikingly, these -omics data reveal a major reduction in one of the major epigenetic mechanisms that has been previously suggested to be important for caste differences in social insects: DNA methylation. Along with a conspicuous loss of a key gene associated with environmentally responsive DNA methylation (the de novo DNA methyltransferase Dnmt3), these wasps have greatly reduced genomewide methylation to almost zero. In addition to providing a valuable resource for comparative analysis of social insect evolution, our integrative -omics data for this important behavioural and evolutionary model system call into question the general importance of DNA methylation in caste differences and evolution in social insects.", "Epidemiological research since the 1980s has highlighted the consequences of early life adversity, particularly during gestation and early infancy, for adult health (the Barker hypothesis). The fast-evolving field of molecular epigenetics is providing explanatory mechanisms concerning phenotypic plasticity in response to developmental stressors and the accumulation of disease risk throughout life. In addition, there is now evidence for the heritability of poor health across generations via epigenetic modifications. This research has the potential to invoke a paradigmatic shift in how we interpret factors such as growth insults and immune response in past skeletal remains. It demonstrates that health cannot be understood in terms of immediate environmental circumstances alone. Furthermore, it requires both a theoretical and practical re-evaluation of disease biographies and the life course more generally. Individual life courses can no longer be regarded as discrete, bounded, life histories, with clearly defined beginning and end points. If socioeconomic circumstances can have intergenerational effects, including disease susceptibility and growth stunting, then individual biographies should be viewed as nested or embedded within the lives of others. This commingling of life courses may prove problematic to unravel; nevertheless, this review aims to consider the potential consequences for bioarchaeological interpretations. These include a greater consideration of: the temporal power of human skeletons and a life course approach to past health; infant health and the implications for maternal well-being; and the impact of non-proximate stressors (e.g., early life and ancestral environments) on the presence of health indicators. (C) 2015 Wiley Periodicals, Inc.", "Organisms develop through multiple life stages that differ in environmental tolerances. The seasonal timing, or phenology, of life-stage transitions determines the environmental conditions to which each life stage is exposed and the length of time required to complete a generation. Both environmental and genetic factors contribute to phenological variation, yet predicting their combined effect on life cycles across a geographic range remains a challenge. We linked submodels of the plasticity of individual life stages to create an integrated model that predicts life-cycle phenology in complex environments. We parameterized the model for Arabidopsis thaliana and simulated life cycles in four locations. We compared multiple genotypes by varying two parameters associated with natural genetic variation in phenology: seed dormancy and floral repression. The model predicted variation in life cycles across locations that qualitatively matches observed natural phenology. Seed dormancy had larger effects on life-cycle length than floral repression, and results suggest that a genetic cline in dormancy maintains a life-cycle length of 1 year across the geographic range of this species. By integrating across life stages, this approach demonstrates how genetic variation in one transition can influence subsequent transitions and the geographic distribution of life cycles more generally.", ]

In [36]:
###import pandas as pd

###df = pd.read_excel (r'C:/Users/brady/Desktop/TM/Evo-Bio All Time/Test1.xlsx')
###print (df)

In [37]:
print(df)

['Chronic inflammation is involved in the onset and development of many diseases, including obesity, atherosclerosis, type 2 diabetes, osteoarthritis, autoimmune and degenerative diseases, asthma, periodontitis, and cirrhosis. The inflammation process is mediated by chemokines, cytokines, and different inflammatory cells. Although the molecules and mechanisms that regulate this primary defense mechanism are not fully understood, recent findings offer a putative role of noncoding RNAs, especially microRNAs (miRNAs), in the progression and management of the inflammatory response. These noncoding RNAs are crucial for the stability and maintenance of gene expression patterns that characterize some cell types, tissues, and biologic responses. Several miRNAs, such as miR-126, miR-132, miR-146, miR-155, and miR-221, have emerged as important transcriptional regulators of some inflammation-related mediators. Additionally, little is known about the involvement of long noncoding RNAs, long inter

In [38]:
###CLEAN DATA###

In [39]:
# Convert to list
data = [simple_preprocess(doc) for doc in df]

In [40]:
print(data)

[['chronic', 'inflammation', 'is', 'involved', 'in', 'the', 'onset', 'and', 'development', 'of', 'many', 'diseases', 'including', 'obesity', 'atherosclerosis', 'type', 'diabetes', 'osteoarthritis', 'autoimmune', 'and', 'degenerative', 'diseases', 'asthma', 'periodontitis', 'and', 'cirrhosis', 'the', 'inflammation', 'process', 'is', 'mediated', 'by', 'chemokines', 'cytokines', 'and', 'different', 'inflammatory', 'cells', 'although', 'the', 'molecules', 'and', 'mechanisms', 'that', 'regulate', 'this', 'primary', 'defense', 'mechanism', 'are', 'not', 'fully', 'understood', 'recent', 'findings', 'offer', 'putative', 'role', 'of', 'noncoding', 'rnas', 'especially', 'micrornas', 'mirnas', 'in', 'the', 'progression', 'and', 'management', 'of', 'the', 'inflammatory', 'response', 'these', 'noncoding', 'rnas', 'are', 'crucial', 'for', 'the', 'stability', 'and', 'maintenance', 'of', 'gene', 'expression', 'patterns', 'that', 'characterize', 'some', 'cell', 'types', 'tissues', 'and', 'biologic', 'r

In [41]:
###MAY NEED CLEANING STEP, MAY NOT BECAUSE ABSTRACTS ARE SO CLEAN###

In [42]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:1])

[['chronic', 'inflammation', 'is', 'involved', 'in', 'the', 'onset', 'and', 'development', 'of', 'many', 'diseases', 'including', 'obesity', 'atherosclerosis', 'type', 'diabetes', 'osteoarthritis', 'autoimmune', 'and', 'degenerative', 'diseases', 'asthma', 'periodontitis', 'and', 'cirrhosis', 'the', 'inflammation', 'process', 'is', 'mediated', 'by', 'chemokines', 'cytokines', 'and', 'different', 'inflammatory', 'cells', 'although', 'the', 'molecules', 'and', 'mechanisms', 'that', 'regulate', 'this', 'primary', 'defense', 'mechanism', 'are', 'not', 'fully', 'understood', 'recent', 'findings', 'offer', 'putative', 'role', 'of', 'noncoding', 'rnas', 'especially', 'micrornas', 'mirnas', 'in', 'the', 'progression', 'and', 'management', 'of', 'the', 'inflammatory', 'response', 'these', 'noncoding', 'rnas', 'are', 'crucial', 'for', 'the', 'stability', 'and', 'maintenance', 'of', 'gene', 'expression', 'patterns', 'that', 'characterize', 'some', 'cell', 'types', 'tissues', 'and', 'biologic', 'r

In [43]:
###THIS STEP NEEDS TO BE CAREFULLY MONITORED TO MAKE SURE IT'S WORKING AS INTENDED###

In [44]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

['chronic', 'inflammation', 'is', 'involved', 'in', 'the', 'onset', 'and', 'development', 'of', 'many', 'diseases', 'including', 'obesity', 'atherosclerosis', 'type', 'diabetes', 'osteoarthritis', 'autoimmune', 'and', 'degenerative', 'diseases', 'asthma', 'periodontitis', 'and', 'cirrhosis', 'the', 'inflammation', 'process', 'is', 'mediated', 'by', 'chemokines', 'cytokines', 'and', 'different', 'inflammatory', 'cells', 'although', 'the', 'molecules', 'and', 'mechanisms', 'that', 'regulate', 'this', 'primary', 'defense', 'mechanism', 'are', 'not', 'fully', 'understood', 'recent', 'findings', 'offer', 'putative', 'role', 'of', 'noncoding_rnas', 'especially', 'micrornas', 'mirnas', 'in', 'the', 'progression', 'and', 'management', 'of', 'the', 'inflammatory', 'response', 'these', 'noncoding_rnas', 'are', 'crucial', 'for', 'the', 'stability', 'and', 'maintenance', 'of', 'gene', 'expression', 'patterns', 'that', 'characterize', 'some', 'cell', 'types', 'tissues', 'and', 'biologic', 'response

In [45]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [46]:
###en_core_web_sm needs to be explicitly loaded and I believe you need to use "python -m spacy download en" in cmd.exe to properly load everything.###

In [47]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['chronic', 'inflammation', 'involve', 'onset', 'development', 'many', 'disease', 'include', 'obesity', 'atherosclerosis', 'type', 'diabetes', 'osteoarthritis', 'autoimmune', 'degenerative', 'disease', 'cirrhosis', 'inflammation', 'process', 'mediate', 'chemokine', 'cytokine', 'different', 'inflammatory', 'cell', 'molecule', 'mechanism', 'regulate', 'primary', 'defense', 'mechanism', 'fully', 'understand', 'recent', 'finding', 'offer', 'putative', 'role', 'especially', 'microrna', 'mirna', 'progression', 'management', 'inflammatory', 'response', 'crucial', 'stability', 'maintenance', 'gene', 'expression', 'pattern', 'characterize', 'cell', 'type', 'tissue', 'biologic', 'response', 'several', 'mirna', 'emerge', 'important', 'transcriptional', 'regulator', 'inflammation', 'relate', 'mediator', 'additionally', 'little', 'know', 'involvement', 'long', 'long', 'intergenic', 'circular', 'mediate', 'process', 'homeostatic', 'imbalance', 'associate', 'disorder', 'emerge', 'biomarker', 'diagno

In [48]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 3), (27, 1), (28, 2), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 6), (44, 2), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1), (52, 2), (53, 1), (54, 1), (55, 1), (56, 2), (57, 2), (58, 2), (59, 1), (60, 3), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 1), (67, 1), (68, 1), (69, 2), (70, 1), (71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1), (77, 2), (78, 1), (79, 1), (80, 3), (81, 2), (82, 1), (83, 1), (84, 1), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 2), (92, 2), (93, 1), (94, 1)]]


In [49]:
#to call an individual dictionary term switch value as appropriate
id2word[0]

'activator'

In [50]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('activator', 1),
  ('additionally', 1),
  ('adhesion', 1),
  ('alpha', 1),
  ('alteration', 1),
  ('antagonism', 1),
  ('associate', 1),
  ('atherosclerosis', 1),
  ('autoimmune', 1),
  ('biologic', 1),
  ('biomarker', 1),
  ('cell', 2),
  ('characterize', 1),
  ('chemokine', 1),
  ('chronic', 1),
  ('circular', 1),
  ('cirrhosis', 1),
  ('context', 1),
  ('crucial', 1),
  ('cytokine', 1),
  ('defense', 1),
  ('degenerative', 1),
  ('development', 1),
  ('diabetes', 1),
  ('diagnosis', 1),
  ('different', 1),
  ('disease', 3),
  ('disorder', 1),
  ('emerge', 2),
  ('emphasis', 1),
  ('especially', 1),
  ('exogenous', 1),
  ('expression', 1),
  ('field', 1),
  ('finding', 1),
  ('fully', 1),
  ('future', 1),
  ('gene', 1),
  ('highlight', 1),
  ('homeostatic', 1),
  ('imbalance', 1),
  ('important', 1),
  ('include', 1),
  ('inflammation', 6),
  ('inflammatory', 2),
  ('inhibitor', 1),
  ('intercellular', 1),
  ('intergenic', 1),
  ('involve', 1),
  ('involvement', 1),
  ('know', 1),

In [51]:
###Loads of room for customization at this step. Read carefully and adjust accordingly.###

In [52]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=14, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [53]:
# Print the Keyword in the topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.033*"dna" + 0.030*"methylation" + 0.023*"silence" + 0.018*"protein" + '
  '0.016*"bind" + 0.013*"methylate" + 0.013*"histone" + 0.011*"gene" + '
  '0.010*"epigenetic" + 0.010*"promoter"'),
 (1,
  '0.020*"epigenetic" + 0.020*"change" + 0.020*"genetic" + 0.017*"effect" + '
  '0.012*"population" + 0.011*"system" + 0.011*"evolutionary" + '
  '0.010*"mechanism" + 0.010*"phenotypic" + 0.010*"developmental"'),
 (2,
  '0.000*"recombination" + 0.000*"dna" + 0.000*"epigenetic" + 0.000*"rate" + '
  '0.000*"cell" + 0.000*"centromere" + 0.000*"cenp" + 0.000*"te" + '
  '0.000*"gene" + 0.000*"genome"'),
 (3,
  '0.026*"factor" + 0.022*"signal" + 0.021*"state" + 0.016*"reprogramme" + '
  '0.014*"reactivity" + 0.012*"carry" + 0.011*"integrate" + '
  '0.009*"pluripotent_stem" + 0.009*"brain" + 0.009*"choline"'),
 (4,
  '0.037*"information" + 0.018*"variation" + 0.016*"structure" + 0.015*"flow" '
  '+ 0.014*"rate" + 0.014*"basic" + 0.013*"recombination" + '
  '0.012*"development" + 0.011*"heart"

In [54]:
###Model perplexity and topic coherence provide a convenient measure to judge how good a given topic model is. 
###In my experience, topic coherence score, in particular, has been more helpful.

###Need to learn more about these values.

In [55]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -8.88467830461454

Coherence Score:  0.3942909497456139


In [56]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

C:\Users\brady\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:232: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8     -0.241414 -0.036580       1        1  30.069869
12    -0.234360 -0.014273       2        1  23.437860
1     -0.152649  0.231189       3        1  12.574124
6     -0.096960 -0.110752       4        1   8.139507
0     -0.100815 -0.155665       5        1   6.164419
7      0.030710  0.159465       6        1   4.350521
11    -0.012421 -0.067798       7        1   3.695348
4      0.042884  0.127884       8        1   3.692290
10     0.067389 -0.058508       9        1   2.465251
13     0.116052 -0.011733      10        1   2.001840
5      0.154626 -0.005516      11        1   1.180761
9      0.149597 -0.024902      12        1   1.136678
3      0.149713 -0.021300      13        1   1.074613
2      0.127648 -0.011509      14        1   0.016920, topic_info=             Term        Freq       Total Category  logprob  loglift
11           cell  486.000000  486.000000  Default  30.0000  30.0000
263   methylation  269.000000  269.000000  Default  29.0000  29.0000
116           dna  262.000000  262.000000  Default  28.0000  28.0000
37           gene  408.000000  408.000000  Default  27.0000  27.0000
561        factor   73.000000   73.000000  Default  26.0000  26.0000
...           ...         ...         ...      ...      ...      ...
1099       report    0.001091   30.758932  Topic14  -8.3509  -1.5623
66        pattern    0.001091   58.015671  Topic14  -8.3509  -2.1968
164         study    0.001091  126.663892  Topic14  -8.3509  -2.9776
32     expression    0.001091  209.001444  Topic14  -8.3509  -3.4784
1360   blastocyst    0.001091    7.239770  Topic14  -8.3509  -0.1157

[787 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
293       1  0.923026      ability
293       7  0.046151      ability
1237      1  0.911715  abnormality
1376      8  0.384651    accompany
1376     11  0.576977    accompany
...     ...       ...          ...
2455      4  0.925084         zinc
1105      8  0.495327       zygote
1105     11  0.495327       zygote
1394      8  0.725116      zygotic
1394     11  0.181279      zygotic

[1331 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 13, 2, 7, 1, 8, 12, 5, 11, 14, 6, 10, 4, 3])

In [ ]:
###rename file.html before running command. Search computer and it should be in the user directory.
p = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
pyLDAvis.save_html(p, 'file.html')